In [1]:

from tensorflow import keras
from tensorflow.keras import layers
 
'''Defining the swish -activation function'''

from tensorflow.keras.backend import sigmoid
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.layers import Activation


class Swish(Activation):
    
    def __init__(self, activation, **kwargs):
        super(Swish, self).__init__(activation, **kwargs)
        self.__name__ = 'swish'


def swish(x, beta = 1):
  
    return (x * sigmoid(beta * x))

get_custom_objects().clear()
get_custom_objects().update({'swish': Swish(swish)})    

class model_brain_inception_only_vin_1:

    def __init__(self, channels=17,tower_min_max_only=False,activation='relu'):
        '''Initialization
        
        activation can be 'selu','swish','nishy_vin1'
        '''
        self.activation = activation
        self.channels = channels
        self.square_height=200
        self.square_width=200
        self.tower_min_max_only = tower_min_max_only

    def layer_1_reducer(self,lay_1_all_output,d_lay_1_to_2,m_p=4):
        
        layer_1_pool = layers.MaxPooling2D(pool_size=(m_p, m_p))(lay_1_all_output)
        #to reduce the depth representation
        incept_1_to_3=layers.Conv2D(d_lay_1_to_2, (1,1),strides=(1,1),padding='same',activation=self.activation)(layer_1_pool)
        incept_1_to_5=layers.Conv2D(d_lay_1_to_2, (1,1),strides=(1,1),padding='same',activation=self.activation)(layer_1_pool)
        return incept_1_to_3,incept_1_to_5,layer_1_pool

    def layer_2_reducer(self,lay_2_all_output,d_lay_1_to_2,m_p=3):
        
        layer_2_pool = layers.MaxPooling2D(pool_size=(m_p, m_p))(lay_2_all_output)       
        #to reduce the depth representation
        incept_1_to_3=layers.Conv2D(d_lay_1_to_2, (1,1),strides=(1,1),padding='same',activation=self.activation)(layer_2_pool)
        incept_1_to_5=layers.Conv2D(d_lay_1_to_2, (1,1),strides=(1,1),padding='same',activation=self.activation)(layer_2_pool)
        return incept_1_to_3,incept_1_to_5,layer_2_pool

    def layer_3_reducer(self,lay_3_all_output,d_lay_3_to_4,m_p=2):
        
        layer_3_pool = layers.MaxPooling2D(pool_size=(m_p, m_p))(lay_3_all_output)
        #to reduce the depth representation
        incept_1_to_3=layers.Conv2D(d_lay_3_to_4, (1,1),strides=(1,1),padding='same',activation=self.activation)(layer_3_pool)
        incept_1_to_5=layers.Conv2D(d_lay_3_to_4, (1,1),strides=(1,1),padding='same',activation=self.activation)(layer_3_pool)
        return incept_1_to_3,incept_1_to_5,layer_3_pool

    def layer_4_final(self,lay_4_all_output,d_lay_3_to_4,m_p=2):
        layer_4_pool = layers.MaxPooling2D(pool_size=(m_p, m_p))(lay_4_all_output)
        incept_1_to_final =layers.Conv2D(d_lay_3_to_4, (1,1),strides=(1,1),padding='same',activation=self.activation)(layer_4_pool)
        return incept_1_to_final

    def model_maker(self,f_inc=1,f_d=1):

        d1=128*f_inc
        d3=64*f_inc
        d5=32*f_inc
        d_max=32*f_inc  
        d_lay_1_to_2 = 32*f_d
        d_lay_3_to_4 = 64*f_d
        
        m_p_l1=4#maxpool layer 1 size
        m_p_l2=3
        m_p_l3=2
        m_p_l4=2
        
        inputs = keras.Input(shape=(self.square_height, self.square_width, self.channels))
        
        inp1 = keras.Input(shape=(self.square_height, self.square_width,  self.channels))
        inp2 = keras.Input(shape=(self.square_height, self.square_width,  self.channels))
        inp3 = keras.Input(shape=(self.square_height, self.square_width,  self.channels))    
         
        lay_1_incept_1 = layers.Conv2D(d1, (1,1),strides=(1,1),padding='same',activation=self.activation)(inputs)
        lay_1_incept_3 = layers.Conv2D(d3, (3,3),strides=(1,1),padding='same',activation=self.activation)(inputs)
        lay_1_incept_5 = layers.Conv2D(d5, (5,5),strides=(1,1),padding='same',activation=self.activation)(inputs)
        lay_1_incept_max_pool= layers.MaxPooling2D(pool_size=(3, 3),strides=(1,1),padding='same')(inputs)
        lay_1_incept_max_pool_depth=layers.Conv2D(d_max, (1,1),strides=(1,1),padding='same',activation='relu')(lay_1_incept_max_pool)
        
        lay_1_all_output = layers.concatenate([lay_1_incept_1, lay_1_incept_3,lay_1_incept_5,lay_1_incept_max_pool_depth], axis=3)
        '''layer 1 general network'''    
        layer_1_INCEPT_Net = keras.models.Model(inputs, lay_1_all_output, name='layer_1_INCEPT')     
        
        '''Applying layer_1 in projections'''
        #layer1 output of projection 1
        inp_1_lay_1_all_output = layer_1_INCEPT_Net(inp1) 
        inp_2_lay_1_all_output = layer_1_INCEPT_Net(inp2) 
        inp_3_lay_1_all_output = layer_1_INCEPT_Net(inp3) 

        lay_1_inp1_incept_1_to_3,lay_1_inp1_incept_1_to_5,inp1_lay_1_pool = self.layer_1_reducer(inp_1_lay_1_all_output,d_lay_1_to_2,m_p=m_p_l1)
        lay_1_inp2_incept_1_to_3,lay_1_inp2_incept_1_to_5,inp2_lay_1_pool = self.layer_1_reducer(inp_2_lay_1_all_output,d_lay_1_to_2,m_p=m_p_l1)
        lay_1_inp3_incept_1_to_3,lay_1_inp3_incept_1_to_5,inp3_lay_1_pool = self.layer_1_reducer(inp_3_lay_1_all_output,d_lay_1_to_2,m_p=m_p_l1)
        '''place the size of NN layer_2 inputs'''
        #since the stride is 1 ans same padding these equation works
        lay_1_height=int(self.square_height/m_p_l1)
        lay_1_width = int(self.square_width/m_p_l1)
        
        
        lay_1_inp_incept_1_to_3 = keras.Input(shape=(lay_1_height,lay_1_width, d_lay_1_to_2))
        lay_1_inp_incept_1_to_5 = keras.Input(shape=(lay_1_height,lay_1_width, d_lay_1_to_2))
        inp_lay_1_pool = keras.Input(shape=(lay_1_height,lay_1_width,d1+d3+d5+d_max))
        
        lay_2_incept_1 = layers.Conv2D(d1, (1,1),strides=(1,1),padding='same',activation=self.activation)(inp_lay_1_pool)
        lay_2_incept_3 = layers.Conv2D(d3, (3,3),strides=(1,1),padding='same',activation=self.activation)(lay_1_inp_incept_1_to_3)
        lay_2_incept_5 = layers.Conv2D(d5, (5,5),strides=(1,1),padding='same',activation=self.activation)(lay_1_inp_incept_1_to_5)
        lay_2_incept_max_pool= layers.MaxPooling2D(pool_size=(3, 3),strides=(1,1),padding='same')(inp_lay_1_pool)
        lay_2_incept_max_pool_depth=layers.Conv2D(d_max, (1,1),strides=(1,1),padding='same',activation='relu')(lay_2_incept_max_pool)
        
        lay_2_all_output = layers.concatenate([lay_2_incept_1, lay_2_incept_3,lay_2_incept_5,lay_2_incept_max_pool_depth], axis=3)
        '''layer 2 general network'''    
        layer_2_INCEPT_Net = keras.models.Model(inputs=[lay_1_inp_incept_1_to_3,lay_1_inp_incept_1_to_5,inp_lay_1_pool],outputs= lay_2_all_output, name='layer_2_INCEPT')     

        '''Applying layer_2 in projections'''
        #layer1 output of projection 1
        inp_1_lay_2_all_output = layer_2_INCEPT_Net([lay_1_inp1_incept_1_to_3,lay_1_inp1_incept_1_to_5,inp1_lay_1_pool]) 
        inp_2_lay_2_all_output = layer_2_INCEPT_Net([lay_1_inp2_incept_1_to_3,lay_1_inp2_incept_1_to_5,inp2_lay_1_pool]) 
        inp_3_lay_2_all_output = layer_2_INCEPT_Net([lay_1_inp3_incept_1_to_3,lay_1_inp3_incept_1_to_5,inp3_lay_1_pool]) 

        #layer2 output of projection 1
        lay_2_inp1_incept_1_to_3,lay_2_inp1_incept_1_to_5,inp1_lay_2_pool = self.layer_2_reducer(inp_1_lay_2_all_output,d_lay_1_to_2,m_p=m_p_l2)
        lay_2_inp2_incept_1_to_3,lay_2_inp2_incept_1_to_5,inp2_lay_2_pool = self.layer_2_reducer(inp_2_lay_2_all_output,d_lay_1_to_2,m_p=m_p_l2)
        lay_2_inp3_incept_1_to_3,lay_2_inp3_incept_1_to_5,inp3_lay_2_pool = self.layer_2_reducer(inp_3_lay_2_all_output,d_lay_1_to_2,m_p=m_p_l2)

        '''place the size of NN layer_3 inputs'''
        #since the stride is 1 ans same padding these equation works
        lay_2_height=int(lay_1_height/m_p_l2)
        lay_2_width = int(lay_1_height/m_p_l2)
        
        lay_2_inp_incept_1_to_3 = keras.Input(shape=(lay_2_height,lay_1_width, d_lay_1_to_2))
        lay_2_inp_incept_1_to_5 = keras.Input(shape=(lay_2_width,lay_1_width, d_lay_1_to_2))      
        inp_lay_2_pool = keras.Input(shape=(lay_2_height,lay_1_width,d1+d3+d5+d_max))

        lay_3_incept_1 = layers.Conv2D(d1, (1,1),strides=(1,1),padding='same',activation=self.activation)(inp_lay_2_pool)
        lay_3_incept_3 = layers.Conv2D(d3, (3,3),strides=(1,1),padding='same',activation=self.activation)(lay_2_inp_incept_1_to_3)
        lay_3_incept_5 = layers.Conv2D(d5, (5,5),strides=(1,1),padding='same',activation=self.activation)(lay_2_inp_incept_1_to_5)
        lay_3_incept_max_pool= layers.MaxPooling2D(pool_size=(3, 3),strides=(1,1),padding='same')(inp_lay_2_pool)
        lay_3_incept_max_pool_depth=layers.Conv2D(d_max, (1,1),strides=(1,1),padding='same',activation='relu')(lay_3_incept_max_pool)
        
        lay_3_all_output = layers.concatenate([lay_3_incept_1, lay_3_incept_3,lay_3_incept_5,lay_3_incept_max_pool_depth], axis=3)
        '''layer 3 general network'''    
        layer_3_INCEPT_Net = keras.models.Model(inputs=[lay_2_inp_incept_1_to_3,lay_2_inp_incept_1_to_5,inp_lay_2_pool],outputs= lay_3_all_output, name='layer_3_INCEPT')     

        '''Applying layer_3 in projections'''
        #layer3 output of projection 1
        inp_1_lay_3_all_output = layer_3_INCEPT_Net([lay_2_inp1_incept_1_to_3,lay_2_inp1_incept_1_to_5,inp1_lay_2_pool]) 
        inp_2_lay_3_all_output = layer_3_INCEPT_Net([lay_2_inp2_incept_1_to_3,lay_2_inp2_incept_1_to_5,inp2_lay_2_pool]) 
        inp_3_lay_3_all_output = layer_3_INCEPT_Net([lay_2_inp3_incept_1_to_3,lay_2_inp3_incept_1_to_5,inp3_lay_2_pool]) 
      
        #layer3 output of projection 1
        lay_3_inp1_incept_1_to_3,lay_3_inp1_incept_1_to_5,inp1_lay_3_pool = self.layer_3_reducer(inp_1_lay_3_all_output,d_lay_3_to_4,m_p=m_p_l3)
        lay_3_inp2_incept_1_to_3,lay_3_inp2_incept_1_to_5,inp2_lay_3_pool = self.layer_3_reducer(inp_2_lay_3_all_output,d_lay_3_to_4,m_p=m_p_l3)
        lay_3_inp3_incept_1_to_3,lay_3_inp3_incept_1_to_5,inp3_lay_3_pool = self.layer_3_reducer(inp_3_lay_3_all_output,d_lay_3_to_4,m_p=m_p_l3)

        '''place the size of NN layer_4 inputs'''
        #since the stride is 1 ans same padding these equation works
        lay_3_height=int(lay_2_height/m_p_l3)
        lay_3_width = int(lay_2_height/m_p_l3)
        
        lay_3_inp_incept_1_to_3 = keras.Input(shape=(lay_3_height, lay_3_width,d_lay_3_to_4))
        lay_3_inp_incept_1_to_5 = keras.Input(shape=(lay_3_height, lay_3_width,d_lay_3_to_4))
        inp_lay_3_pool = keras.Input(shape=(lay_3_height, lay_3_width,d1+d3+d5+d_max))       
        
        lay_4_incept_1 = layers.Conv2D(d1, (1,1),strides=(1,1),padding='same',activation=self.activation)(inp_lay_3_pool)
        lay_4_incept_3 = layers.Conv2D(d3, (3,3),strides=(1,1),padding='same',activation=self.activation)(lay_3_inp_incept_1_to_3)
        lay_4_incept_5 = layers.Conv2D(d5, (5,5),strides=(1,1),padding='same',activation=self.activation)(lay_3_inp_incept_1_to_5)
        lay_4_incept_max_pool= layers.MaxPooling2D(pool_size=(3, 3),strides=(1,1),padding='same')(inp_lay_3_pool)
        lay_4_incept_max_pool_depth=layers.Conv2D(d_max, (1,1),strides=(1,1),padding='same',activation='relu')(lay_4_incept_max_pool)
        
        lay_4_all_output = layers.concatenate([lay_4_incept_1, lay_4_incept_3,lay_4_incept_5,lay_4_incept_max_pool_depth], axis=3)
    
        '''layer 3 general network'''    
        layer_4_INCEPT_Net = keras.models.Model(inputs=[lay_3_inp_incept_1_to_3,lay_3_inp_incept_1_to_5,inp_lay_3_pool], outputs=lay_4_all_output, name='layer_4_INCEPT')     
        '''Applying layer_4 in projections'''
        inp_1_lay_4_all_output = layer_4_INCEPT_Net([lay_3_inp1_incept_1_to_3,lay_3_inp1_incept_1_to_5,inp1_lay_3_pool]) 
        inp_2_lay_4_all_output = layer_4_INCEPT_Net([lay_3_inp2_incept_1_to_3,lay_3_inp2_incept_1_to_5,inp2_lay_3_pool]) 
        inp_3_lay_4_all_output = layer_4_INCEPT_Net([lay_3_inp3_incept_1_to_3,lay_3_inp3_incept_1_to_5,inp3_lay_3_pool]) 

        
        #parallel = keras.models.Model(inputs,all_output, name='parallel') 
        tower_1 = self.layer_4_final(inp_1_lay_4_all_output,d_lay_3_to_4,m_p=m_p_l4)
        tower_2 = self.layer_4_final(inp_2_lay_4_all_output,d_lay_3_to_4,m_p=m_p_l4)
        tower_3 = self.layer_4_final(inp_3_lay_4_all_output,d_lay_3_to_4,m_p=m_p_l4)

        
        if self.tower_min_max_only:
            tower_max = layers.maximum([tower_1,tower_2,tower_3])
            tower_min=layers.minimum([tower_max, tower_2,tower_3])
            #tower_average=layers.average([tower_1, tower_2,tower_3])
            #tower_mul= layers.Multiply([tower_1, tower_2,tower_3])#since multiplication is expensive
            merged = layers.concatenate([tower_max, tower_min], axis=1)
            model_name = ''.join(['MIN_MAX_model_brain_inception_only_mentor_vin_1_act_',self.activation,'_f_inc_',str(f_inc),'_f_d_',str(f_d),'.h5'])
        
        else:
            merged = layers.concatenate([tower_1, tower_2, tower_3], axis=1)
            model_name = ''.join(['model_brain_inception_only_mentor_vin_1_act_',self.activation,'_f_inc_',str(f_inc),'_f_d_',str(f_d),'.h5'])
        
        merged = layers.Flatten()(merged)
        all_co = layers.Dense(100, activation=self.activation)(merged)
        all_co = layers.Dropout(0.5)(all_co)
        all_co = layers.Dense(50, activation=self.activation)(all_co)
        all_co= layers.Dropout(0.5)(all_co)
        outputs = layers.Dense(3, activation='softmax')(all_co)
        
        model = keras.models.Model(inputs=[inp1, inp2,inp3], outputs=outputs, name=model_name)
        
        return model,model_name    


class model_brain_incept_RESIDUAL_only_vin_1(model_brain_inception_only_vin_1):

    def __init__(self, channels=17,tower_min_max_only=False,activation='relu'):
        '''Initialization
        
        activation can be 'selu','swish','nishy_vin1'
        '''
        self.activation = activation
        self.channels = channels
        self.square_height=200
        self.square_width=200
        self.tower_min_max_only = tower_min_max_only


    def model_maker(self,f_inc=1,f_d=1):

        d1=128*f_inc
        d3=64*f_inc
        d5=32*f_inc
        d_max=32*f_inc  
        d_lay_1_to_2 = 32*f_d
        d_lay_3_to_4 = 64*f_d
        
        m_p_l1=4#maxpool layer 1 size
        m_p_l2=3
        m_p_l3=2
        m_p_l4=2
        
        inputs = keras.Input(shape=(self.square_height, self.square_width, self.channels))
        
        inp1 = keras.Input(shape=(self.square_height, self.square_width,  self.channels))
        inp2 = keras.Input(shape=(self.square_height, self.square_width,  self.channels))
        inp3 = keras.Input(shape=(self.square_height, self.square_width,  self.channels))    
         
        lay_1_incept_1 = layers.Conv2D(d1, (1,1),strides=(1,1),padding='same',activation=self.activation)(inputs)
        lay_1_incept_3 = layers.Conv2D(d3, (3,3),strides=(1,1),padding='same',activation=self.activation)(inputs)
        lay_1_incept_5 = layers.Conv2D(d5, (5,5),strides=(1,1),padding='same',activation=self.activation)(inputs)
        lay_1_incept_max_pool= layers.MaxPooling2D(pool_size=(3, 3),strides=(1,1),padding='same')(inputs)
        lay_1_incept_max_pool_depth=layers.Conv2D(d_max, (1,1),strides=(1,1),padding='same',activation='relu')(lay_1_incept_max_pool)
        
        lay_1_all_output = layers.concatenate([lay_1_incept_1, lay_1_incept_3,lay_1_incept_5,lay_1_incept_max_pool_depth], axis=3)
        '''layer 1 general network'''    
        layer_1_INCEPT_Net = keras.models.Model(inputs, lay_1_all_output, name='layer_1_INCEPT')     
        
        '''Applying layer_1 in projections'''
        #layer1 output of projection 1
        inp_1_lay_1_all_output = layer_1_INCEPT_Net(inp1) 
        inp_2_lay_1_all_output = layer_1_INCEPT_Net(inp2) 
        inp_3_lay_1_all_output = layer_1_INCEPT_Net(inp3) 

        lay_1_inp1_incept_1_to_3,lay_1_inp1_incept_1_to_5,inp1_lay_1_pool = self.layer_1_reducer(inp_1_lay_1_all_output,d_lay_1_to_2,m_p=m_p_l1)
        lay_1_inp2_incept_1_to_3,lay_1_inp2_incept_1_to_5,inp2_lay_1_pool = self.layer_1_reducer(inp_2_lay_1_all_output,d_lay_1_to_2,m_p=m_p_l1)
        lay_1_inp3_incept_1_to_3,lay_1_inp3_incept_1_to_5,inp3_lay_1_pool = self.layer_1_reducer(inp_3_lay_1_all_output,d_lay_1_to_2,m_p=m_p_l1)
        '''place the size of NN layer_2 inputs'''
        #since the stride is 1 ans same padding these equation works
        lay_1_height=int(self.square_height/m_p_l1)
        lay_1_width = int(self.square_width/m_p_l1)
        
        
        lay_1_inp_incept_1_to_3 = keras.Input(shape=(lay_1_height,lay_1_width, d_lay_1_to_2))
        lay_1_inp_incept_1_to_5 = keras.Input(shape=(lay_1_height,lay_1_width, d_lay_1_to_2))
        inp_lay_1_pool = keras.Input(shape=(lay_1_height,lay_1_width,d1+d3+d5+d_max))
        
        lay_2_incept_1 = layers.Conv2D(d1, (1,1),strides=(1,1),padding='same',activation=self.activation)(inp_lay_1_pool)
        lay_2_incept_3 = layers.Conv2D(d3, (3,3),strides=(1,1),padding='same',activation=self.activation)(lay_1_inp_incept_1_to_3)
        lay_2_incept_5 = layers.Conv2D(d5, (5,5),strides=(1,1),padding='same',activation=self.activation)(lay_1_inp_incept_1_to_5)
        lay_2_incept_max_pool= layers.MaxPooling2D(pool_size=(3, 3),strides=(1,1),padding='same')(inp_lay_1_pool)
        lay_2_incept_max_pool_depth=layers.Conv2D(d_max, (1,1),strides=(1,1),padding='same',activation='relu')(lay_2_incept_max_pool)
        

        lay_2_all_output = layers.concatenate([lay_2_incept_1, lay_2_incept_3,lay_2_incept_5,lay_2_incept_max_pool_depth], axis=3)
        '''layer 2 general network'''    
        layer_2_INCEPT_Net = keras.models.Model(inputs=[lay_1_inp_incept_1_to_3,lay_1_inp_incept_1_to_5,inp_lay_1_pool],outputs= lay_2_all_output, name='layer_2_INCEPT')     

        '''Applying layer_2 in projections'''
        #layer1 output of projection 1
        inp_1_lay_2_incept_all_output = layer_2_INCEPT_Net([lay_1_inp1_incept_1_to_3,lay_1_inp1_incept_1_to_5,inp1_lay_1_pool]) 
        inp_2_lay_2_incept_all_output = layer_2_INCEPT_Net([lay_1_inp2_incept_1_to_3,lay_1_inp2_incept_1_to_5,inp2_lay_1_pool]) 
        inp_3_lay_2_incept_all_output = layer_2_INCEPT_Net([lay_1_inp3_incept_1_to_3,lay_1_inp3_incept_1_to_5,inp3_lay_1_pool]) 
        #make residUAL NETWORK
        inp_1_lay_2_all_output= layers.add([inp1_lay_1_pool, inp_1_lay_2_incept_all_output])
        inp_2_lay_2_all_output= layers.add([inp2_lay_1_pool, inp_2_lay_2_incept_all_output])
        inp_3_lay_2_all_output= layers.add([inp3_lay_1_pool, inp_3_lay_2_incept_all_output])

        #layer2 output of projection 1
        lay_2_inp1_incept_1_to_3,lay_2_inp1_incept_1_to_5,inp1_lay_2_pool = self.layer_2_reducer(inp_1_lay_2_all_output,d_lay_1_to_2,m_p=m_p_l2)
        lay_2_inp2_incept_1_to_3,lay_2_inp2_incept_1_to_5,inp2_lay_2_pool = self.layer_2_reducer(inp_2_lay_2_all_output,d_lay_1_to_2,m_p=m_p_l2)
        lay_2_inp3_incept_1_to_3,lay_2_inp3_incept_1_to_5,inp3_lay_2_pool = self.layer_2_reducer(inp_3_lay_2_all_output,d_lay_1_to_2,m_p=m_p_l2)

        '''place the size of NN layer_3 inputs'''
        #since the stride is 1 ans same padding these equation works
        lay_2_height=int(lay_1_height/m_p_l2)
        lay_2_width = int(lay_1_height/m_p_l2)
        
        lay_2_inp_incept_1_to_3 = keras.Input(shape=(lay_2_height,lay_1_width, d_lay_1_to_2))
        lay_2_inp_incept_1_to_5 = keras.Input(shape=(lay_2_width,lay_1_width, d_lay_1_to_2))      
        inp_lay_2_pool = keras.Input(shape=(lay_2_height,lay_1_width,d1+d3+d5+d_max))

        lay_3_incept_1 = layers.Conv2D(d1, (1,1),strides=(1,1),padding='same',activation=self.activation)(inp_lay_2_pool)
        lay_3_incept_3 = layers.Conv2D(d3, (3,3),strides=(1,1),padding='same',activation=self.activation)(lay_2_inp_incept_1_to_3)
        lay_3_incept_5 = layers.Conv2D(d5, (5,5),strides=(1,1),padding='same',activation=self.activation)(lay_2_inp_incept_1_to_5)
        lay_3_incept_max_pool= layers.MaxPooling2D(pool_size=(3, 3),strides=(1,1),padding='same')(inp_lay_2_pool)
        lay_3_incept_max_pool_depth=layers.Conv2D(d_max, (1,1),strides=(1,1),padding='same',activation='relu')(lay_3_incept_max_pool)
        
        lay_3_all_output = layers.concatenate([lay_3_incept_1, lay_3_incept_3,lay_3_incept_5,lay_3_incept_max_pool_depth], axis=3)
        '''layer 3 general network'''    
        layer_3_INCEPT_Net = keras.models.Model(inputs=[lay_2_inp_incept_1_to_3,lay_2_inp_incept_1_to_5,inp_lay_2_pool],outputs= lay_3_all_output, name='layer_3_INCEPT')     

        '''Applying layer_3 in projections'''
        #layer3 output of projection 1
        inp_1_lay_3_incept_all_output = layer_3_INCEPT_Net([lay_2_inp1_incept_1_to_3,lay_2_inp1_incept_1_to_5,inp1_lay_2_pool]) 
        inp_2_lay_3_incept_all_output = layer_3_INCEPT_Net([lay_2_inp2_incept_1_to_3,lay_2_inp2_incept_1_to_5,inp2_lay_2_pool]) 
        inp_3_lay_3_incept_all_output = layer_3_INCEPT_Net([lay_2_inp3_incept_1_to_3,lay_2_inp3_incept_1_to_5,inp3_lay_2_pool]) 
      
        #make residUAL NETWORK
        inp_1_lay_3_all_output = layers.add([inp1_lay_2_pool, inp_1_lay_3_incept_all_output])
        inp_2_lay_3_all_output = layers.add([inp2_lay_2_pool, inp_2_lay_3_incept_all_output])
        inp_3_lay_3_all_output = layers.add([inp3_lay_2_pool, inp_3_lay_3_incept_all_output])

        #layer3 output of projection 1
        lay_3_inp1_incept_1_to_3,lay_3_inp1_incept_1_to_5,inp1_lay_3_pool = self.layer_3_reducer(inp_1_lay_3_all_output,d_lay_3_to_4,m_p=m_p_l3)
        lay_3_inp2_incept_1_to_3,lay_3_inp2_incept_1_to_5,inp2_lay_3_pool = self.layer_3_reducer(inp_2_lay_3_all_output,d_lay_3_to_4,m_p=m_p_l3)
        lay_3_inp3_incept_1_to_3,lay_3_inp3_incept_1_to_5,inp3_lay_3_pool = self.layer_3_reducer(inp_3_lay_3_all_output,d_lay_3_to_4,m_p=m_p_l3)

        '''place the size of NN layer_4 inputs'''
        #since the stride is 1 ans same padding these equation works
        lay_3_height=int(lay_2_height/m_p_l3)
        lay_3_width = int(lay_2_height/m_p_l3)
        
        lay_3_inp_incept_1_to_3 = keras.Input(shape=(lay_3_height, lay_3_width,d_lay_3_to_4))
        lay_3_inp_incept_1_to_5 = keras.Input(shape=(lay_3_height, lay_3_width,d_lay_3_to_4))
        inp_lay_3_pool = keras.Input(shape=(lay_3_height, lay_3_width,d1+d3+d5+d_max))       
        
        lay_4_incept_1 = layers.Conv2D(d1, (1,1),strides=(1,1),padding='same',activation=self.activation)(inp_lay_3_pool)
        lay_4_incept_3 = layers.Conv2D(d3, (3,3),strides=(1,1),padding='same',activation=self.activation)(lay_3_inp_incept_1_to_3)
        lay_4_incept_5 = layers.Conv2D(d5, (5,5),strides=(1,1),padding='same',activation=self.activation)(lay_3_inp_incept_1_to_5)
        lay_4_incept_max_pool= layers.MaxPooling2D(pool_size=(3, 3),strides=(1,1),padding='same')(inp_lay_3_pool)
        lay_4_incept_max_pool_depth=layers.Conv2D(d_max, (1,1),strides=(1,1),padding='same',activation='relu')(lay_4_incept_max_pool)
        
        lay_4_all_output = layers.concatenate([lay_4_incept_1, lay_4_incept_3,lay_4_incept_5,lay_4_incept_max_pool_depth], axis=3)
    
        '''layer 3 general network'''    
        layer_4_INCEPT_Net = keras.models.Model(inputs=[lay_3_inp_incept_1_to_3,lay_3_inp_incept_1_to_5,inp_lay_3_pool], outputs=lay_4_all_output, name='layer_4_INCEPT')     
        '''Applying layer_4 in projections'''
        inp_1_lay_4_incept_all_output = layer_4_INCEPT_Net([lay_3_inp1_incept_1_to_3,lay_3_inp1_incept_1_to_5,inp1_lay_3_pool]) 
        inp_2_lay_4_incept_all_output = layer_4_INCEPT_Net([lay_3_inp2_incept_1_to_3,lay_3_inp2_incept_1_to_5,inp2_lay_3_pool]) 
        inp_3_lay_4_incept_all_output = layer_4_INCEPT_Net([lay_3_inp3_incept_1_to_3,lay_3_inp3_incept_1_to_5,inp3_lay_3_pool]) 
        #make residUAL NETWORK
        inp_1_lay_4_all_output = layers.add([inp1_lay_3_pool, inp_1_lay_4_incept_all_output])
        inp_2_lay_4_all_output = layers.add([inp2_lay_3_pool, inp_2_lay_4_incept_all_output])
        inp_3_lay_4_all_output = layers.add([inp3_lay_3_pool, inp_3_lay_4_incept_all_output])

        
        #parallel = keras.models.Model(inputs,all_output, name='parallel') 
        tower_1 = self.layer_4_final(inp_1_lay_4_all_output,d_lay_3_to_4,m_p=m_p_l4)
        tower_2 = self.layer_4_final(inp_2_lay_4_all_output,d_lay_3_to_4,m_p=m_p_l4)
        tower_3 = self.layer_4_final(inp_3_lay_4_all_output,d_lay_3_to_4,m_p=m_p_l4)

        
        if self.tower_min_max_only:
            tower_max = layers.maximum([tower_1,tower_2,tower_3])
            tower_min=layers.minimum([tower_max, tower_2,tower_3])
            #tower_average=layers.average([tower_1, tower_2,tower_3])
            #tower_mul= layers.Multiply([tower_1, tower_2,tower_3])#since multiplication is expensive
            merged = layers.concatenate([tower_max, tower_min], axis=1)
            model_name = ''.join(['MIN_MAX_model_brain_incept_RESIDUAL_only_vin_1_act_',self.activation,'_f_inc_',str(f_inc),'_f_d_',str(f_d),'.h5'])
        
        else:
            merged = layers.concatenate([tower_1, tower_2, tower_3], axis=1)
            model_name = ''.join(['model_brain_incept_RESIDUAL_only_vin_1_act_',self.activation,'_f_inc_',str(f_inc),'_f_d_',str(f_d),'.h5'])
        
        merged = layers.Flatten()(merged)
        all_co = layers.Dense(100, activation=self.activation)(merged)
        all_co = layers.Dropout(0.5)(all_co)
        all_co = layers.Dense(50, activation=self.activation)(all_co)
        all_co= layers.Dropout(0.5)(all_co)
        outputs = layers.Dense(3, activation='softmax')(all_co)
        
        model = keras.models.Model(inputs=[inp1, inp2,inp3], outputs=outputs, name=model_name)
        
        return model,model_name    
    
    
class model_inception_only_mentor_vin_1:

    def __init__(self, channels=17,tower_min_max_only=False,activation='relu'):
        '''Initialization
        
        activation can be 'selu','swish','nishy_vin1'
        '''
        self.activation = activation
        self.channels = channels
        self.tower_min_max_only = tower_min_max_only

    def parallel(self,inputs,f_inc,f_d):

        d1=128*f_inc
        d3=64*f_inc
        d5=32*f_inc
        d_max=32*f_inc  
        d_lay_1_to_2 = 32*f_d
        d_lay_3_to_4 = 64*f_d
         
        incept_1 = layers.Conv2D(d1, (1,1),strides=(1,1),padding='same',activation=self.activation)(inputs)
        incept_3 = layers.Conv2D(d3, (3,3),strides=(1,1),padding='same',activation=self.activation)(inputs)
        incept_5 = layers.Conv2D(d5, (5,5),strides=(1,1),padding='same',activation=self.activation)(inputs)
        incept_max_pool= layers.MaxPooling2D(pool_size=(3, 3),strides=(1,1),padding='same')(inputs)
        incept_max_pool_depth=layers.Conv2D(d_max, (1,1),strides=(1,1),padding='same',activation='relu')(incept_max_pool)
        
        all_output = layers.concatenate([incept_1, incept_3,incept_5,incept_max_pool_depth], axis=3)
        layer_1_pool = layers.MaxPooling2D(pool_size=(4, 4))(all_output)
        
        #to reduce the depth representation
        incept_1_to_3=layers.Conv2D(d_lay_1_to_2, (1,1),strides=(1,1),padding='same',activation=self.activation)(layer_1_pool)
        incept_1_to_5=layers.Conv2D(d_lay_1_to_2, (1,1),strides=(1,1),padding='same',activation=self.activation)(layer_1_pool)
        
        incept_1 = layers.Conv2D(d1, (1,1),strides=(1,1),padding='same',activation=self.activation)(layer_1_pool)
        incept_3 = layers.Conv2D(d3, (3,3),strides=(1,1),padding='same',activation=self.activation)(incept_1_to_3)
        incept_5 = layers.Conv2D(d5, (5,5),strides=(1,1),padding='same',activation=self.activation)(incept_1_to_5)
        incept_max_pool= layers.MaxPooling2D(pool_size=(3, 3),strides=(1,1),padding='same')(layer_1_pool)
        incept_max_pool_depth=layers.Conv2D(d_max, (1,1),strides=(1,1),padding='same',activation='relu')(incept_max_pool)
        
        all_output = layers.concatenate([incept_1, incept_3,incept_5,incept_max_pool_depth], axis=3)
        layer_2_pool = layers.MaxPooling2D(pool_size=(3, 3))(all_output)
        
        #to reduce the depth representation
        incept_1_to_3=layers.Conv2D(d_lay_1_to_2, (1,1),strides=(1,1),padding='same',activation=self.activation)(layer_2_pool)
        incept_1_to_5=layers.Conv2D(d_lay_1_to_2, (1,1),strides=(1,1),padding='same',activation=self.activation)(layer_2_pool)
        
        incept_1 = layers.Conv2D(d1, (1,1),strides=(1,1),padding='same',activation=self.activation)(layer_2_pool)
        incept_3 = layers.Conv2D(d3, (3,3),strides=(1,1),padding='same',activation=self.activation)(incept_1_to_3)
        incept_5 = layers.Conv2D(d5, (5,5),strides=(1,1),padding='same',activation=self.activation)(incept_1_to_5)
        incept_max_pool= layers.MaxPooling2D(pool_size=(3, 3),strides=(1,1),padding='same')(layer_2_pool)
        incept_max_pool_depth=layers.Conv2D(d_max, (1,1),strides=(1,1),padding='same',activation='relu')(incept_max_pool)
        
        all_output = layers.concatenate([incept_1, incept_3,incept_5,incept_max_pool_depth], axis=3)
        layer_3_pool = layers.MaxPooling2D(pool_size=(2, 2))(all_output)
        #to reduce the depth representation
        incept_1_to_3=layers.Conv2D(d_lay_3_to_4, (1,1),strides=(1,1),padding='same',activation=self.activation)(layer_3_pool)
        incept_1_to_5=layers.Conv2D(d_lay_3_to_4, (1,1),strides=(1,1),padding='same',activation=self.activation)(layer_3_pool)
        
        incept_1 = layers.Conv2D(d1, (1,1),strides=(1,1),padding='same',activation=self.activation)(layer_3_pool)
        incept_3 = layers.Conv2D(d3, (3,3),strides=(1,1),padding='same',activation=self.activation)(incept_1_to_3)
        incept_5 = layers.Conv2D(d5, (5,5),strides=(1,1),padding='same',activation=self.activation)(incept_1_to_5)
        incept_max_pool= layers.MaxPooling2D(pool_size=(3, 3),strides=(1,1),padding='same')(layer_3_pool)
        incept_max_pool_depth=layers.Conv2D(d_max, (1,1),strides=(1,1),padding='same',activation='relu')(incept_max_pool)
        
        all_output = layers.concatenate([incept_1, incept_3,incept_5,incept_max_pool_depth], axis=3)
        layer_4_pool = layers.MaxPooling2D(pool_size=(2, 2))(all_output)
        #
        incept_1_to_final =layers.Conv2D(d_lay_3_to_4, (1,1),strides=(1,1),padding='same',activation=self.activation)(layer_4_pool)
        return incept_1_to_final
    
    def model_maker(self,f_inc=1,f_d=1):

        inp1 = keras.Input(shape=(200, 200, self.channels))
        inp2 = keras.Input(shape=(200, 200, self.channels))
        inp3 = keras.Input(shape=(200, 200, self.channels))
        
        tower_1 = self.parallel(inp1,f_inc,f_d)
        tower_2 = self.parallel(inp2,f_inc,f_d)
        tower_3 = self.parallel(inp3,f_inc,f_d)
        
        if self.tower_min_max_only:
            tower_max = layers.maximum([tower_1,tower_2,tower_3])
            tower_min=layers.minimum([tower_max, tower_2,tower_3])
            #tower_average=layers.average([tower_1, tower_2,tower_3])
            #tower_mul= layers.Multiply([tower_1, tower_2,tower_3])#since multiplication is expensive
            merged = layers.concatenate([tower_max, tower_min], axis=1)
            model_name = ''.join(['MIN_MAX_model_inception_only_mentor_vin_1_act_',self.activation,'_f_inc_',str(f_inc),'_f_d_',str(f_d),'.h5'])
        
        else:
            merged = layers.concatenate([tower_1, tower_2, tower_3], axis=1)
            model_name = ''.join(['model_inception_only_mentor_vin_1_act_',self.activation,'_f_inc_',str(f_inc),'_f_d_',str(f_d),'.h5'])
        
        merged = layers.Flatten()(merged)
        all_co = layers.Dense(100, activation=self.activation)(merged)
        all_co = layers.Dropout(0.5)(all_co)
        all_co = layers.Dense(50, activation=self.activation)(all_co)
        all_co= layers.Dropout(0.5)(all_co)
        outputs = layers.Dense(3, activation='softmax')(all_co)
        
        model = keras.models.Model(inputs=[inp1, inp2,inp3], outputs=outputs, name=model_name)
        
        return model,model_name    

class model_inception_residual_mentor_vin_1:

    def __init__(self, channels=17,tower_min_max_only=False,activation='relu'):
        '''Initialization
        
        activation can be 'selu','swish','nishy_vin1'
        '''
        self.activation = activation
        self.channels = channels
        self.tower_min_max_only = tower_min_max_only

    def parallel(self,inputs,f_inc,f_d):

        d1=128*f_inc
        d3=64*f_inc
        d5=32*f_inc
        d_max=32*f_inc  
        d_lay_1_to_2 = 32*f_d
        d_lay_3_to_4 = 64*f_d
         
        incept_1 = layers.Conv2D(d1, (1,1),strides=(1,1),padding='same',activation=self.activation)(inputs)
        incept_3 = layers.Conv2D(d3, (3,3),strides=(1,1),padding='same',activation=self.activation)(inputs)
        incept_5 = layers.Conv2D(d5, (5,5),strides=(1,1),padding='same',activation=self.activation)(inputs)
        incept_max_pool= layers.MaxPooling2D(pool_size=(3, 3),strides=(1,1),padding='same')(inputs)
        incept_max_pool_depth=layers.Conv2D(d_max, (1,1),strides=(1,1),padding='same',activation='relu')(incept_max_pool)
        
        all_output = layers.concatenate([incept_1, incept_3,incept_5,incept_max_pool_depth], axis=3)
        layer_1_pool = layers.MaxPooling2D(pool_size=(4, 4))(all_output)
        
        #to reduce the depth representation
        incept_1_to_3=layers.Conv2D(d_lay_1_to_2, (1,1),strides=(1,1),padding='same',activation=self.activation)(layer_1_pool)
        incept_1_to_5=layers.Conv2D(d_lay_1_to_2, (1,1),strides=(1,1),padding='same',activation=self.activation)(layer_1_pool)
        
        incept_1 = layers.Conv2D(d1, (1,1),strides=(1,1),padding='same',activation=self.activation)(layer_1_pool)
        incept_3 = layers.Conv2D(d3, (3,3),strides=(1,1),padding='same',activation=self.activation)(incept_1_to_3)
        incept_5 = layers.Conv2D(d5, (5,5),strides=(1,1),padding='same',activation=self.activation)(incept_1_to_5)
        incept_max_pool= layers.MaxPooling2D(pool_size=(3, 3),strides=(1,1),padding='same')(layer_1_pool)
        incept_max_pool_depth=layers.Conv2D(d_max, (1,1),strides=(1,1),padding='same',activation='relu')(incept_max_pool)
        
        all_output = layers.concatenate([incept_1, incept_3,incept_5,incept_max_pool_depth], axis=3)
        
        layer_1_2_pool=layers.Conv2D(d1+d3+d5+d_max, (1,1),strides=(1,1),padding='same',activation=self.activation)(layer_1_pool)
        layer_21_residual = layers.add([layer_1_2_pool, all_output])
        
        layer_2_pool = layers.MaxPooling2D(pool_size=(3, 3))(layer_21_residual)
        #to reduce the depth representation
        incept_1_to_3=layers.Conv2D(d_lay_1_to_2, (1,1),strides=(1,1),padding='same',activation=self.activation)(layer_2_pool)
        incept_1_to_5=layers.Conv2D(d_lay_1_to_2, (1,1),strides=(1,1),padding='same',activation=self.activation)(layer_2_pool)
        
        incept_1 = layers.Conv2D(d1, (1,1),strides=(1,1),padding='same',activation=self.activation)(layer_2_pool)
        incept_3 = layers.Conv2D(d3, (3,3),strides=(1,1),padding='same',activation=self.activation)(incept_1_to_3)
        incept_5 = layers.Conv2D(d5, (5,5),strides=(1,1),padding='same',activation=self.activation)(incept_1_to_5)
        incept_max_pool= layers.MaxPooling2D(pool_size=(3, 3),strides=(1,1),padding='same')(layer_2_pool)
        incept_max_pool_depth=layers.Conv2D(d_max, (1,1),strides=(1,1),padding='same',activation='relu')(incept_max_pool)
        
        all_output = layers.concatenate([incept_1, incept_3,incept_5,incept_max_pool_depth], axis=3)
        layer_2_3_pool=layers.Conv2D(d1+d3+d5+d_max, (1,1),strides=(1,1),padding='same',activation=self.activation)(layer_2_pool)
        layer_32_residual = layers.add([layer_2_3_pool, all_output])
        
        layer_3_pool = layers.MaxPooling2D(pool_size=(3, 3))(layer_32_residual)
        #to reduce the depth representation
        incept_1_to_3=layers.Conv2D(d_lay_3_to_4, (1,1),strides=(1,1),padding='same',activation=self.activation)(layer_3_pool)
        incept_1_to_5=layers.Conv2D(d_lay_3_to_4, (1,1),strides=(1,1),padding='same',activation=self.activation)(layer_3_pool)
        
        incept_1 = layers.Conv2D(d1, (1,1),strides=(1,1),padding='same',activation=self.activation)(layer_3_pool)
        incept_3 = layers.Conv2D(d3, (3,3),strides=(1,1),padding='same',activation=self.activation)(incept_1_to_3)
        incept_5 = layers.Conv2D(d5, (5,5),strides=(1,1),padding='same',activation=self.activation)(incept_1_to_5)
        incept_max_pool= layers.MaxPooling2D(pool_size=(3, 3),strides=(1,1),padding='same')(layer_3_pool)
        incept_max_pool_depth=layers.Conv2D(d_max, (1,1),strides=(1,1),padding='same',activation='relu')(incept_max_pool)
        
        all_output = layers.concatenate([incept_1, incept_3,incept_5,incept_max_pool_depth], axis=3)
        layer_3_4_pool=layers.Conv2D(d1+d3+d5+d_max, (1,1),strides=(1,1),padding='same',activation=self.activation)(layer_3_pool)
        layer_34_residual = layers.add([layer_3_4_pool, all_output])
        
        layer_4_pool = layers.MaxPooling2D(pool_size=(3, 3))(layer_34_residual)
        #
        incept_1_to_final =layers.Conv2D(d_lay_3_to_4, (1,1),strides=(1,1),padding='same',activation=self.activation)(layer_4_pool)
        return incept_1_to_final
    
    def model_maker(self,f_inc=1,f_d=1):

        inp1 = keras.Input(shape=(200, 200, self.channels))
        inp2 = keras.Input(shape=(200, 200, self.channels))
        inp3 = keras.Input(shape=(200, 200, self.channels))
        
        tower_1 = self.parallel(inp1,f_inc,f_d)
        tower_2 = self.parallel(inp2,f_inc,f_d)
        tower_3 = self.parallel(inp3,f_inc,f_d)
        
        if self.tower_min_max_only:
            tower_max = layers.maximum([tower_1,tower_2,tower_3])
            tower_min=layers.minimum([tower_max, tower_2,tower_3])
            #tower_average=layers.average([tower_1, tower_2,tower_3])
            #tower_mul= layers.Multiply([tower_1, tower_2,tower_3])#since multiplication is expensive
            merged = layers.concatenate([tower_max, tower_min], axis=1)
            model_name = ''.join(['MIN_MAX_model_inception_residual_mentor_vin_1_act_',self.activation,'_f_inc_',str(f_inc),'_f_d_',str(f_d),'.h5'])
        
        else:
            merged = layers.concatenate([tower_1, tower_2, tower_3], axis=1)
            model_name = ''.join(['model_inception_residual_mentor_vin_1_act_',self.activation,'_f_inc_',str(f_inc),'_f_d_',str(f_d),'.h5'])
        
        merged = layers.Flatten()(merged)
        all_co = layers.Dense(100, activation=self.activation)(merged)
        all_co = layers.Dropout(0.5)(all_co)
        all_co = layers.Dense(50, activation=self.activation)(all_co)
        all_co= layers.Dropout(0.5)(all_co)
        outputs = layers.Dense(3, activation='softmax')(all_co)
        
        model = keras.models.Model(inputs=[inp1, inp2,inp3], outputs=outputs, name=model_name)
        
        return model,model_name    
    
'''parallel inception models'''
class model_par_inception_only_mentor_vin_1:

    def __init__(self, channels=17,tower_min_max_only=False,activation='relu'):
        '''Initialization
        
        activation can be 'selu','swish','nishy_vin1'
        '''
        self.activation = activation
        self.channels = channels
        self.tower_min_max_only = tower_min_max_only

    def model_maker(self,f_inc=1,f_d=1):

        d1=128*f_inc
        d3=64*f_inc
        d5=32*f_inc
        d_max=32*f_inc  
        d_lay_1_to_2 = 32*f_d
        d_lay_3_to_4 = 64*f_d

        inputs = keras.Input(shape=(200, 200, self.channels))
        
        inp1 = keras.Input(shape=(200, 200, self.channels))
        inp2 = keras.Input(shape=(200, 200, self.channels))
        inp3 = keras.Input(shape=(200, 200, self.channels))    
         
        incept_1 = layers.Conv2D(d1, (1,1),strides=(1,1),padding='same',activation=self.activation)(inputs)
        incept_3 = layers.Conv2D(d3, (3,3),strides=(1,1),padding='same',activation=self.activation)(inputs)
        incept_5 = layers.Conv2D(d5, (5,5),strides=(1,1),padding='same',activation=self.activation)(inputs)
        incept_max_pool= layers.MaxPooling2D(pool_size=(3, 3),strides=(1,1),padding='same')(inputs)
        incept_max_pool_depth=layers.Conv2D(d_max, (1,1),strides=(1,1),padding='same',activation='relu')(incept_max_pool)
        
        all_output = layers.concatenate([incept_1, incept_3,incept_5,incept_max_pool_depth], axis=3)
        layer_1_pool = layers.MaxPooling2D(pool_size=(4, 4))(all_output)
        
        #to reduce the depth representation
        incept_1_to_3=layers.Conv2D(d_lay_1_to_2, (1,1),strides=(1,1),padding='same',activation=self.activation)(layer_1_pool)
        incept_1_to_5=layers.Conv2D(d_lay_1_to_2, (1,1),strides=(1,1),padding='same',activation=self.activation)(layer_1_pool)
        
        incept_1 = layers.Conv2D(d1, (1,1),strides=(1,1),padding='same',activation=self.activation)(layer_1_pool)
        incept_3 = layers.Conv2D(d3, (3,3),strides=(1,1),padding='same',activation=self.activation)(incept_1_to_3)
        incept_5 = layers.Conv2D(d5, (5,5),strides=(1,1),padding='same',activation=self.activation)(incept_1_to_5)
        incept_max_pool= layers.MaxPooling2D(pool_size=(3, 3),strides=(1,1),padding='same')(layer_1_pool)
        incept_max_pool_depth=layers.Conv2D(d_max, (1,1),strides=(1,1),padding='same',activation='relu')(incept_max_pool)
        
        all_output = layers.concatenate([incept_1, incept_3,incept_5,incept_max_pool_depth], axis=3)
        layer_2_pool = layers.MaxPooling2D(pool_size=(3, 3))(all_output)
        
        #to reduce the depth representation
        incept_1_to_3=layers.Conv2D(d_lay_1_to_2, (1,1),strides=(1,1),padding='same',activation=self.activation)(layer_2_pool)
        incept_1_to_5=layers.Conv2D(d_lay_1_to_2, (1,1),strides=(1,1),padding='same',activation=self.activation)(layer_2_pool)
        
        incept_1 = layers.Conv2D(d1, (1,1),strides=(1,1),padding='same',activation=self.activation)(layer_2_pool)
        incept_3 = layers.Conv2D(d3, (3,3),strides=(1,1),padding='same',activation=self.activation)(incept_1_to_3)
        incept_5 = layers.Conv2D(d5, (5,5),strides=(1,1),padding='same',activation=self.activation)(incept_1_to_5)
        incept_max_pool= layers.MaxPooling2D(pool_size=(3, 3),strides=(1,1),padding='same')(layer_2_pool)
        incept_max_pool_depth=layers.Conv2D(d_max, (1,1),strides=(1,1),padding='same',activation='relu')(incept_max_pool)
        
        all_output = layers.concatenate([incept_1, incept_3,incept_5,incept_max_pool_depth], axis=3)
        layer_3_pool = layers.MaxPooling2D(pool_size=(2, 2))(all_output)
        #to reduce the depth representation
        incept_1_to_3=layers.Conv2D(d_lay_3_to_4, (1,1),strides=(1,1),padding='same',activation=self.activation)(layer_3_pool)
        incept_1_to_5=layers.Conv2D(d_lay_3_to_4, (1,1),strides=(1,1),padding='same',activation=self.activation)(layer_3_pool)
        
        incept_1 = layers.Conv2D(d1, (1,1),strides=(1,1),padding='same',activation=self.activation)(layer_3_pool)
        incept_3 = layers.Conv2D(d3, (3,3),strides=(1,1),padding='same',activation=self.activation)(incept_1_to_3)
        incept_5 = layers.Conv2D(d5, (5,5),strides=(1,1),padding='same',activation=self.activation)(incept_1_to_5)
        incept_max_pool= layers.MaxPooling2D(pool_size=(3, 3),strides=(1,1),padding='same')(layer_3_pool)
        incept_max_pool_depth=layers.Conv2D(d_max, (1,1),strides=(1,1),padding='same',activation='relu')(incept_max_pool)
        
        all_output = layers.concatenate([incept_1, incept_3,incept_5,incept_max_pool_depth], axis=3)
        layer_4_pool = layers.MaxPooling2D(pool_size=(2, 2))(all_output)
        #
        incept_1_to_final =layers.Conv2D(d_lay_3_to_4, (1,1),strides=(1,1),padding='same',activation=self.activation)(layer_4_pool)
        
        parallel = keras.models.Model(inputs, incept_1_to_final, name='parallel')     
        #parallel = keras.models.Model(inputs,all_output, name='parallel') 
        tower_1 = parallel(inp1)
        tower_2 = parallel(inp2)
        tower_3 = parallel(inp3)
        
        if self.tower_min_max_only:
            tower_max = layers.maximum([tower_1,tower_2,tower_3])
            tower_min=layers.minimum([tower_max, tower_2,tower_3])
            #tower_average=layers.average([tower_1, tower_2,tower_3])
            #tower_mul= layers.Multiply([tower_1, tower_2,tower_3])#since multiplication is expensive
            merged = layers.concatenate([tower_max, tower_min], axis=1)
            model_name = ''.join(['MIN_MAX_model_par_inception_only_mentor_vin_1_act_',self.activation,'_f_inc_',str(f_inc),'_f_d_',str(f_d),'.h5'])
        
        else:
            merged = layers.concatenate([tower_1, tower_2, tower_3], axis=1)
            model_name = ''.join(['model_par_inception_only_mentor_vin_1_act_',self.activation,'_f_inc_',str(f_inc),'_f_d_',str(f_d),'.h5'])
        
        merged = layers.Flatten()(merged)
        all_co = layers.Dense(100, activation=self.activation)(merged)
        all_co = layers.Dropout(0.5)(all_co)
        all_co = layers.Dense(50, activation=self.activation)(all_co)
        all_co= layers.Dropout(0.5)(all_co)
        outputs = layers.Dense(3, activation='softmax')(all_co)
        
        model = keras.models.Model(inputs=[inp1, inp2,inp3], outputs=outputs, name=model_name)
        
        return model,model_name    

class model_par_inception_residual_mentor_vin_1:

    def __init__(self, channels=17,tower_min_max_only=False,activation='relu'):
        '''Initialization
        
        activation can be 'selu','swish','nishy_vin1'
        '''
        self.activation = activation
        self.channels = channels
        self.tower_min_max_only = tower_min_max_only

    def model_maker(self,f_inc=1,f_d=1):

        d1=128*f_inc
        d3=64*f_inc
        d5=32*f_inc
        d_max=32*f_inc  
        d_lay_1_to_2 = 32*f_d
        d_lay_3_to_4 = 64*f_d

        inputs = keras.Input(shape=(200, 200, self.channels))
        
        inp1 = keras.Input(shape=(200, 200, self.channels))
        inp2 = keras.Input(shape=(200, 200, self.channels))
        inp3 = keras.Input(shape=(200, 200, self.channels))      
         
        incept_1 = layers.Conv2D(d1, (1,1),strides=(1,1),padding='same',activation=self.activation)(inputs)
        incept_3 = layers.Conv2D(d3, (3,3),strides=(1,1),padding='same',activation=self.activation)(inputs)
        incept_5 = layers.Conv2D(d5, (5,5),strides=(1,1),padding='same',activation=self.activation)(inputs)
        incept_max_pool= layers.MaxPooling2D(pool_size=(3, 3),strides=(1,1),padding='same')(inputs)
        incept_max_pool_depth=layers.Conv2D(d_max, (1,1),strides=(1,1),padding='same',activation='relu')(incept_max_pool)
        
        all_output = layers.concatenate([incept_1, incept_3,incept_5,incept_max_pool_depth], axis=3)
        layer_1_pool = layers.MaxPooling2D(pool_size=(4, 4))(all_output)
        
        #to reduce the depth representation
        incept_1_to_3=layers.Conv2D(d_lay_1_to_2, (1,1),strides=(1,1),padding='same',activation=self.activation)(layer_1_pool)
        incept_1_to_5=layers.Conv2D(d_lay_1_to_2, (1,1),strides=(1,1),padding='same',activation=self.activation)(layer_1_pool)
        
        incept_1 = layers.Conv2D(d1, (1,1),strides=(1,1),padding='same',activation=self.activation)(layer_1_pool)
        incept_3 = layers.Conv2D(d3, (3,3),strides=(1,1),padding='same',activation=self.activation)(incept_1_to_3)
        incept_5 = layers.Conv2D(d5, (5,5),strides=(1,1),padding='same',activation=self.activation)(incept_1_to_5)
        incept_max_pool= layers.MaxPooling2D(pool_size=(3, 3),strides=(1,1),padding='same')(layer_1_pool)
        incept_max_pool_depth=layers.Conv2D(d_max, (1,1),strides=(1,1),padding='same',activation='relu')(incept_max_pool)
        
        all_output = layers.concatenate([incept_1, incept_3,incept_5,incept_max_pool_depth], axis=3)
        
        layer_1_2_pool=layers.Conv2D(d1+d3+d5+d_max, (1,1),strides=(1,1),padding='same',activation=self.activation)(layer_1_pool)
        layer_21_residual = layers.add([layer_1_2_pool, all_output])
        
        layer_2_pool = layers.MaxPooling2D(pool_size=(3, 3))(layer_21_residual)
        #to reduce the depth representation
        incept_1_to_3=layers.Conv2D(d_lay_1_to_2, (1,1),strides=(1,1),padding='same',activation=self.activation)(layer_2_pool)
        incept_1_to_5=layers.Conv2D(d_lay_1_to_2, (1,1),strides=(1,1),padding='same',activation=self.activation)(layer_2_pool)
        
        incept_1 = layers.Conv2D(d1, (1,1),strides=(1,1),padding='same',activation=self.activation)(layer_2_pool)
        incept_3 = layers.Conv2D(d3, (3,3),strides=(1,1),padding='same',activation=self.activation)(incept_1_to_3)
        incept_5 = layers.Conv2D(d5, (5,5),strides=(1,1),padding='same',activation=self.activation)(incept_1_to_5)
        incept_max_pool= layers.MaxPooling2D(pool_size=(3, 3),strides=(1,1),padding='same')(layer_2_pool)
        incept_max_pool_depth=layers.Conv2D(d_max, (1,1),strides=(1,1),padding='same',activation='relu')(incept_max_pool)
        
        all_output = layers.concatenate([incept_1, incept_3,incept_5,incept_max_pool_depth], axis=3)
        layer_2_3_pool=layers.Conv2D(d1+d3+d5+d_max, (1,1),strides=(1,1),padding='same',activation=self.activation)(layer_2_pool)
        layer_32_residual = layers.add([layer_2_3_pool, all_output])
        
        layer_3_pool = layers.MaxPooling2D(pool_size=(3, 3))(layer_32_residual)
        #to reduce the depth representation
        incept_1_to_3=layers.Conv2D(d_lay_3_to_4, (1,1),strides=(1,1),padding='same',activation=self.activation)(layer_3_pool)
        incept_1_to_5=layers.Conv2D(d_lay_3_to_4, (1,1),strides=(1,1),padding='same',activation=self.activation)(layer_3_pool)
        
        incept_1 = layers.Conv2D(d1, (1,1),strides=(1,1),padding='same',activation=self.activation)(layer_3_pool)
        incept_3 = layers.Conv2D(d3, (3,3),strides=(1,1),padding='same',activation=self.activation)(incept_1_to_3)
        incept_5 = layers.Conv2D(d5, (5,5),strides=(1,1),padding='same',activation=self.activation)(incept_1_to_5)
        incept_max_pool= layers.MaxPooling2D(pool_size=(3, 3),strides=(1,1),padding='same')(layer_3_pool)
        incept_max_pool_depth=layers.Conv2D(d_max, (1,1),strides=(1,1),padding='same',activation='relu')(incept_max_pool)
        
        all_output = layers.concatenate([incept_1, incept_3,incept_5,incept_max_pool_depth], axis=3)
        layer_3_4_pool=layers.Conv2D(d1+d3+d5+d_max, (1,1),strides=(1,1),padding='same',activation=self.activation)(layer_3_pool)
        layer_34_residual = layers.add([layer_3_4_pool, all_output])
        
        layer_4_pool = layers.MaxPooling2D(pool_size=(3, 3))(layer_34_residual)
        #
        incept_1_to_final =layers.Conv2D(d_lay_3_to_4, (1,1),strides=(1,1),padding='same',activation=self.activation)(layer_4_pool)
        parallel = keras.models.Model(inputs, incept_1_to_final, name='parallel')     
        #parallel = keras.models.Model(inputs,all_output, name='parallel') 
        tower_1 = parallel(inp1)
        tower_2 = parallel(inp2)
        tower_3 = parallel(inp3)
        
        if self.tower_min_max_only:
            tower_max = layers.maximum([tower_1,tower_2,tower_3])
            tower_min=layers.minimum([tower_max, tower_2,tower_3])
            #tower_average=layers.average([tower_1, tower_2,tower_3])
            #tower_mul= layers.Multiply([tower_1, tower_2,tower_3])#since multiplication is expensive
            merged = layers.concatenate([tower_max, tower_min], axis=1)
            model_name = ''.join(['MIN_MAX_model_par_inception_residual_mentor_vin_1_act_',self.activation,'_f_inc_',str(f_inc),'_f_d_',str(f_d),'.h5'])
        
        else:
            merged = layers.concatenate([tower_1, tower_2, tower_3], axis=1)
            model_name = ''.join(['model_par_inception_residual_mentor_vin_1_act_',self.activation,'_f_inc_',str(f_inc),'_f_d_',str(f_d),'.h5'])
        
        merged = layers.Flatten()(merged)
        all_co = layers.Dense(100, activation=self.activation)(merged)
        all_co = layers.Dropout(0.5)(all_co)
        all_co = layers.Dense(50, activation=self.activation)(all_co)
        all_co= layers.Dropout(0.5)(all_co)
        outputs = layers.Dense(3, activation='softmax')(all_co)
        
        model = keras.models.Model(inputs=[inp1, inp2,inp3], outputs=outputs, name=model_name)
        
        return model,model_name    

class model_vin_4:
    
    def __init__(self,channels=17,tower_min_max_only=False):
        'Initialization'
        self.channels = channels
        self.tower_min_max_only = tower_min_max_only
    
    def parallel(self,inputs,k1,k2,k3,k4):
    
        xy = layers.Conv2D(32, (k1,k1),strides=(1,1),padding='same',activation='relu')(inputs)
        block_1_xy_output = layers.MaxPooling2D(pool_size=(4, 4))(xy)
        
        xy = layers.Conv2D(32, (k2,k2),strides=(1,1),padding='same',activation='relu')(block_1_xy_output)
        block_2_output = layers.add([xy, block_1_xy_output])
    
        block_2_xy_output = layers.MaxPooling2D(pool_size=(2, 2))(block_2_output)
    
        xy = layers.Conv2D(64, (k3,k3), activation='relu', padding='same')(block_2_xy_output)
        '''to be removed'''
        xy = layers.MaxPooling2D(pool_size=(2, 2))(xy)
        xy = layers.Conv2D(64, (k4,k4), activation='relu', padding='same')(xy)
        block_3_xy_output = layers.MaxPooling2D(pool_size=(2, 2))(xy)
        return block_3_xy_output
 
#    def model_maker_same_kernal_projections(self,p1=7,p2=5,p3=3):
    def model_maker(self,p1=3,p2=5):
        
        inp1 = keras.Input(shape=(200, 200, self.channels))
        inp2 = keras.Input(shape=(200, 200, self.channels))
        inp3 = keras.Input(shape=(200, 200, self.channels))
        
        tower_1 = self.parallel(inp1,p2,p2,p1,p1)
        tower_2 = self.parallel(inp2,p2,p2,p1,p1)
        tower_3 = self.parallel(inp3,p2,p2,p1,p1)
        
        if self.tower_min_max_only:
            tower_max = layers.maximum([tower_1,tower_2,tower_3])
            tower_min=layers.minimum([tower_max, tower_2,tower_3])
            #tower_average=layers.average([tower_1, tower_2,tower_3])
            #tower_mul= layers.Multiply([tower_1, tower_2,tower_3])#since multiplication is expensive
            merged = layers.concatenate([tower_max, tower_min], axis=1)
            model_name =''.join(['MIN_MAX_model_vin_4_p1_',str(p1),'_p2_',str(p2),'.h5'])

        else:
            merged = layers.concatenate([tower_1, tower_2, tower_3], axis=1)
            model_name =''.join(['model_vin_4_p1_',str(p1),'_p2_',str(p2),'.h5'])
      
        
        merged = layers.Flatten()(merged)
        all_co = layers.Dense(100, activation='relu')(merged)
        all_co = layers.Dropout(0.5)(all_co)
        all_co = layers.Dense(50, activation='relu')(all_co)
        all_co= layers.Dropout(0.5)(all_co)
        outputs = layers.Dense(3, activation='softmax')(all_co)

        model_name = 'model_vin_4_same.h5'
        model = keras.Model(inputs=[inp1, inp2,inp3], outputs=outputs, name='model_vin_4_same')
        
        return model,model_name
    
#    def model_maker_same_kernal_combo_projections(self,p11=7,p12=3,p21=5,p22=5,p31=3,p32=7):
#        
#        inp1 = keras.Input(shape=(200, 200, self.channels))
#        inp2 = keras.Input(shape=(200, 200, self.channels))
#        inp3 = keras.Input(shape=(200, 200, self.channels))
#        
#        tower_1 = self.parallel(inp1,p11,p11,p12,p12)
#        tower_2 = self.parallel(inp2,p21,p21,p22,p22)
#        tower_3 = self.parallel(inp3,p31,p32,p31,p32)
#        
#        merged = layers.concatenate([tower_1, tower_2, tower_3], axis=1)
#        merged = layers.Flatten()(merged)
#        all_co = layers.Dense(100, activation='relu')(merged)
#        all_co = layers.Dropout(0.5)(all_co)
#        all_co = layers.Dense(50, activation='relu')(all_co)
#        all_co= layers.Dropout(0.5)(all_co)
#        outputs = layers.Dense(3, activation='softmax')(all_co)
#
#        model_name = 'model_vin_4_combo.h5'
#        model = keras.Model(inputs=[inp1, inp2,inp3], outputs=outputs, name='model_vin_4_combo')
#
#        return model,model_name


class model_accidental:
    
    def __init__(self,channels=17,tower_min_max_only=False):
        'Initialization'
        self.channels = channels
        self.tower_min_max_only = tower_min_max_only
    
    def parallel(self,inputs,k1,k2,k3,k4,d1):
    
        xy = layers.Conv2D(d1, (k1,k1),strides=(1,1),padding='same',activation='relu')(inputs)
        block_1_xy_output = layers.MaxPooling2D(pool_size=(4, 4))(xy)
        
        xy = layers.Conv2D(d1, (k2,k2),strides=(1,1),padding='same',activation='relu')(block_1_xy_output)
        block_2_xy_output = layers.MaxPooling2D(pool_size=(2, 2))(xy)
    
        xy = layers.Conv2D(2*d1, (k3,k3), activation='relu', padding='same')(block_2_xy_output)
        xy = layers.MaxPooling2D(pool_size=(2, 2))(xy)

        xy = layers.Conv2D(2*d1, (k4,k4), activation='relu', padding='same')(xy)
        block_3_xy_output = layers.MaxPooling2D(pool_size=(2, 2))(xy)
        return block_3_xy_output
 
    def model_maker(self,p1=3,p2=3,p3=3,d1=32):
        
        inp1 = keras.Input(shape=(200, 200, self.channels))
        inp2 = keras.Input(shape=(200, 200, self.channels))
        inp3 = keras.Input(shape=(200, 200, self.channels))
        
        tower_1 = self.parallel(inp1,p1,p2,p3,p3,d1)
        tower_2 = self.parallel(inp2,p1,p2,p3,p3,d1)
        tower_3 = self.parallel(inp3,p1,p2,p3,p3,d1)
        
        if self.tower_min_max_only:
            tower_max = layers.maximum([tower_1,tower_2,tower_3])
            tower_min=layers.minimum([tower_max, tower_2,tower_3])
            #tower_average=layers.average([tower_1, tower_2,tower_3])
            #tower_mul= layers.Multiply([tower_1, tower_2,tower_3])#since multiplication is expensive
            merged = layers.concatenate([tower_max, tower_min], axis=1)
            model_name =''.join(['MIN_MAX_model_accidental_p1_',str(p1),'_p2_',str(p2),'_p3_',str(p3),'_d1_',str(d1),'.h5'])

        else:
            merged = layers.concatenate([tower_1, tower_2, tower_3], axis=1)
            model_name =''.join(['model_accidental_p1_',str(p1),'_p2_',str(p2),'_p3_',str(p3),'_d1_',str(d1),'.h5'])
            
        merged = layers.Flatten()(merged)
        all_co = layers.Dense(100, activation='relu')(merged)
        all_co = layers.Dropout(0.5)(all_co)
        all_co = layers.Dense(50, activation='relu')(all_co)
        all_co= layers.Dropout(0.5)(all_co)
        outputs = layers.Dense(3, activation='softmax')(all_co)

        model = keras.Model(inputs=[inp1, inp2,inp3], outputs=outputs, name=model_name)
        
        return model,model_name

'''Models with inception'''
class model_inception_layer_1:
    
    def __init__(self,channels=19,tower_min_max_only=False):
        'Initialization'
        self.channels = channels
        self.tower_min_max_only = tower_min_max_only

    def parallel(self,inputs,d1,k2,k3,k4,k_b1,k_b2,k_b3,k_b4):
    
        incept_3 = layers.Conv2D(d1, (k_b1,k_b1),strides=(1,1),padding='same',activation='relu')(inputs)
        incept_5 = layers.Conv2D(d1, (k_b2,k_b2),strides=(1,1),padding='same',activation='relu')(inputs)
        incept_7 = layers.Conv2D(d1, (k_b3,k_b3),strides=(1,1),padding='same',activation='relu')(inputs)
        incept_9 = layers.Conv2D(d1, (k_b4,k_b4),strides=(1,1),padding='same',activation='relu')(inputs)
#        incept_10 = layers.Conv2D(d1, (k_b5,k_b5),strides=(1,1),padding='same',activation='relu')(inputs)

        data = layers.concatenate([incept_3, incept_5, incept_7,incept_9], axis=3)
        data = layers.MaxPooling2D(pool_size=(4, 4))(data)
        
        data_2 = layers.Conv2D(d1*4, (k2,k2),strides=(1,1),padding='same',activation='relu')(data)
        data = layers.add([data_2, data])
    
        data = layers.MaxPooling2D(pool_size=(2, 2))(data)
    
        data = layers.Conv2D(64, (k3,k3), activation='relu', padding='same')(data)
        '''to be removed'''
        data = layers.MaxPooling2D(pool_size=(2, 2))(data)
        data = layers.Conv2D(64, (k4,k4), activation='relu', padding='same')(data)
        data = layers.MaxPooling2D(pool_size=(2, 2))(data)
        return data

    def model_maker(self,d1=8,k2=3,k3=3,k4=3,k_b1=3,k_b2=3,k_b3=3,k_b4=5,k_b5=5):
         
        inp1 = keras.Input(shape=(200, 200, self.channels))
        inp2 = keras.Input(shape=(200, 200, self.channels))
        inp3 = keras.Input(shape=(200, 200, self.channels))
        
        tower_1 = self.parallel(inp1,d1,k2,k3,k4,k_b1,k_b2,k_b3,k_b4)
        tower_2 = self.parallel(inp2,d1,k2,k3,k4,k_b1,k_b2,k_b3,k_b4)
        tower_3 = self.parallel(inp3,d1,k2,k3,k4,k_b1,k_b2,k_b3,k_b4)

        if self.tower_min_max_only:
            tower_max = layers.maximum([tower_1,tower_2,tower_3])
            tower_min=layers.minimum([tower_max, tower_2,tower_3])
            #tower_average=layers.average([tower_1, tower_2,tower_3])
            #tower_mul= layers.Multiply([tower_1, tower_2,tower_3])#since multiplication is expensive
            merged = layers.concatenate([tower_max, tower_min], axis=1)
            model_name =''.join(['MIN_MAX_model_inception_layer_1_d1_',str(d1),'_k2_',str(k2),'_k3_',str(k3),'_k4_',str(k4),'_kb1_',str(k_b1),'_kb2_',str(k_b2),'_kb3_',str(k_b3),'_kb4_',str(k_b4),'.h5'])

        else:
            merged = layers.concatenate([tower_1, tower_2, tower_3], axis=1)
            model_name =''.join(['model_inception_layer_1_d1_',str(d1),'_k2_',str(k2),'_k3_',str(k3),'_k4_',str(k4),'_kb1_',str(k_b1),'_kb2_',str(k_b2),'_kb3_',str(k_b3),'_kb4_',str(k_b4),'.h5'])
        

        merged = layers.concatenate([tower_1, tower_2, tower_3], axis=1)
        merged = layers.Flatten()(merged)
        all_co = layers.Dense(100, activation='relu')(merged)
        all_co = layers.Dropout(0.5)(all_co)
        all_co = layers.Dense(50, activation='relu')(all_co)
        all_co= layers.Dropout(0.5)(all_co)
        outputs = layers.Dense(3, activation='softmax')(all_co)

        model = keras.Model(inputs=[inp1, inp2,inp3], outputs=outputs, name=model_name)
        
        return model,model_name

'''Models with inception'''
class model_inception_layer_1_normal:
    
    def __init__(self,channels=19,tower_min_max_only=False):
        'Initialization'
        self.channels = channels
        self.tower_min_max_only = tower_min_max_only

    def parallel(self,inputs,d1,k2,k3,k4,k_b1,k_b2,k_b3,k_b4):
    
        incept_3 = layers.Conv2D(d1, (k_b1,k_b1),strides=(1,1),padding='same',activation='relu')(inputs)
        incept_5 = layers.Conv2D(d1, (k_b2,k_b2),strides=(1,1),padding='same',activation='relu')(inputs)
        incept_7 = layers.Conv2D(d1, (k_b3,k_b3),strides=(1,1),padding='same',activation='relu')(inputs)
        incept_9 = layers.Conv2D(d1, (k_b4,k_b4),strides=(1,1),padding='same',activation='relu')(inputs)
        
        data = layers.concatenate([incept_3, incept_5, incept_7,incept_9], axis=3)
        data = layers.MaxPooling2D(pool_size=(4, 4))(data)
        
        data = layers.Conv2D(d1*4, (k2,k2),strides=(1,1),padding='same',activation='relu')(data)

        data = layers.MaxPooling2D(pool_size=(2, 2))(data)
    
        data = layers.Conv2D(64, (k3,k3), activation='relu', padding='same')(data)
        '''to be removed'''
        data = layers.MaxPooling2D(pool_size=(2, 2))(data)
        data = layers.Conv2D(64, (k4,k4), activation='relu', padding='same')(data)
        data = layers.MaxPooling2D(pool_size=(2, 2))(data)
        return data

    def model_maker(self,d1=8,k2=3,k3=3,k4=3,k_b1=3,k_b2=3,k_b3=3,k_b4=5):
         
        inp1 = keras.Input(shape=(200, 200, self.channels))
        inp2 = keras.Input(shape=(200, 200, self.channels))
        inp3 = keras.Input(shape=(200, 200, self.channels))
        
        tower_1 = self.parallel(inp1,d1,k2,k3,k4,k_b1,k_b2,k_b3,k_b4)
        tower_2 = self.parallel(inp2,d1,k2,k3,k4,k_b1,k_b2,k_b3,k_b4)
        tower_3 = self.parallel(inp3,d1,k2,k3,k4,k_b1,k_b2,k_b3,k_b4)

        if self.tower_min_max_only:
            tower_max = layers.maximum([tower_1,tower_2,tower_3])
            tower_min=layers.minimum([tower_max, tower_2,tower_3])
            #tower_average=layers.average([tower_1, tower_2,tower_3])
            #tower_mul= layers.Multiply([tower_1, tower_2,tower_3])#since multiplication is expensive
            merged = layers.concatenate([tower_max, tower_min], axis=1)
            model_name =''.join(['MIN_MAX_model_inception_layer_1_d1_',str(d1),'_k2_',str(k2),'_k3_',str(k3),'_k4_',str(k4),'_kb1_',str(k_b1),'_kb2_',str(k_b2),'_kb3_',str(k_b3),'_kb4_',str(k_b4),'.h5'])

        else:
            merged = layers.concatenate([tower_1, tower_2, tower_3], axis=1)
            model_name =''.join(['model_inception_layer_1_d1_',str(d1),'_k2_',str(k2),'_k3_',str(k3),'_k4_',str(k4),'_kb1_',str(k_b1),'_kb2_',str(k_b2),'_kb3_',str(k_b3),'_kb4_',str(k_b4),'.h5'])
        

        merged = layers.concatenate([tower_1, tower_2, tower_3], axis=1)
        merged = layers.Flatten()(merged)
        all_co = layers.Dense(100, activation='relu')(merged)
        all_co = layers.Dropout(0.5)(all_co)
        all_co = layers.Dense(50, activation='relu')(all_co)
        all_co= layers.Dropout(0.5)(all_co)
        outputs = layers.Dense(3, activation='softmax')(all_co)

        model = keras.Model(inputs=[inp1, inp2,inp3], outputs=outputs, name=model_name)
        
        return model,model_name
'''Models with inception_alias'''
class model_inception_layer_1_alias_1:
    
    def __init__(self,channels=19,tower_min_max_only=False):
        'Initialization'
        self.channels = channels
        self.tower_min_max_only = tower_min_max_only

    def parallel(self,inputs,d1,k2,k3,k4,k_b1,k_b2,k_b3,k_b4,k_b5):
    
        incept_3 = layers.Conv2D(d1, (k_b1,k_b1),strides=(1,1),padding='same',activation='relu')(inputs)
        incept_5 = layers.Conv2D(d1, (k_b2,k_b2),strides=(1,1),padding='same',activation='relu')(inputs)
        incept_7 = layers.Conv2D(d1, (k_b3,k_b3),strides=(1,1),padding='same',activation='relu')(inputs)
        incept_9 = layers.Conv2D(d1, (k_b4,k_b4),strides=(1,1),padding='same',activation='relu')(inputs)
        incept_10 = layers.Conv2D(d1, (k_b5,k_b5),strides=(1,1),padding='same',activation='relu')(inputs)
#        incept_10 = layers.Conv2D(d1, (k_b5,k_b5),strides=(1,1),padding='same',activation='relu')(inputs)

        data = layers.concatenate([incept_3, incept_5, incept_7,incept_9,incept_10], axis=3)
        data = layers.MaxPooling2D(pool_size=(4, 4))(data)
        
        data_2 = layers.Conv2D(d1*5, (k2,k2),strides=(1,1),padding='same',activation='relu')(data)
        data = layers.add([data_2, data])
    
        data = layers.MaxPooling2D(pool_size=(2, 2))(data)
    
        data = layers.Conv2D(64, (k3,k3), activation='relu', padding='same')(data)
        '''to be removed'''
        data = layers.MaxPooling2D(pool_size=(2, 2))(data)
        data = layers.Conv2D(64, (k4,k4), activation='relu', padding='same')(data)
        data = layers.MaxPooling2D(pool_size=(2, 2))(data)
        return data

    def model_maker(self,d1=8,k2=3,k3=3,k4=3,k_b1=3,k_b2=3,k_b3=3,k_b4=5,k_b5=5):
         
        inp1 = keras.Input(shape=(200, 200, self.channels))
        inp2 = keras.Input(shape=(200, 200, self.channels))
        inp3 = keras.Input(shape=(200, 200, self.channels))
        
        tower_1 = self.parallel(inp1,d1,k2,k3,k4,k_b1,k_b2,k_b3,k_b4,k_b5)
        tower_2 = self.parallel(inp2,d1,k2,k3,k4,k_b1,k_b2,k_b3,k_b4,k_b5)
        tower_3 = self.parallel(inp3,d1,k2,k3,k4,k_b1,k_b2,k_b3,k_b4,k_b5)

        if self.tower_min_max_only:
            tower_max = layers.maximum([tower_1,tower_2,tower_3])
            tower_min=layers.minimum([tower_max, tower_2,tower_3])
            #tower_average=layers.average([tower_1, tower_2,tower_3])
            #tower_mul= layers.Multiply([tower_1, tower_2,tower_3])#since multiplication is expensive
            merged = layers.concatenate([tower_max, tower_min], axis=1)
            model_name =''.join(['MIN_MAX_model_inception_layer_1_d1_',str(d1),'_k2_',str(k2),'_k3_',str(k3),'_k4_',str(k4),'_kb1_',str(k_b1),'_kb2_',str(k_b2),'_kb3_',str(k_b3),'_kb4_',str(k_b4),'.h5'])

        else:
            merged = layers.concatenate([tower_1, tower_2, tower_3], axis=1)
            model_name =''.join(['model_inception_layer_1_d1_',str(d1),'_k2_',str(k2),'_k3_',str(k3),'_k4_',str(k4),'_kb1_',str(k_b1),'_kb2_',str(k_b2),'_kb3_',str(k_b3),'_kb4_',str(k_b4),'.h5'])
        

        merged = layers.concatenate([tower_1, tower_2, tower_3], axis=1)
        merged = layers.Flatten()(merged)
        all_co = layers.Dense(100, activation='relu')(merged)
        all_co = layers.Dropout(0.5)(all_co)
        all_co = layers.Dense(50, activation='relu')(all_co)
        all_co= layers.Dropout(0.5)(all_co)
        outputs = layers.Dense(3, activation='softmax')(all_co)

        model = keras.Model(inputs=[inp1, inp2,inp3], outputs=outputs, name=model_name)
        
        return model,model_name
'''Models with inception_alias'''
class model_inception_layer_1_normal_alias_1:
    
    def __init__(self,channels=19,tower_min_max_only=False):
        'Initialization'
        self.channels = channels
        self.tower_min_max_only = tower_min_max_only

    def parallel(self,inputs,d1,k2,k3,k4,k_b1,k_b2,k_b3,k_b4,k_b5):
    
        incept_3 = layers.Conv2D(d1, (k_b1,k_b1),strides=(1,1),padding='same',activation='relu')(inputs)
        incept_5 = layers.Conv2D(d1, (k_b2,k_b2),strides=(1,1),padding='same',activation='relu')(inputs)
        incept_7 = layers.Conv2D(d1, (k_b3,k_b3),strides=(1,1),padding='same',activation='relu')(inputs)
        incept_9 = layers.Conv2D(d1, (k_b4,k_b4),strides=(1,1),padding='same',activation='relu')(inputs)
        incept_10 = layers.Conv2D(d1, (k_b5,k_b5),strides=(1,1),padding='same',activation='relu')(inputs)
#        incept_10 = layers.Conv2D(d1, (k_b5,k_b5),strides=(1,1),padding='same',activation='relu')(inputs)

        data = layers.concatenate([incept_3, incept_5, incept_7,incept_9,incept_10], axis=3)
        data = layers.MaxPooling2D(pool_size=(4, 4))(data)
        
        data = layers.Conv2D(d1*5, (k2,k2),strides=(1,1),padding='same',activation='relu')(data)
    
        data = layers.MaxPooling2D(pool_size=(2, 2))(data)
    
        data = layers.Conv2D(64, (k3,k3), activation='relu', padding='same')(data)
        '''to be removed'''
        data = layers.MaxPooling2D(pool_size=(2, 2))(data)
        data = layers.Conv2D(64, (k4,k4), activation='relu', padding='same')(data)
        data = layers.MaxPooling2D(pool_size=(2, 2))(data)
        return data

    def model_maker(self,d1=8,k2=3,k3=3,k4=3,k_b1=3,k_b2=3,k_b3=3,k_b4=5,k_b5=5):
         
        inp1 = keras.Input(shape=(200, 200, self.channels))
        inp2 = keras.Input(shape=(200, 200, self.channels))
        inp3 = keras.Input(shape=(200, 200, self.channels))
        
        tower_1 = self.parallel(inp1,d1,k2,k3,k4,k_b1,k_b2,k_b3,k_b4,k_b5)
        tower_2 = self.parallel(inp2,d1,k2,k3,k4,k_b1,k_b2,k_b3,k_b4,k_b5)
        tower_3 = self.parallel(inp3,d1,k2,k3,k4,k_b1,k_b2,k_b3,k_b4,k_b5)

        if self.tower_min_max_only:
            tower_max = layers.maximum([tower_1,tower_2,tower_3])
            tower_min=layers.minimum([tower_max, tower_2,tower_3])
            #tower_average=layers.average([tower_1, tower_2,tower_3])
            #tower_mul= layers.Multiply([tower_1, tower_2,tower_3])#since multiplication is expensive
            merged = layers.concatenate([tower_max, tower_min], axis=1)
            model_name =''.join(['MIN_MAX_model_inception_layer_1_d1_',str(d1),'_k2_',str(k2),'_k3_',str(k3),'_k4_',str(k4),'_kb1_',str(k_b1),'_kb2_',str(k_b2),'_kb3_',str(k_b3),'_kb4_',str(k_b4),'.h5'])

        else:
            merged = layers.concatenate([tower_1, tower_2, tower_3], axis=1)
            model_name =''.join(['model_inception_layer_1_d1_',str(d1),'_k2_',str(k2),'_k3_',str(k3),'_k4_',str(k4),'_kb1_',str(k_b1),'_kb2_',str(k_b2),'_kb3_',str(k_b3),'_kb4_',str(k_b4),'.h5'])
        

        merged = layers.concatenate([tower_1, tower_2, tower_3], axis=1)
        merged = layers.Flatten()(merged)
        all_co = layers.Dense(100, activation='relu')(merged)
        all_co = layers.Dropout(0.5)(all_co)
        all_co = layers.Dense(50, activation='relu')(all_co)
        all_co= layers.Dropout(0.5)(all_co)
        outputs = layers.Dense(3, activation='softmax')(all_co)

        model = keras.Model(inputs=[inp1, inp2,inp3], outputs=outputs, name=model_name)
        
        return model,model_name
'''Models with inception_alias'''
class model_inception_layer_1_normal_alias_2:
    
    def __init__(self,channels=19,tower_min_max_only=False):
        'Initialization'
        self.channels = channels
        self.tower_min_max_only = tower_min_max_only

    def parallel(self,inputs,d1,k2,k3,k4,k_b1,k_b2,k_b3,k_b4,k_b5,k_b6):
    
        incept_3 = layers.Conv2D(d1, (k_b1,k_b1),strides=(1,1),padding='same',activation='relu')(inputs)
        incept_5 = layers.Conv2D(d1, (k_b2,k_b2),strides=(1,1),padding='same',activation='relu')(inputs)
        incept_7 = layers.Conv2D(d1, (k_b3,k_b3),strides=(1,1),padding='same',activation='relu')(inputs)
        incept_9 = layers.Conv2D(d1, (k_b4,k_b4),strides=(1,1),padding='same',activation='relu')(inputs)
        incept_10 = layers.Conv2D(d1, (k_b5,k_b5),strides=(1,1),padding='same',activation='relu')(inputs)
        incept_11 = layers.Conv2D(d1, (k_b6,k_b6),strides=(1,1),padding='same',activation='relu')(inputs)

        data = layers.concatenate([incept_3, incept_5, incept_7,incept_9,incept_10,incept_11], axis=3)
        data = layers.MaxPooling2D(pool_size=(4, 4))(data)
        
        data = layers.Conv2D(d1*6, (k2,k2),strides=(1,1),padding='same',activation='relu')(data)
    
        data = layers.MaxPooling2D(pool_size=(2, 2))(data)
    
        data = layers.Conv2D(64, (k3,k3), activation='relu', padding='same')(data)
        '''to be removed'''
        data = layers.MaxPooling2D(pool_size=(2, 2))(data)
        data = layers.Conv2D(64, (k4,k4), activation='relu', padding='same')(data)
        data = layers.MaxPooling2D(pool_size=(2, 2))(data)
        return data

    def model_maker(self,d1=8,k2=3,k3=3,k4=3,k_b1=3,k_b2=3,k_b3=3,k_b4=5,k_b5=5,k_b6=5):
         
        inp1 = keras.Input(shape=(200, 200, self.channels))
        inp2 = keras.Input(shape=(200, 200, self.channels))
        inp3 = keras.Input(shape=(200, 200, self.channels))
        
        tower_1 = self.parallel(inp1,d1,k2,k3,k4,k_b1,k_b2,k_b3,k_b4,k_b5,k_b6)
        tower_2 = self.parallel(inp2,d1,k2,k3,k4,k_b1,k_b2,k_b3,k_b4,k_b5,k_b6)
        tower_3 = self.parallel(inp3,d1,k2,k3,k4,k_b1,k_b2,k_b3,k_b4,k_b5,k_b6)

        if self.tower_min_max_only:
            tower_max = layers.maximum([tower_1,tower_2,tower_3])
            tower_min=layers.minimum([tower_max, tower_2,tower_3])
            #tower_average=layers.average([tower_1, tower_2,tower_3])
            #tower_mul= layers.Multiply([tower_1, tower_2,tower_3])#since multiplication is expensive
            merged = layers.concatenate([tower_max, tower_min], axis=1)
            model_name =''.join(['MIN_MAX_model_inception_layer_1_d1_',str(d1),'_k2_',str(k2),'_k3_',str(k3),'_k4_',str(k4),'_kb1_',str(k_b1),'_kb2_',str(k_b2),'_kb3_',str(k_b3),'_kb4_',str(k_b4),'.h5'])

        else:
            merged = layers.concatenate([tower_1, tower_2, tower_3], axis=1)
            model_name =''.join(['model_inception_layer_1_d1_',str(d1),'_k2_',str(k2),'_k3_',str(k3),'_k4_',str(k4),'_kb1_',str(k_b1),'_kb2_',str(k_b2),'_kb3_',str(k_b3),'_kb4_',str(k_b4),'.h5'])
        

        merged = layers.concatenate([tower_1, tower_2, tower_3], axis=1)
        merged = layers.Flatten()(merged)
        all_co = layers.Dense(100, activation='relu')(merged)
        all_co = layers.Dropout(0.5)(all_co)
        all_co = layers.Dense(50, activation='relu')(all_co)
        all_co= layers.Dropout(0.5)(all_co)
        outputs = layers.Dense(3, activation='softmax')(all_co)

        model = keras.Model(inputs=[inp1, inp2,inp3], outputs=outputs, name=model_name)
        
        return model,model_name
'''model-1'''
class model_inception_vin_1:
    
    def __init__(self,channels=17):
        'Initialization'
        self.channels = channels
        

    def parallel(self,inputs,d1,k2,k3,k4):
    
        incept_3 = layers.Conv2D(d1, (3,3),strides=(1,1),padding='same',activation='relu')(inputs)
        incept_5 = layers.Conv2D(d1, (5,5),strides=(1,1),padding='same',activation='relu')(inputs)
        incept_7 = layers.Conv2D(d1, (7,7),strides=(1,1),padding='same',activation='relu')(inputs)
        incept_9 = layers.Conv2D(d1, (9,9),strides=(1,1),padding='same',activation='relu')(inputs)
        data = layers.concatenate([incept_3, incept_5, incept_7,incept_9], axis=3)
        data = layers.MaxPooling2D(pool_size=(4, 4))(data)
        
        data_2 = layers.Conv2D(d1*4, (k2,k2),strides=(1,1),padding='same',activation='relu')(data)
        data = layers.add([data_2, data])
    
        data = layers.MaxPooling2D(pool_size=(2, 2))(data)
    
        data = layers.Conv2D(64, (k3,k3), activation='relu', padding='same')(data)
        '''to be removed'''
        data = layers.MaxPooling2D(pool_size=(2, 2))(data)
        data = layers.Conv2D(64, (k4,k4), activation='relu', padding='same')(data)
        data = layers.MaxPooling2D(pool_size=(2, 2))(data)
        return data

    def model_maker(self,d1=8,k2=3,k3=3,k4=3):
         
        inp1 = keras.Input(shape=(200, 200, self.channels))
        inp2 = keras.Input(shape=(200, 200, self.channels))
        inp3 = keras.Input(shape=(200, 200, self.channels))
        
        tower_1 = self.parallel(inp1,d1,k2,k3,k4)
        tower_2 = self.parallel(inp2,d1,k2,k3,k4)
        tower_3 = self.parallel(inp3,d1,k2,k3,k4)

        merged = layers.concatenate([tower_1, tower_2, tower_3], axis=1)
        merged = layers.Flatten()(merged)
        all_co = layers.Dense(100, activation='relu')(merged)
        all_co = layers.Dropout(0.5)(all_co)
        all_co = layers.Dense(50, activation='relu')(all_co)
        all_co= layers.Dropout(0.5)(all_co)
        outputs = layers.Dense(3, activation='softmax')(all_co)

        model_name = 'model_inception_vin_1.h5'
        model = keras.Model(inputs=[inp1, inp2,inp3], outputs=outputs, name='model_inception_vin_1')
        
        return model,model_name


'''model-2'''
class model_inception_all__depths__inception_vin_1:
    
    def __init__(self,channels=17):
        'Initialization'
        self.channels = channels
        
    def parallel(self,inputs,d1,d2,d3,d4):
    
        incept_3 = layers.Conv2D(d1, (3,3),strides=(1,1),padding='same',activation='relu')(inputs)
        incept_5 = layers.Conv2D(d1, (5,5),strides=(1,1),padding='same',activation='relu')(inputs)
        incept_7 = layers.Conv2D(d1, (7,7),strides=(1,1),padding='same',activation='relu')(inputs)
        incept_9 = layers.Conv2D(d1, (9,9),strides=(1,1),padding='same',activation='relu')(inputs)
        
        incept_3_pool = layers.MaxPooling2D(pool_size=(4, 4))(incept_3)
        incept_5_pool = layers.MaxPooling2D(pool_size=(4, 4))(incept_5)
        incept_7_pool = layers.MaxPooling2D(pool_size=(4, 4))(incept_7)
        incept_9_pool = layers.MaxPooling2D(pool_size=(4, 4))(incept_9)
        
        incept_3 = layers.Conv2D(d2, (3,3),strides=(1,1),padding='same',activation='relu')(incept_3_pool)
        incept_5 = layers.Conv2D(d2, (5,5),strides=(1,1),padding='same',activation='relu')(incept_5_pool)
        incept_7 = layers.Conv2D(d2, (7,7),strides=(1,1),padding='same',activation='relu')(incept_7_pool)
        incept_9 = layers.Conv2D(d2, (9,9),strides=(1,1),padding='same',activation='relu')(incept_9_pool)
        
        incept_3_pool = layers.MaxPooling2D(pool_size=(2, 2))(incept_3)
        incept_5_pool = layers.MaxPooling2D(pool_size=(2, 2))(incept_5)
        incept_7_pool = layers.MaxPooling2D(pool_size=(2, 2))(incept_7)
        incept_9_pool = layers.MaxPooling2D(pool_size=(2, 2))(incept_9)
        
        incept_3 = layers.Conv2D(d3, (3,3),strides=(1,1),padding='same',activation='relu')(incept_3_pool)
        incept_5 = layers.Conv2D(d3, (5,5),strides=(1,1),padding='same',activation='relu')(incept_5_pool)
        incept_7 = layers.Conv2D(d3, (7,7),strides=(1,1),padding='same',activation='relu')(incept_7_pool)
        incept_9 = layers.Conv2D(d3, (9,9),strides=(1,1),padding='same',activation='relu')(incept_9_pool)
        
        incept_3_pool = layers.MaxPooling2D(pool_size=(2, 2))(incept_3)
        incept_5_pool = layers.MaxPooling2D(pool_size=(2, 2))(incept_5)
        incept_7_pool = layers.MaxPooling2D(pool_size=(2, 2))(incept_7)
        incept_9_pool = layers.MaxPooling2D(pool_size=(2, 2))(incept_9)
        
        incept_3 = layers.Conv2D(d4, (3,3),strides=(1,1),padding='same',activation='relu')(incept_3_pool)
        incept_5 = layers.Conv2D(d4, (5,5),strides=(1,1),padding='same',activation='relu')(incept_5_pool)
        incept_7 = layers.Conv2D(d4, (7,7),strides=(1,1),padding='same',activation='relu')(incept_7_pool)
        incept_9 = layers.Conv2D(d4, (9,9),strides=(1,1),padding='same',activation='relu')(incept_9_pool)
        
        incept_3_pool = layers.MaxPooling2D(pool_size=(2, 2))(incept_3)
        incept_5_pool = layers.MaxPooling2D(pool_size=(2, 2))(incept_5)
        incept_7_pool = layers.MaxPooling2D(pool_size=(2, 2))(incept_7)
        incept_9_pool = layers.MaxPooling2D(pool_size=(2, 2))(incept_9)
         
        all_output = layers.concatenate([incept_3_pool, incept_5_pool, incept_7_pool,incept_9_pool], axis=3)
    
        return all_output

    def model_maker(self,d1=8,d2=8,d3=16,d4=16):
         
        inp1 = keras.Input(shape=(200, 200, self.channels))
        inp2 = keras.Input(shape=(200, 200, self.channels))
        inp3 = keras.Input(shape=(200, 200, self.channels))
        
        tower_1 = self.parallel(inp1,d1,d2,d3,d4)
        tower_2 = self.parallel(inp2,d1,d2,d3,d4)
        tower_3 = self.parallel(inp3,d1,d2,d3,d4)
        
        merged = layers.concatenate([tower_1, tower_2, tower_3], axis=1)
        merged = layers.Flatten()(merged)
        all_co = layers.Dense(100, activation='relu')(merged)
        all_co = layers.Dropout(0.5)(all_co)
        all_co = layers.Dense(50, activation='relu')(all_co)
        all_co= layers.Dropout(0.5)(all_co)
        outputs = layers.Dense(3, activation='softmax')(all_co)
    
        model_name = 'model_inception_all__depths__inception_vin_1.h5'
        model = keras.Model(inputs=[inp1, inp2,inp3], outputs=outputs, name='model_inception_all__depths__inception_vin_1')
        return model,model_name
    
'''model type 3'''
class model_inception_all__depths__inception_complic_vin_1:
    
    def __init__(self,channels=17):
        'Initialization'
        self.channels = channels
        
    def parallel(self,inputs,d1,d3):
    
        incept_3 = layers.Conv2D(d1, (3,3),strides=(1,1),padding='same',activation='relu')(inputs)
        incept_5 = layers.Conv2D(d1, (5,5),strides=(1,1),padding='same',activation='relu')(inputs)
        incept_7 = layers.Conv2D(d1, (7,7),strides=(1,1),padding='same',activation='relu')(inputs)
        incept_9 = layers.Conv2D(d1, (9,9),strides=(1,1),padding='same',activation='relu')(inputs)
        
        incept_3_pool = layers.MaxPooling2D(pool_size=(4, 4))(incept_3)
        incept_5_pool = layers.MaxPooling2D(pool_size=(4, 4))(incept_5)
        incept_7_pool = layers.MaxPooling2D(pool_size=(4, 4))(incept_7)
        incept_9_pool = layers.MaxPooling2D(pool_size=(4, 4))(incept_9)
        
        all_output = layers.concatenate([incept_3_pool, incept_5_pool, incept_7_pool,incept_9_pool], axis=3)
    
        incept_3 = layers.Conv2D(d1, (3,3),strides=(1,1),padding='same',activation='relu')(all_output)
        incept_5 = layers.Conv2D(d1, (5,5),strides=(1,1),padding='same',activation='relu')(all_output)
        incept_7 = layers.Conv2D(d1, (7,7),strides=(1,1),padding='same',activation='relu')(all_output)
        incept_9 = layers.Conv2D(d1, (9,9),strides=(1,1),padding='same',activation='relu')(all_output)
          
        all_output_2 = layers.concatenate([incept_3, incept_5, incept_7,incept_9], axis=3)
        all_output=  layers.add([all_output,all_output_2])
        all_output = layers.MaxPooling2D(pool_size=(2,2))(all_output)
     
        incept_3 = layers.Conv2D(d3, (3,3),strides=(1,1),padding='same',activation='relu')(all_output)
        incept_5 = layers.Conv2D(d3, (5,5),strides=(1,1),padding='same',activation='relu')(all_output)
        incept_7 = layers.Conv2D(d3, (7,7),strides=(1,1),padding='same',activation='relu')(all_output)
        incept_9 = layers.Conv2D(d3, (9,9),strides=(1,1),padding='same',activation='relu')(all_output)
        
        incept_3_pool = layers.MaxPooling2D(pool_size=(2, 2))(incept_3)
        incept_5_pool = layers.MaxPooling2D(pool_size=(2, 2))(incept_5)
        incept_7_pool = layers.MaxPooling2D(pool_size=(2, 2))(incept_7)
        incept_9_pool = layers.MaxPooling2D(pool_size=(2, 2))(incept_9)
        
        all_output = layers.concatenate([incept_3_pool, incept_5_pool, incept_7_pool,incept_9_pool], axis=3)
        
        incept_3 = layers.Conv2D(d3, (3,3),strides=(1,1),padding='same',activation='relu')(all_output)
        incept_5 = layers.Conv2D(d3, (5,5),strides=(1,1),padding='same',activation='relu')(all_output)
        incept_7 = layers.Conv2D(d3, (7,7),strides=(1,1),padding='same',activation='relu')(all_output)
        incept_9 = layers.Conv2D(d3, (9,9),strides=(1,1),padding='same',activation='relu')(all_output)
        
        all_output_4 = layers.concatenate([incept_3, incept_5, incept_7,incept_9], axis=3)
        all_output=  layers.add([all_output,all_output_4])
        all_output = layers.MaxPooling2D(pool_size=(2,2))(all_output)
    
        return all_output

    def model_maker(self,d1=8,d3=16):
         
        inp1 = keras.Input(shape=(200, 200, self.channels))
        inp2 = keras.Input(shape=(200, 200, self.channels))
        inp3 = keras.Input(shape=(200, 200, self.channels))
        
        tower_1 = self.parallel(inp1,d1,d3)
        tower_2 = self.parallel(inp2,d1,d3)
        tower_3 = self.parallel(inp3,d1,d3)
        
        merged = layers.concatenate([tower_1, tower_2, tower_3], axis=1)
        merged = layers.Flatten()(merged)
        all_co = layers.Dense(100, activation='relu')(merged)
        all_co = layers.Dropout(0.5)(all_co)
        all_co = layers.Dense(50, activation='relu')(all_co)
        all_co= layers.Dropout(0.5)(all_co)
        outputs = layers.Dense(3, activation='softmax')(all_co)
        model_name = 'model_inception_all__depths__inception_complic_vin_1.h5'
        model = keras.Model(inputs=[inp1, inp2,inp3], outputs=outputs, name='model_inception_all__depths__inception_complic_vin_1')
        return model,model_name

'''Models have parallel towers'''

'''Models with inception'''
'''model parallel tower-1'''
class model_par_inception_vin_1:
    
    def __init__(self,channels=17,tower_min_max_only=False):
        'Initialization'
        self.channels = channels
        self.tower_min_max_only = tower_min_max_only
        

    def model_maker(self,d1=8,k2=3,k3=3,k4=3):
        
        inputs = keras.Input(shape=(200, 200, self.channels))

        inp1 = keras.Input(shape=(200, 200, self.channels))
        inp2 = keras.Input(shape=(200, 200, self.channels))
        inp3 = keras.Input(shape=(200, 200, self.channels))
        
        incept_3 = layers.Conv2D(d1, (3,3),strides=(1,1),padding='same',activation='relu')(inputs)
        incept_5 = layers.Conv2D(d1, (5,5),strides=(1,1),padding='same',activation='relu')(inputs)
        incept_7 = layers.Conv2D(d1, (7,7),strides=(1,1),padding='same',activation='relu')(inputs)
        incept_9 = layers.Conv2D(d1, (9,9),strides=(1,1),padding='same',activation='relu')(inputs)
        data = layers.concatenate([incept_3, incept_5, incept_7,incept_9], axis=3)
        data = layers.MaxPooling2D(pool_size=(4, 4))(data)
        
        data_2 = layers.Conv2D(d1*4, (k2,k2),strides=(1,1),padding='same',activation='relu')(data)
        data = layers.add([data_2, data])
    
        data = layers.MaxPooling2D(pool_size=(2, 2))(data)
    
        data = layers.Conv2D(64, (k3,k3), activation='relu', padding='same')(data)
        '''to be removed'''
        data = layers.MaxPooling2D(pool_size=(2, 2))(data)
        data = layers.Conv2D(64, (k4,k4), activation='relu', padding='same')(data)
        data = layers.MaxPooling2D(pool_size=(2, 2))(data)
        
        parallel = keras.models.Model(inputs, data, name='parallel')     

        tower_1 = parallel(inp1)
        tower_2 = parallel(inp2)
        tower_3 = parallel(inp3)
       
        if self.tower_min_max_only:
            tower_max = layers.maximum([tower_1,tower_2,tower_3])
            tower_min=layers.minimum([tower_max, tower_2,tower_3])
            #tower_average=layers.average([tower_1, tower_2,tower_3])
            #tower_mul= layers.Multiply([tower_1, tower_2,tower_3])#since multiplication is expensive
            merged = layers.concatenate([tower_max, tower_min], axis=1)
            model_name = ''.join(['MIN_MAX_model_inception_vin_1_d1',str(d1),'_k2_',str(k2),'_k3_',str(k3),'_k4_',str(k4),'.h5'])

        else:
            merged = layers.concatenate([tower_1, tower_2, tower_3], axis=1)
            model_name = ''.join(['model_inception_vin_1_d1',str(d1),'_k2_',str(k2),'_k3_',str(k3),'_k4_',str(k4),'.h5'])

        merged = layers.Flatten()(merged)
        all_co = layers.Dense(100, activation='relu')(merged)
        all_co = layers.Dropout(0.5)(all_co)
        all_co = layers.Dense(50, activation='relu')(all_co)
        all_co= layers.Dropout(0.5)(all_co)
        outputs = layers.Dense(3, activation='softmax')(all_co)

        model = keras.models.Model(inputs=[inp1, inp2,inp3], outputs=outputs, name=model_name)
        
        return model,model_name
'''model parallel tower-2'''
class model_par_inception_w_o_addition_vin_1:
    
    def __init__(self,channels=17,tower_min_max_only=False):
        'Initialization'
        self.channels = channels
        self.tower_min_max_only = tower_min_max_only
        

    def model_maker(self,d1=8,k2=3,k3=3,k4=3):
        
        inputs = keras.Input(shape=(200, 200, self.channels))

        inp1 = keras.Input(shape=(200, 200, self.channels))
        inp2 = keras.Input(shape=(200, 200, self.channels))
        inp3 = keras.Input(shape=(200, 200, self.channels))
        
        incept_3 = layers.Conv2D(d1, (3,3),strides=(1,1),padding='same',activation='relu')(inputs)
        incept_5 = layers.Conv2D(d1, (5,5),strides=(1,1),padding='same',activation='relu')(inputs)
        incept_7 = layers.Conv2D(d1, (7,7),strides=(1,1),padding='same',activation='relu')(inputs)
        incept_9 = layers.Conv2D(d1, (9,9),strides=(1,1),padding='same',activation='relu')(inputs)
        data = layers.concatenate([incept_3, incept_5, incept_7,incept_9], axis=3)
        data = layers.MaxPooling2D(pool_size=(4, 4))(data)
        
        data = layers.Conv2D(d1*4, (k2,k2),strides=(1,1),padding='same',activation='relu')(data)   
        data = layers.MaxPooling2D(pool_size=(2, 2))(data)
    
        data = layers.Conv2D(64, (k3,k3), activation='relu', padding='same')(data)
        '''to be removed'''
        data = layers.MaxPooling2D(pool_size=(2, 2))(data)
        data = layers.Conv2D(64, (k4,k4), activation='relu', padding='same')(data)
        data = layers.MaxPooling2D(pool_size=(2, 2))(data)
        
        parallel = keras.models.Model(inputs, data, name='parallel')     

        tower_1 = parallel(inp1)
        tower_2 = parallel(inp2)
        tower_3 = parallel(inp3)
        
        if self.tower_min_max_only:
            tower_max = layers.maximum([tower_1,tower_2,tower_3])
            tower_min=layers.minimum([tower_max, tower_2,tower_3])
            #tower_average=layers.average([tower_1, tower_2,tower_3])
            #tower_mul= layers.Multiply([tower_1, tower_2,tower_3])#since multiplication is expensive
            merged = layers.concatenate([tower_max, tower_min], axis=1)
            model_name = ''.join(['MIN_MAX_model_inception_w_o_addition_vin_1_d1',str(d1),'_k2_',str(k2),'_k3_',str(k3),'_k4_',str(k4),'.h5'])

        else:
            merged = layers.concatenate([tower_1, tower_2, tower_3], axis=1)
            model_name = ''.join(['model_inception_w_o_addition_vin_1_d1',str(d1),'_k2_',str(k2),'_k3_',str(k3),'_k4_',str(k4),'.h5'])

        merged = layers.Flatten()(merged)
        all_co = layers.Dense(100, activation='relu')(merged)
        all_co = layers.Dropout(0.5)(all_co)
        all_co = layers.Dense(50, activation='relu')(all_co)
        all_co= layers.Dropout(0.5)(all_co)
        outputs = layers.Dense(3, activation='softmax')(all_co)

        model = keras.models.Model(inputs=[inp1, inp2,inp3], outputs=outputs, name=model_name)
        
        return model,model_name

'''model parallel tower -3'''
class model_par_inception_all__depths__inception_vin_1:
    
    def __init__(self,channels=17,tower_min_max_only=False):
        'Initialization'
        self.channels = channels
        self.tower_min_max_only = tower_min_max_only
        
    def model_maker(self,d1=4,d2=4,d3=8,d4=8):
        inputs = keras.Input(shape=(200, 200, self.channels))

        inp1 = keras.Input(shape=(200, 200, self.channels))
        inp2 = keras.Input(shape=(200, 200, self.channels))
        inp3 = keras.Input(shape=(200, 200, self.channels))    
       
        incept_3 = layers.Conv2D(d1, (3,3),strides=(1,1),padding='same',activation='relu')(inputs)
        incept_5 = layers.Conv2D(d1, (5,5),strides=(1,1),padding='same',activation='relu')(inputs)
        incept_7 = layers.Conv2D(d1, (7,7),strides=(1,1),padding='same',activation='relu')(inputs)
        incept_9 = layers.Conv2D(d1, (9,9),strides=(1,1),padding='same',activation='relu')(inputs)
        
        incept_3_pool = layers.MaxPooling2D(pool_size=(4, 4))(incept_3)
        incept_5_pool = layers.MaxPooling2D(pool_size=(4, 4))(incept_5)
        incept_7_pool = layers.MaxPooling2D(pool_size=(4, 4))(incept_7)
        incept_9_pool = layers.MaxPooling2D(pool_size=(4, 4))(incept_9)
        
        incept_3 = layers.Conv2D(d2, (3,3),strides=(1,1),padding='same',activation='relu')(incept_3_pool)
        incept_5 = layers.Conv2D(d2, (5,5),strides=(1,1),padding='same',activation='relu')(incept_5_pool)
        incept_7 = layers.Conv2D(d2, (7,7),strides=(1,1),padding='same',activation='relu')(incept_7_pool)
        incept_9 = layers.Conv2D(d2, (9,9),strides=(1,1),padding='same',activation='relu')(incept_9_pool)
        
        incept_3_pool = layers.MaxPooling2D(pool_size=(2, 2))(incept_3)
        incept_5_pool = layers.MaxPooling2D(pool_size=(2, 2))(incept_5)
        incept_7_pool = layers.MaxPooling2D(pool_size=(2, 2))(incept_7)
        incept_9_pool = layers.MaxPooling2D(pool_size=(2, 2))(incept_9)
        
        incept_3 = layers.Conv2D(d3, (3,3),strides=(1,1),padding='same',activation='relu')(incept_3_pool)
        incept_5 = layers.Conv2D(d3, (5,5),strides=(1,1),padding='same',activation='relu')(incept_5_pool)
        incept_7 = layers.Conv2D(d3, (7,7),strides=(1,1),padding='same',activation='relu')(incept_7_pool)
        incept_9 = layers.Conv2D(d3, (9,9),strides=(1,1),padding='same',activation='relu')(incept_9_pool)
        
        incept_3_pool = layers.MaxPooling2D(pool_size=(2, 2))(incept_3)
        incept_5_pool = layers.MaxPooling2D(pool_size=(2, 2))(incept_5)
        incept_7_pool = layers.MaxPooling2D(pool_size=(2, 2))(incept_7)
        incept_9_pool = layers.MaxPooling2D(pool_size=(2, 2))(incept_9)
        
        incept_3 = layers.Conv2D(d4, (3,3),strides=(1,1),padding='same',activation='relu')(incept_3_pool)
        incept_5 = layers.Conv2D(d4, (5,5),strides=(1,1),padding='same',activation='relu')(incept_5_pool)
        incept_7 = layers.Conv2D(d4, (7,7),strides=(1,1),padding='same',activation='relu')(incept_7_pool)
        incept_9 = layers.Conv2D(d4, (9,9),strides=(1,1),padding='same',activation='relu')(incept_9_pool)
        
        incept_3_pool = layers.MaxPooling2D(pool_size=(2, 2))(incept_3)
        incept_5_pool = layers.MaxPooling2D(pool_size=(2, 2))(incept_5)
        incept_7_pool = layers.MaxPooling2D(pool_size=(2, 2))(incept_7)
        incept_9_pool = layers.MaxPooling2D(pool_size=(2, 2))(incept_9)
         
        data = layers.concatenate([incept_3_pool, incept_5_pool, incept_7_pool,incept_9_pool], axis=3)

    
        parallel = keras.models.Model(inputs, data, name='parallel')     

        tower_1 = parallel(inp1)
        tower_2 = parallel(inp2)
        tower_3 = parallel(inp3)
        
        if self.tower_min_max_only:
            tower_max = layers.maximum([tower_1,tower_2,tower_3])
            tower_min=layers.minimum([tower_max, tower_2,tower_3])
            #tower_average=layers.average([tower_1, tower_2,tower_3])
            #tower_mul= layers.Multiply([tower_1, tower_2,tower_3])#since multiplication is expensive
            merged = layers.concatenate([tower_max, tower_min], axis=1)
            model_name = ''.join(['MIN_MAX_model_inception_all__depths__inception_vin_1_d1',str(d1),'_d2_',str(d2),'_d3_',str(d3),'_d4_',str(d4),'.h5'])

        else:
            merged = layers.concatenate([tower_1, tower_2, tower_3], axis=1)
            model_name = ''.join(['model_inception_all__depths__inception_vin_1_d1',str(d1),'_d2_',str(d2),'_d3_',str(d3),'_d4_',str(d4),'.h5'])
        
        merged = layers.Flatten()(merged)
        all_co = layers.Dense(100, activation='relu')(merged)
        all_co = layers.Dropout(0.5)(all_co)
        all_co = layers.Dense(50, activation='relu')(all_co)
        all_co= layers.Dropout(0.5)(all_co)
        outputs = layers.Dense(3, activation='softmax')(all_co)

        model = keras.models.Model(inputs=[inp1, inp2,inp3], outputs=outputs, name=model_name)
    
        return model,model_name


'''model parallel tower -4'''
class model_par_parallel_inception_all__depths_min_max_inception_vin_1:
    
    def __init__(self,channels=17,tower_min_max_only=False):
        'Initialization'
        self.channels = channels
        self.tower_min_max_only = tower_min_max_only
        
    def model_maker(self,d1=8,d2=8,d3=16,d4=16):
        inputs = keras.Input(shape=(200, 200, self.channels))

        inp1 = keras.Input(shape=(200, 200, self.channels))
        inp2 = keras.Input(shape=(200, 200, self.channels))
        inp3 = keras.Input(shape=(200, 200, self.channels))    
       
        incept_3 = layers.Conv2D(d1, (3,3),strides=(1,1),padding='same',activation='relu')(inputs)
        incept_5 = layers.Conv2D(d1, (5,5),strides=(1,1),padding='same',activation='relu')(inputs)
        incept_7 = layers.Conv2D(d1, (7,7),strides=(1,1),padding='same',activation='relu')(inputs)
        incept_9 = layers.Conv2D(d1, (9,9),strides=(1,1),padding='same',activation='relu')(inputs)
        
        incept_3_pool = layers.MaxPooling2D(pool_size=(4, 4))(incept_3)
        incept_5_pool = layers.MaxPooling2D(pool_size=(4, 4))(incept_5)
        incept_7_pool = layers.MaxPooling2D(pool_size=(4, 4))(incept_7)
        incept_9_pool = layers.MaxPooling2D(pool_size=(4, 4))(incept_9)
        
        all_output_max = layers.maximum([incept_3_pool, incept_5_pool, incept_7_pool,incept_9_pool])
        all_output_min =layers.minimum([incept_3_pool, incept_5_pool, incept_7_pool,incept_9_pool])
        all_output = layers.concatenate([all_output_max, all_output_min], axis=3)
        
        incept_3 = layers.Conv2D(d2, (3,3),strides=(1,1),padding='same',activation='relu')(all_output)
        incept_5 = layers.Conv2D(d2, (5,5),strides=(1,1),padding='same',activation='relu')(all_output)
        incept_7 = layers.Conv2D(d2, (7,7),strides=(1,1),padding='same',activation='relu')(all_output)
        incept_9 = layers.Conv2D(d2, (9,9),strides=(1,1),padding='same',activation='relu')(all_output)
        
        incept_3_pool = layers.MaxPooling2D(pool_size=(2, 2))(incept_3)
        incept_5_pool = layers.MaxPooling2D(pool_size=(2, 2))(incept_5)
        incept_7_pool = layers.MaxPooling2D(pool_size=(2, 2))(incept_7)
        incept_9_pool = layers.MaxPooling2D(pool_size=(2, 2))(incept_9)
        
        all_output_max = layers.maximum([incept_3_pool, incept_5_pool, incept_7_pool,incept_9_pool])
        all_output_min =layers.minimum([incept_3_pool, incept_5_pool, incept_7_pool,incept_9_pool])
        all_output = layers.concatenate([all_output_max, all_output_min], axis=3)
        
        incept_3 = layers.Conv2D(d3, (3,3),strides=(1,1),padding='same',activation='relu')(all_output)
        incept_5 = layers.Conv2D(d3, (5,5),strides=(1,1),padding='same',activation='relu')(all_output)
        incept_7 = layers.Conv2D(d3, (7,7),strides=(1,1),padding='same',activation='relu')(all_output)
        incept_9 = layers.Conv2D(d3, (9,9),strides=(1,1),padding='same',activation='relu')(all_output)
        
        incept_3_pool = layers.MaxPooling2D(pool_size=(2, 2))(incept_3)
        incept_5_pool = layers.MaxPooling2D(pool_size=(2, 2))(incept_5)
        incept_7_pool = layers.MaxPooling2D(pool_size=(2, 2))(incept_7)
        incept_9_pool = layers.MaxPooling2D(pool_size=(2, 2))(incept_9)
        
        all_output_max = layers.maximum([incept_3_pool, incept_5_pool, incept_7_pool,incept_9_pool])
        all_output_min =layers.minimum([incept_3_pool, incept_5_pool, incept_7_pool,incept_9_pool])
        all_output = layers.concatenate([all_output_max, all_output_min], axis=3)
        
        incept_3 = layers.Conv2D(d4, (3,3),strides=(1,1),padding='same',activation='relu')(all_output)
        incept_5 = layers.Conv2D(d4, (5,5),strides=(1,1),padding='same',activation='relu')(all_output)
        incept_7 = layers.Conv2D(d4, (7,7),strides=(1,1),padding='same',activation='relu')(all_output)
        incept_9 = layers.Conv2D(d4, (9,9),strides=(1,1),padding='same',activation='relu')(all_output)
        
        incept_3_pool = layers.MaxPooling2D(pool_size=(2, 2))(incept_3)
        incept_5_pool = layers.MaxPooling2D(pool_size=(2, 2))(incept_5)
        incept_7_pool = layers.MaxPooling2D(pool_size=(2, 2))(incept_7)
        incept_9_pool = layers.MaxPooling2D(pool_size=(2, 2))(incept_9)
         
        all_output_max = layers.maximum([incept_3_pool, incept_5_pool, incept_7_pool,incept_9_pool])
        all_output_min =layers.minimum([incept_3_pool, incept_5_pool, incept_7_pool,incept_9_pool])
        all_output = layers.concatenate([all_output_max, all_output_min], axis=3)
    
        parallel = keras.models.Model(inputs, all_output, name='parallel')     

        tower_1 = parallel(inp1)
        tower_2 = parallel(inp2)
        tower_3 = parallel(inp3)
        
        if self.tower_min_max_only:
            tower_max = layers.maximum([tower_1,tower_2,tower_3])
            tower_min=layers.minimum([tower_max, tower_2,tower_3])
            #tower_average=layers.average([tower_1, tower_2,tower_3])
            #tower_mul= layers.Multiply([tower_1, tower_2,tower_3])#since multiplication is expensive
            merged = layers.concatenate([tower_max, tower_min], axis=1)
            model_name = ''.join(['MIN_MAX_model_inception_all__depths__inception_vin_1_d1',str(d1),'_d2_',str(d2),'_d3_',str(d3),'_d4_',str(d4),'.h5'])

        else:
            merged = layers.concatenate([tower_1, tower_2, tower_3], axis=1)
            model_name = ''.join(['model_inception_all__depths__inception_vin_1_d1',str(d1),'_d2_',str(d2),'_d3_',str(d3),'_d4_',str(d4),'.h5'])
        
        merged = layers.Flatten()(merged)
        all_co = layers.Dense(100, activation='relu')(merged)
        all_co = layers.Dropout(0.5)(all_co)
        all_co = layers.Dense(50, activation='relu')(all_co)
        all_co= layers.Dropout(0.5)(all_co)
        outputs = layers.Dense(3, activation='softmax')(all_co)

        model = keras.models.Model(inputs=[inp1, inp2,inp3], outputs=outputs, name=model_name)
    
        return model,model_name

    
'''model parallel tower -5'''
class model_par_inception_all__depths__inception_complic_vin_1:
    
    def __init__(self,channels=17,tower_min_max_only=False):
        'Initialization'
        self.channels = channels
        self.tower_min_max_only = tower_min_max_only
        

    def model_maker(self,d1=8,d3=16):
                
        inputs = keras.Input(shape=(200, 200, self.channels))

        inp1 = keras.Input(shape=(200, 200, self.channels))
        inp2 = keras.Input(shape=(200, 200, self.channels))
        inp3 = keras.Input(shape=(200, 200, self.channels))   
        
        incept_3 = layers.Conv2D(d1, (3,3),strides=(1,1),padding='same',activation='relu')(inputs)
        incept_5 = layers.Conv2D(d1, (5,5),strides=(1,1),padding='same',activation='relu')(inputs)
        incept_7 = layers.Conv2D(d1, (7,7),strides=(1,1),padding='same',activation='relu')(inputs)
        incept_9 = layers.Conv2D(d1, (9,9),strides=(1,1),padding='same',activation='relu')(inputs)
        
        incept_3_pool = layers.MaxPooling2D(pool_size=(4, 4))(incept_3)
        incept_5_pool = layers.MaxPooling2D(pool_size=(4, 4))(incept_5)
        incept_7_pool = layers.MaxPooling2D(pool_size=(4, 4))(incept_7)
        incept_9_pool = layers.MaxPooling2D(pool_size=(4, 4))(incept_9)
        
        all_output = layers.concatenate([incept_3_pool, incept_5_pool, incept_7_pool,incept_9_pool], axis=3)
    
        incept_3 = layers.Conv2D(d1, (3,3),strides=(1,1),padding='same',activation='relu')(all_output)
        incept_5 = layers.Conv2D(d1, (5,5),strides=(1,1),padding='same',activation='relu')(all_output)
        incept_7 = layers.Conv2D(d1, (7,7),strides=(1,1),padding='same',activation='relu')(all_output)
        incept_9 = layers.Conv2D(d1, (9,9),strides=(1,1),padding='same',activation='relu')(all_output)
          
        all_output_2 = layers.concatenate([incept_3, incept_5, incept_7,incept_9], axis=3)
        all_output=  layers.add([all_output,all_output_2])
        all_output = layers.MaxPooling2D(pool_size=(2,2))(all_output)
     
        incept_3 = layers.Conv2D(d3, (3,3),strides=(1,1),padding='same',activation='relu')(all_output)
        incept_5 = layers.Conv2D(d3, (5,5),strides=(1,1),padding='same',activation='relu')(all_output)
        incept_7 = layers.Conv2D(d3, (7,7),strides=(1,1),padding='same',activation='relu')(all_output)
        incept_9 = layers.Conv2D(d3, (9,9),strides=(1,1),padding='same',activation='relu')(all_output)
        
        incept_3_pool = layers.MaxPooling2D(pool_size=(2, 2))(incept_3)
        incept_5_pool = layers.MaxPooling2D(pool_size=(2, 2))(incept_5)
        incept_7_pool = layers.MaxPooling2D(pool_size=(2, 2))(incept_7)
        incept_9_pool = layers.MaxPooling2D(pool_size=(2, 2))(incept_9)
        
        all_output = layers.concatenate([incept_3_pool, incept_5_pool, incept_7_pool,incept_9_pool], axis=3)
        
        incept_3 = layers.Conv2D(d3, (3,3),strides=(1,1),padding='same',activation='relu')(all_output)
        incept_5 = layers.Conv2D(d3, (5,5),strides=(1,1),padding='same',activation='relu')(all_output)
        incept_7 = layers.Conv2D(d3, (7,7),strides=(1,1),padding='same',activation='relu')(all_output)
        incept_9 = layers.Conv2D(d3, (9,9),strides=(1,1),padding='same',activation='relu')(all_output)
        
        all_output_4 = layers.concatenate([incept_3, incept_5, incept_7,incept_9], axis=3)
        all_output=  layers.add([all_output,all_output_4])
        all_output = layers.MaxPooling2D(pool_size=(2,2))(all_output)

        
        parallel = keras.models.Model(inputs, all_output, name='parallel')     

        tower_1 = parallel(inp1)
        tower_2 = parallel(inp2)
        tower_3 = parallel(inp3)
        
        if self.tower_min_max_only:
            tower_max = layers.maximum([tower_1,tower_2,tower_3])
            tower_min=layers.minimum([tower_max, tower_2,tower_3])
            #tower_average=layers.average([tower_1, tower_2,tower_3])
            #tower_mul= layers.Multiply([tower_1, tower_2,tower_3])#since multiplication is expensive
            merged = layers.concatenate([tower_max, tower_min], axis=1)
            model_name = ''.join(['MIN_MAX_model_inception_all__depths__inception_complic_vin_1_d1',str(d1),'_d3_',str(d3),'.h5'])

        else:
            merged = layers.concatenate([tower_1, tower_2, tower_3], axis=1)
            model_name = ''.join(['model_inception_all__depths__inception_complic_vin_1_d1',str(d1),'_d3_',str(d3),'.h5'])

        merged = layers.Flatten()(merged)
        all_co = layers.Dense(100, activation='relu')(merged)
        all_co = layers.Dropout(0.5)(all_co)
        all_co = layers.Dense(50, activation='relu')(all_co)
        all_co= layers.Dropout(0.5)(all_co)
        outputs = layers.Dense(3, activation='softmax')(all_co)

        model = keras.models.Model(inputs=[inp1, inp2,inp3], outputs=outputs, name=model_name)
    
        return model,model_name

'''model parallel tower -6'''
class model_par_parallel_inception_all__depths_min_max_inception_complic_vin_1:
    
    def __init__(self,channels=17,tower_min_max_only=False):
        'Initialization'
        self.channels = channels
        self.tower_min_max_only = tower_min_max_only
        

    def model_maker(self,d1=8,d3=16):
                
        inputs = keras.Input(shape=(200, 200, self.channels))

        inp1 = keras.Input(shape=(200, 200, self.channels))
        inp2 = keras.Input(shape=(200, 200, self.channels))
        inp3 = keras.Input(shape=(200, 200, self.channels))   
        
        incept_3 = layers.Conv2D(d1, (3,3),strides=(1,1),padding='same',activation='relu')(inputs)
        incept_5 = layers.Conv2D(d1, (5,5),strides=(1,1),padding='same',activation='relu')(inputs)
        incept_7 = layers.Conv2D(d1, (7,7),strides=(1,1),padding='same',activation='relu')(inputs)
        incept_9 = layers.Conv2D(d1, (9,9),strides=(1,1),padding='same',activation='relu')(inputs)
        
        incept_3_pool = layers.MaxPooling2D(pool_size=(4, 4))(incept_3)
        incept_5_pool = layers.MaxPooling2D(pool_size=(4, 4))(incept_5)
        incept_7_pool = layers.MaxPooling2D(pool_size=(4, 4))(incept_7)
        incept_9_pool = layers.MaxPooling2D(pool_size=(4, 4))(incept_9)
        
        all_output_max = layers.maximum([incept_3_pool, incept_5_pool, incept_7_pool,incept_9_pool])
        all_output_min =layers.minimum([incept_3_pool, incept_5_pool, incept_7_pool,incept_9_pool])
        all_output = layers.concatenate([all_output_max, all_output_min], axis=3)
        
        incept_3 = layers.Conv2D(d1, (3,3),strides=(1,1),padding='same',activation='relu')(all_output)
        incept_5 = layers.Conv2D(d1, (5,5),strides=(1,1),padding='same',activation='relu')(all_output)
        incept_7 = layers.Conv2D(d1, (7,7),strides=(1,1),padding='same',activation='relu')(all_output)
        incept_9 = layers.Conv2D(d1, (9,9),strides=(1,1),padding='same',activation='relu')(all_output)
        
        all_output_2_max = layers.maximum([incept_3, incept_5, incept_7,incept_9]) 
        all_output_2_min =layers.minimum([incept_3, incept_5, incept_7,incept_9]) 
        all_output_2 = layers.concatenate([all_output_2_max, all_output_2_min], axis=3)
        
        all_output=  layers.add([all_output,all_output_2])
        all_output = layers.MaxPooling2D(pool_size=(2,2))(all_output)
         
        incept_3 = layers.Conv2D(d3, (3,3),strides=(1,1),padding='same',activation='relu')(all_output)
        incept_5 = layers.Conv2D(d3, (5,5),strides=(1,1),padding='same',activation='relu')(all_output)
        incept_7 = layers.Conv2D(d3, (7,7),strides=(1,1),padding='same',activation='relu')(all_output)
        incept_9 = layers.Conv2D(d3, (9,9),strides=(1,1),padding='same',activation='relu')(all_output)
        
        incept_3_pool = layers.MaxPooling2D(pool_size=(2, 2))(incept_3)
        incept_5_pool = layers.MaxPooling2D(pool_size=(2, 2))(incept_5)
        incept_7_pool = layers.MaxPooling2D(pool_size=(2, 2))(incept_7)
        incept_9_pool = layers.MaxPooling2D(pool_size=(2, 2))(incept_9)
        
        all_output_max = layers.maximum([incept_3_pool, incept_5_pool, incept_7_pool,incept_9_pool])
        all_output_min =layers.minimum([incept_3_pool, incept_5_pool, incept_7_pool,incept_9_pool])
        all_output = layers.concatenate([all_output_max, all_output_min], axis=3)
        
        incept_3 = layers.Conv2D(d3, (3,3),strides=(1,1),padding='same',activation='relu')(all_output)
        incept_5 = layers.Conv2D(d3, (5,5),strides=(1,1),padding='same',activation='relu')(all_output)
        incept_7 = layers.Conv2D(d3, (7,7),strides=(1,1),padding='same',activation='relu')(all_output)
        incept_9 = layers.Conv2D(d3, (9,9),strides=(1,1),padding='same',activation='relu')(all_output)
        
        all_output_2_max = layers.maximum([incept_3, incept_5, incept_7,incept_9]) 
        all_output_2_min =layers.minimum([incept_3, incept_5, incept_7,incept_9]) 
        all_output_2 = layers.concatenate([all_output_2_max, all_output_2_min], axis=3)
        
        all_output=  layers.add([all_output,all_output_2])
        all_output = layers.MaxPooling2D(pool_size=(2,2))(all_output)

        
        parallel = keras.models.Model(inputs, all_output, name='parallel')     

        tower_1 = parallel(inp1)
        tower_2 = parallel(inp2)
        tower_3 = parallel(inp3)
        
        if self.tower_min_max_only:
            tower_max = layers.maximum([tower_1,tower_2,tower_3])
            tower_min=layers.minimum([tower_max, tower_2,tower_3])
            #tower_average=layers.average([tower_1, tower_2,tower_3])
            #tower_mul= layers.Multiply([tower_1, tower_2,tower_3])#since multiplication is expensive
            merged = layers.concatenate([tower_max, tower_min], axis=1)
            model_name = ''.join(['MIN_MAX_model_parallel_inception_all__depths_min_max_inception_complic_vin_1_d1',str(d1),'_d3_',str(d3),'.h5'])
        else:
            merged = layers.concatenate([tower_1, tower_2, tower_3], axis=1)
            model_name = ''.join(['model_parallel_inception_all__depths_min_max_inception_complic_vin_1_d1',str(d1),'_d3_',str(d3),'.h5'])

        merged = layers.Flatten()(merged)
        all_co = layers.Dense(100, activation='relu')(merged)
        all_co = layers.Dropout(0.5)(all_co)
        all_co = layers.Dense(50, activation='relu')(all_co)
        all_co= layers.Dropout(0.5)(all_co)
        outputs = layers.Dense(3, activation='softmax')(all_co)

        model = keras.models.Model(inputs=[inp1, inp2,inp3], outputs=outputs, name=model_name)
    
        return model,model_name
    
class model_par_inception_w_o_addition_vin_exp:
    
    def __init__(self,channels=17,tower_min_max_only=False):
        'Initialization'
        self.channels = channels
        self.tower_min_max_only = tower_min_max_only       

    def model_maker(self,d1=96,d2=32,k2=3,k3=3,k4=3):#or d1=64 (32 x 3)
        
        inputs = keras.Input(shape=(200, 200, self.channels))

        inp1 = keras.Input(shape=(200, 200, self.channels))
        inp2 = keras.Input(shape=(200, 200, self.channels))
        inp3 = keras.Input(shape=(200, 200, self.channels))
        
        incept_3 = layers.Conv2D(d1, (3,3),strides=(1,1),padding='same',activation='relu')(inputs)
        incept_5 = layers.Conv2D(d2, (5,5),strides=(1,1),padding='same',activation='relu')(inputs)
        data = layers.concatenate([incept_3, incept_5], axis=3)
        data = layers.MaxPooling2D(pool_size=(4, 4))(data)
        
        data = layers.Conv2D(d1+d2, (k2,k2),strides=(1,1),padding='same',activation='relu')(data)   
        data = layers.MaxPooling2D(pool_size=(2, 2))(data)
    
        data = layers.Conv2D(d1+d2, (k3,k3), activation='relu', padding='same')(data)
        '''to be removed'''
        data = layers.MaxPooling2D(pool_size=(2, 2))(data)
        data = layers.Conv2D(64, (k4,k4), activation='relu', padding='same')(data)
        data = layers.MaxPooling2D(pool_size=(2, 2))(data)
        
        parallel = keras.models.Model(inputs, data, name='parallel')     

        tower_1 = parallel(inp1)
        tower_2 = parallel(inp2)
        tower_3 = parallel(inp3)
        
        if self.tower_min_max_only:
            tower_max = layers.maximum([tower_1,tower_2,tower_3])
            tower_min=layers.minimum([tower_max, tower_2,tower_3])
            #tower_average=layers.average([tower_1, tower_2,tower_3])
            #tower_mul= layers.Multiply([tower_1, tower_2,tower_3])#since multiplication is expensive
            merged = layers.concatenate([tower_max, tower_min], axis=1)
            model_name = ''.join(['MIN_MAX_model_par_inception_w_o_addition_vin_exp_d1_',str(d1),'_d2',str(d2),'_k2_',str(k2),'_k3_',str(k3),'_k4_',str(k4),'.h5'])

        else:
            merged = layers.concatenate([tower_1, tower_2, tower_3], axis=1)
            model_name = ''.join(['model_par_inception_w_o_addition_vin_exp_d1_',str(d1),'_d2_',str(d2),'_k2_',str(k2),'_k3_',str(k3),'_k4_',str(k4),'.h5'])

        merged = layers.Flatten()(merged)
        all_co = layers.Dense(100, activation='relu')(merged)
        all_co = layers.Dropout(0.5)(all_co)
        all_co = layers.Dense(50, activation='relu')(all_co)
        all_co= layers.Dropout(0.5)(all_co)
        outputs = layers.Dense(3, activation='softmax')(all_co)

        model = keras.models.Model(inputs=[inp1, inp2,inp3], outputs=outputs, name=model_name)
        
        return model,model_name

class model_par_parallel_inception_all_k3x3_k_5_depths_min_max_inception_vin_1:
    
    def __init__(self,channels=17,tower_min_max_only=False):
        'Initialization'
        self.channels = channels
        self.tower_min_max_only = tower_min_max_only
        
    def model_maker(self,d3=96,d5=32):
        inputs = keras.Input(shape=(200, 200, self.channels))

        inp1 = keras.Input(shape=(200, 200, self.channels))
        inp2 = keras.Input(shape=(200, 200, self.channels))
        inp3 = keras.Input(shape=(200, 200, self.channels))    
       
        incept_3 = layers.Conv2D(d3, (3,3),strides=(1,1),padding='same',activation='relu')(inputs)
        incept_5 = layers.Conv2D(d5, (5,5),strides=(1,1),padding='same',activation='relu')(inputs)
        
        incept_3_pool = layers.MaxPooling2D(pool_size=(4, 4))(incept_3)
        incept_5_pool = layers.MaxPooling2D(pool_size=(4, 4))(incept_5)

        all_output = layers.concatenate([incept_3_pool, incept_5_pool], axis=3)
        
        incept_3 = layers.Conv2D(d3, (3,3),strides=(1,1),padding='same',activation='relu')(all_output)
        incept_5 = layers.Conv2D(d5, (5,5),strides=(1,1),padding='same',activation='relu')(all_output)
        
        incept_3_pool = layers.MaxPooling2D(pool_size=(2, 2))(incept_3)
        incept_5_pool = layers.MaxPooling2D(pool_size=(2, 2))(incept_5)

        all_output = layers.concatenate([incept_3_pool, incept_5_pool], axis=3)
        
        incept_3 = layers.Conv2D(2*d3, (3,3),strides=(1,1),padding='same',activation='relu')(all_output)
        incept_5 = layers.Conv2D(2*d5, (5,5),strides=(1,1),padding='same',activation='relu')(all_output)
        
        incept_3_pool = layers.MaxPooling2D(pool_size=(2, 2))(incept_3)
        incept_5_pool = layers.MaxPooling2D(pool_size=(2, 2))(incept_5)

        all_output = layers.concatenate([incept_3_pool, incept_5_pool], axis=3)
        
        incept_3 = layers.Conv2D(2*d3, (3,3),strides=(1,1),padding='same',activation='relu')(all_output)
        incept_5 = layers.Conv2D(2*d5, (5,5),strides=(1,1),padding='same',activation='relu')(all_output)
        
        incept_3_pool = layers.MaxPooling2D(pool_size=(2, 2))(incept_3)
        incept_5_pool = layers.MaxPooling2D(pool_size=(2, 2))(incept_5)

        all_output = layers.concatenate([incept_3_pool, incept_5_pool], axis=3)
    
        parallel = keras.models.Model(inputs, all_output, name='parallel')     

        tower_1 = parallel(inp1)
        tower_2 = parallel(inp2)
        tower_3 = parallel(inp3)
        
        if self.tower_min_max_only:
            tower_max = layers.maximum([tower_1,tower_2,tower_3])
            tower_min=layers.minimum([tower_max, tower_2,tower_3])
            #tower_average=layers.average([tower_1, tower_2,tower_3])
            #tower_mul= layers.Multiply([tower_1, tower_2,tower_3])#since multiplication is expensive
            merged = layers.concatenate([tower_max, tower_min], axis=1)
            model_name = ''.join(['MIN_MAX_model_par_parallel_inception_all_k3x3_k_5_depths_min_max_inception_vin_1_d3_',str(d3),'_d5_',str(d5),'.h5'])

        else:
            merged = layers.concatenate([tower_1, tower_2, tower_3], axis=1)
            model_name = ''.join(['model_par_parallel_inception_all_k3x3_k_5_depths_min_max_inception_vin_1_d3_',str(d3),'_d5_',str(d5),'.h5'])
        
        merged = layers.Flatten()(merged)
        all_co = layers.Dense(100, activation='relu')(merged)
        all_co = layers.Dropout(0.5)(all_co)
        all_co = layers.Dense(50, activation='relu')(all_co)
        all_co= layers.Dropout(0.5)(all_co)
        outputs = layers.Dense(3, activation='softmax')(all_co)

        model = keras.models.Model(inputs=[inp1, inp2,inp3], outputs=outputs, name=model_name)
    
        return model,model_name    

ModuleNotFoundError: No module named 'tensorflow'

In [1]:

import numpy as np
import keras
from copy import deepcopy
import pickle
import os
import random

class DataGenerator_splited:
    'Generates 2D projected data for Keras'
    def __init__(self, labels,main_directory, batch_size=32, dim = (200,200), n_channels=17 ,n_classes=3,iter_train=199, shuffle=True,Full_clean=True,balance_batch_must=True,validation_split=0.2):
        '''Initialization
        labels
        main_directory:
        batch_size=32
        dim = (200,200)
        n_channels=17 
        n_classes=3
        iter_train=199
        shuffle=True
        Full_clean=True
        balance_batch_must: True make sure all batches has same class distribution
                            False not forcing the all batches have same class distribution
        validation_split: Fraction split the validation data from the whole training set
                        Default value is 20%
                     **   Especially the TSG split is same as OG split size
                       **  and the Fusion split is 0.5 x Split persent to avoid models biased to OG
        '''
        self.batch_size = batch_size
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.dim = dim
        self.Full_clean= Full_clean
        self.iter_train=iter_train
        self.labels=labels
        self.main_directory= main_directory
        self.balance_batch_must=balance_batch_must
        self.validation_split=validation_split
     
        os.chdir('/')
        os.chdir(main_directory)    
        if validation_split>0:
            #split the training and validation data
            #first load the all training data
            all_OG_train_PDBs = pickle.load(open("OG_train_PDBs.p", "rb")) 
            all_TSG_train_PDBs = pickle.load(open("TSG_train_PDBs.p", "rb"))      
            all_Fusion_train_PDBs = pickle.load(open("Fusion_train_PDBs.p", "rb"))  
            
            print('')
            print('Over all training data in:')
            print('OG: ',len(all_OG_train_PDBs))
            print('TSG: ',len(all_TSG_train_PDBs))        
            print('Fusion: ',len(all_Fusion_train_PDBs))    
            #to avoid validation class has overlapping classes
            self.overlapped_PDBs_ONGO = pickle.load(open("overlapped_PDBs_ONGO.p", "rb"))    
            self.overlapped_PDBs_TSG = pickle.load(open("overlapped_PDBs_TSG.p", "rb"))  
            # shuffle the data first
            chk_OG=deepcopy(list(range(len(all_OG_train_PDBs))))
            chk_TSG=deepcopy(list(range(len(all_TSG_train_PDBs))))
            chk_Fusion=deepcopy(list(range(len(all_Fusion_train_PDBs))))    
            if shuffle:
                random.shuffle(chk_OG)
                random.shuffle(chk_TSG)
                random.shuffle(chk_Fusion)    

            validation_OG = []
            over_chk_OG=0
            # print(' self.overlapped_PDBs_ONGO: ',len(self.overlapped_PDBs_ONGO))
            # print('all_OG_train_PDBs: ',len(all_OG_train_PDBs))
            # print('len chk_OG: ',len(chk_OG))
            OG_valid_split_size = int(validation_split*len(chk_OG))
            for i in range(0,OG_valid_split_size):
                # print(' all_OG_train_PDBs[chk_OG[i+over_chk_OG]] i: ',i,'over_chk_OG ',over_chk_OG)
                if all_OG_train_PDBs[chk_OG[i+over_chk_OG]] not in self.overlapped_PDBs_ONGO:
                    validation_OG.append(all_OG_train_PDBs.pop(chk_OG[i+over_chk_OG]))
                else:
                    while (all_OG_train_PDBs[chk_OG[i+over_chk_OG]] in self.overlapped_PDBs_ONGO):
                        over_chk_OG=over_chk_OG+1
                    validation_OG.append(all_OG_train_PDBs.pop(chk_OG[i+over_chk_OG]))         
                chk_OG = self.pop_range_fix(chk_OG,chk_OG[i+over_chk_OG])

            self.validation_OG=validation_OG
            self.OG_train_PDBs = all_OG_train_PDBs
            print('validation creation ONGO done :)')
            print('')
            print('Training data size OG: ',len(self.OG_train_PDBs))
            print('Validati data size OG: ',len(validation_OG))
            print('')

            validation_TSG=[]
            over_chk_TSG=0
            # print(int(validation_split*len(chk_TSG)))
            # print('overlapped_PDBs_TSG: ',len(self.overlapped_PDBs_TSG))
            for i in range(0,OG_valid_split_size):
                '''here the validation split is equal as OG split'''
                if all_TSG_train_PDBs[chk_TSG[i+over_chk_TSG]] not in self.overlapped_PDBs_TSG:
                    validation_TSG.append(all_TSG_train_PDBs.pop(chk_TSG[i+over_chk_TSG]))
                else:
                    while (all_TSG_train_PDBs[chk_TSG[i+over_chk_TSG]] in self.overlapped_PDBs_TSG):
                        over_chk_TSG=over_chk_TSG+1
                    validation_TSG.append(all_TSG_train_PDBs.pop(chk_TSG[i+over_chk_TSG]))   
                chk_TSG = self.pop_range_fix(chk_TSG,chk_TSG[i+over_chk_TSG])

            print('validation creation TSG done :)')
            print('')
      
            validation_Fusion = []
            over_chk_Fusion=0
            for i in range(0,int(validation_split*0.5*len(chk_Fusion))):
                if (all_Fusion_train_PDBs[chk_Fusion[i+over_chk_Fusion]] not in self.overlapped_PDBs_TSG) and (all_Fusion_train_PDBs[chk_Fusion[i+over_chk_Fusion]] not in self.overlapped_PDBs_ONGO):
                    validation_Fusion.append(all_Fusion_train_PDBs.pop(chk_Fusion[i+over_chk_Fusion]))    
                else:
                    while  (all_Fusion_train_PDBs[chk_Fusion[i+over_chk_Fusion]] in self.overlapped_PDBs_TSG) or (all_Fusion_train_PDBs[chk_Fusion[i+over_chk_Fusion]] in self.overlapped_PDBs_ONGO):
                        over_chk_Fusion = over_chk_Fusion + 1
                    validation_Fusion.append(all_Fusion_train_PDBs.pop(chk_Fusion[i+over_chk_Fusion]))    
                chk_Fusion = self.pop_range_fix(chk_Fusion,chk_Fusion[i+over_chk_Fusion])

            print('validation creation Fusion done :)')
            print('')

            self.validation_TSG=validation_TSG
            self.validation_Fusion=validation_Fusion
            
            self.TSG_train_PDBs = all_TSG_train_PDBs  
            self.Fusion_train_PDBs = all_Fusion_train_PDBs
            print('')
            print('-------------After vlidation split ------------')

            print('Training data size TSG: ',len(self.TSG_train_PDBs))
            print('Validati data size TSG: ',len(validation_TSG))
            print('Training data size Fusion: ',len(self.Fusion_train_PDBs))
            print('Validati data size Fusion: ',len(validation_Fusion))
            print('')
            print('')

            valid_labels_dic={}
            valid_ids=validation_OG+validation_TSG+validation_Fusion
            
            for i in range(0,len(validation_OG)):
                valid_labels_dic.update({validation_OG[i]: 0})
            for i in range(0,len(validation_TSG)):
                valid_labels_dic.update({validation_TSG[i]: 1})
            for i in range(0,len(validation_Fusion)):
                valid_labels_dic.update({validation_Fusion[i]: 2})

            pickle.dump(valid_labels_dic, open("valid_labels_dic.p", "wb"))  
            pickle.dump(valid_ids, open("valid_ids.p", "wb"))  
            lengths_summery_dic={}
            lengths_summery_dic.update({'valid_len':len(self.validation_OG)+ len(self.validation_TSG)+ len(self.validation_Fusion)})
            lengths_summery_dic.update({'train_len':len(self.OG_train_PDBs)+len(self.TSG_train_PDBs)+len(self.Fusion_train_PDBs)})
            lengths_summery_dic.update({'valid_OG_len':len(self.validation_OG)})
            lengths_summery_dic.update({'valid_TSG_len':len(self.validation_TSG)})
            lengths_summery_dic.update({'valid_Fusion_len':len(self.validation_Fusion)})
            pickle.dump(lengths_summery_dic, open("lengths_summery_dic.p", "wb")) 

            self.lengths_summery_dic = lengths_summery_dic
            self.iter_train=len(self.OG_train_PDBs)+len(self.TSG_train_PDBs)+len(self.Fusion_train_PDBs)#or whole train set size
            #according to this only the data is fetched
        else:
            self.OG_train_PDBs = pickle.load(open("OG_train_PDBs.p", "rb"))  
            self.TSG_train_PDBs = pickle.load(open("TSG_train_PDBs.p", "rb"))      
            self.Fusion_train_PDBs = pickle.load(open("Fusion_train_PDBs.p", "rb"))  
            lengths_summery_dic={}
            lengths_summery_dic.update({'train_len':len(self.OG_train_PDBs)+len(self.TSG_train_PDBs)+len(self.Fusion_train_PDBs)})
            lengths_summery_dic.update({'train_OG_len':len(self.OG_train_PDBs)})
            lengths_summery_dic.update({'train_TSG_len':len(self.TSG_train_PDBs)})
            lengths_summery_dic.update({'train_Fusion_len':len(self.Fusion_train_PDBs)})

            self.lengths_summery_dic = lengths_summery_dic
            self.iter_train=len(self.OG_train_PDBs)+len(self.TSG_train_PDBs)+len(self.Fusion_train_PDBs)#or whole train set size
            #according to this only the data is fetched
        if self.Full_clean:
           self.clean_list_of_ids= []
           self.clean_list_of_id_classes = []
        else:
           self.overlapped_PDBs_ONGO = pickle.load(open("overlapped_PDBs_ONGO.p", "rb"))    
           self.overlapped_PDBs_TSG = pickle.load(open("overlapped_PDBs_TSG.p", "rb"))  
           self.list_of_ids= []
           self.list_of_id_classes = []
       
        # thisindex_former create the randomly shuffled data
        self.index_former()

    def fetch_size(self):
        return self.iter_train

    def pop_range_fix(self,list_1,pop_element_num):
        '''
        To fixing the overlap remoaval for validation data creation
        '''
        list_2=deepcopy(list_1)
        for k in range(0,len(list_1)):
            if list_1[k]>pop_element_num:
                list_2[k]=list_2[k]-1
        return list_2
    
    
    def index_former(self):
        
        self.indexes = np.arange(self.iter_train)
        #to creat the indexes easy
        index_ongo=[]
        index_tsg=[]
        index_fusion=[]
        # since the Fusion indexes runing out early 
        ''' reuse the Fusion class again or combined class'''
        # since the TSG has higher number of labels where iter means the number of irteration each training
        # and since the batch size is 32 and data taken by model is 16 thus the iter is factored by 2
        if self.Full_clean:
            if self.balance_batch_must:
                for i in range(0,(self.iter_train)//16): 
                    #choose 5 from ONGO 7 from TSG and 4 From Fusion to make a batch that distribute the all most all PDBs
                    #Fusion_chk=True
                    '''Since the Models performance always biased to ONGO'''
                    index_ongo,index_tsg,index_fusion=self.adding_fully_clean(True,index_ongo,index_tsg,index_fusion)
                    index_ongo,index_tsg,index_fusion=self.adding_fully_clean(False,index_ongo,index_tsg,index_fusion,OG_IN=False)
                    index_ongo,index_tsg,index_fusion=self.adding_fully_clean(True,index_ongo,index_tsg,index_fusion)
                    index_ongo,index_tsg,index_fusion=self.adding_fully_clean(False,index_ongo,index_tsg,index_fusion)
                    index_ongo,index_tsg,index_fusion=self.adding_fully_clean(True,index_ongo,index_tsg,index_fusion)
                    index_ongo,index_tsg,index_fusion=self.adding_fully_clean(True,index_ongo,index_tsg,index_fusion)
                    index_ongo,index_tsg,index_fusion=self.adding_fully_clean(False,index_ongo,index_tsg,index_fusion,OG_IN=False)

                if (self.iter_train)%16>0: 
                    index_ongo,index_tsg,index_fusion=self.adding_fully_clean(True,index_ongo,index_tsg,index_fusion,OG_IN=False)
                    index_ongo,index_tsg,index_fusion=self.adding_fully_clean(True,index_ongo,index_tsg,index_fusion,OG_IN=False)
                    index_ongo,index_tsg,index_fusion=self.adding_fully_clean(False,index_ongo,index_tsg,index_fusion)
                    index_ongo,index_tsg,index_fusion=self.adding_fully_clean(True,index_ongo,index_tsg,index_fusion)
                    index_ongo,index_tsg,index_fusion=self.adding_fully_clean(False,index_ongo,index_tsg,index_fusion)
                    index_ongo,index_tsg,index_fusion=self.adding_fully_clean(False,index_ongo,index_tsg,index_fusion)
                    index_ongo,index_tsg,index_fusion=self.adding_fully_clean(True,index_ongo,index_tsg,index_fusion)
                    
                os.chdir('/')
                os.chdir(self.main_directory)               
                if self.validation_split>0:
                    pickle.dump(self.clean_list_of_ids, open("V_clean_balnce_must_list_of_ids.p", "wb"))  
                    pickle.dump(self.clean_list_of_id_classes, open("V_clean_balnce_must_list_of_id_classes.p", "wb")) 
                else:
                    pickle.dump(self.clean_list_of_ids, open("clean_balnce_must_list_of_ids.p", "wb"))  
                    pickle.dump(self.clean_list_of_id_classes, open("clean_balnce_must_list_of_id_classes.p", "wb")) 
            else:
                self.not_balance_must_and_Fully_clean()
            
        else:
            if self.balance_batch_must:
                '''First create the indexes for overlapped PDBs'''
                index_ongo_fusion = self.index_stack_overlapped_chk(True,False)
                index_tsg_fusion = self.index_stack_overlapped_chk(False,True)
                for i in range(0,(self.iter_train)//16): 
                    #choose 6 from ONGO 6 from TSG and 4 From Fusion to make a batch that distribute the all most all PDBs
                    #Fusion_chk=True
                    index_ongo,index_tsg,index_fusion,index_ongo_fusion,index_tsg_fusion=self.adding(True,index_ongo,index_tsg,index_fusion,index_ongo_fusion,index_tsg_fusion)
                    index_ongo,index_tsg,index_fusion,index_ongo_fusion,index_tsg_fusion=self.adding(False,index_ongo,index_tsg,index_fusion,index_ongo_fusion,index_tsg_fusion,OG_IN=False)
                    index_ongo,index_tsg,index_fusion,index_ongo_fusion,index_tsg_fusion=self.adding(True,index_ongo,index_tsg,index_fusion,index_ongo_fusion,index_tsg_fusion)
                    index_ongo,index_tsg,index_fusion,index_ongo_fusion,index_tsg_fusion=self.adding(False,index_ongo,index_tsg,index_fusion,index_ongo_fusion,index_tsg_fusion)
                    index_ongo,index_tsg,index_fusion,index_ongo_fusion,index_tsg_fusion=self.adding(True,index_ongo,index_tsg,index_fusion,index_ongo_fusion,index_tsg_fusion)
                    index_ongo,index_tsg,index_fusion,index_ongo_fusion,index_tsg_fusion=self.adding(True,index_ongo,index_tsg,index_fusion,index_ongo_fusion,index_tsg_fusion)
                    index_ongo,index_tsg,index_fusion,index_ongo_fusion,index_tsg_fusion=self.adding(False,index_ongo,index_tsg,index_fusion,index_ongo_fusion,index_tsg_fusion,OG_IN=False)

                if (self.iter_train)%16>0: 
                    index_ongo,index_tsg,index_fusion,index_ongo_fusion,index_tsg_fusion=self.adding(True,index_ongo,index_tsg,index_fusion,index_ongo_fusion,index_tsg_fusion,OG_IN=False)
                    index_ongo,index_tsg,index_fusion,index_ongo_fusion,index_tsg_fusion=self.adding(False,index_ongo,index_tsg,index_fusion,index_ongo_fusion,index_tsg_fusion)
                    index_ongo,index_tsg,index_fusion,index_ongo_fusion,index_tsg_fusion=self.adding(True,index_ongo,index_tsg,index_fusion,index_ongo_fusion,index_tsg_fusion,OG_IN=False)
                    index_ongo,index_tsg,index_fusion,index_ongo_fusion,index_tsg_fusion=self.adding(False,index_ongo,index_tsg,index_fusion,index_ongo_fusion,index_tsg_fusion)
                    index_ongo,index_tsg,index_fusion,index_ongo_fusion,index_tsg_fusion=self.adding(True,index_ongo,index_tsg,index_fusion,index_ongo_fusion,index_tsg_fusion)
                    index_ongo,index_tsg,index_fusion,index_ongo_fusion,index_tsg_fusion=self.adding(True,index_ongo,index_tsg,index_fusion,index_ongo_fusion,index_tsg_fusion)
                    index_ongo,index_tsg,index_fusion,index_ongo_fusion,index_tsg_fusion=self.adding(False,index_ongo,index_tsg,index_fusion,index_ongo_fusion,index_tsg_fusion)

                os.chdir('/')
                os.chdir(self.main_directory)               
                if self.validation_split>0:
                    pickle.dump(self.list_of_ids, open("V_list_of_ids.p", "wb"))  
                    pickle.dump(self.list_of_id_classes, open("V_list_of_id_classes.p", "wb")) 
                else:
                    pickle.dump(self.list_of_ids, open("list_of_ids.p", "wb"))  
                    pickle.dump(self.list_of_id_classes, open("list_of_id_classes.p", "wb")) 
            else:
                self.overalp_included_but_not_balance_must()
    
    def not_balance_must_and_Fully_clean(self):
        '''
        This function craete the IDS and corresponding lable by randomly shuffling the data
        
        Not balance data only shuffled once; thus the data order doesn't changed
       
        Since the TSG class higer number of PDBs remove some of the PDBs from TSG to keep the dataset unique in all sessions
        (using the same data as train and validation)
        ''' 
       
        raise("This function has to be reconstructed to avoid  remove PDBs randomly from the TSG group")
        
        
        clean_list_of_ids_temp=self.OG_train_PDBs+deepcopy(self.TSG_train_PDBs)+self.Fusion_train_PDBs#since removing some PDBs from TSG PDBs
        clean_list_of_ids=[]
        removed_list_tsg=[]
        if (len(clean_list_of_ids_temp)%self.batch_size)>0:
            chk=list(range(len(self.OG_train_PDBs),len(self.OG_train_PDBs)+len(self.TSG_train_PDBs)))
            random.shuffle(chk)#shuffle to remove PDBs randomly from the TSG group
            for i in range(0,len(clean_list_of_ids_temp)%self.batch_size):
                removed_list_tsg.append(clean_list_of_ids_temp.pop(chk[i]))
  
        if self.shuffle:
            random.shuffle(clean_list_of_ids_temp)
            
        for i in range(0,(self.batch_size*self.iter_train)//len(clean_list_of_ids_temp)):
            clean_list_of_ids=clean_list_of_ids + deepcopy(clean_list_of_ids_temp)

        clean_list_of_ids = clean_list_of_ids + deepcopy(clean_list_of_ids_temp[0:(self.batch_size*self.iter_train)%len(clean_list_of_ids_temp)])           
        self.clean_list_of_ids = deepcopy(clean_list_of_ids)
        
        print('')
        print("Not balanced Clean list_of_ids lenth: ",len(self.clean_list_of_ids))
        print('')
 
        os.chdir('/')
        os.chdir(self.main_directory)
        if self.validation_split>0:
            pickle.dump(removed_list_tsg, open("V_removed_list_tsg_unbalanced_clean.p", "wb"))  
            pickle.dump(self.clean_list_of_ids, open("V_clean_list_of_ids.p", "wb"))  
            pickle.dump(self.clean_list_of_id_classes, open("V_clean_list_of_id_classes.p", "wb")) 
        else:
            pickle.dump(removed_list_tsg, open("removed_list_tsg_unbalanced_clean.p", "wb"))  
            pickle.dump(self.clean_list_of_ids, open("clean_list_of_ids.p", "wb"))  
            pickle.dump(self.clean_list_of_id_classes, open("clean_list_of_id_classes.p", "wb"))   
            
    def overalp_included_but_not_balance_must(self):
        '''
        This function craete the IDS and corresponding lable by randomly shuffling the data
        '''
        raise("This function has to be reconstructed to avoid  remove PDBs randomly from the TSG group")

        
        import random 
        list_of_ids_temp=self.OG_train_PDBs+self.TSG_train_PDBs+self.Fusion_train_PDBs+self.overlapped_PDBs_ONGO+self.overlapped_PDBs_TSG
        
        removed_list_tsg=[]
        if (list_of_ids_temp%self.batch_size)>0:
            chk=list(range(len(self.OG_train_PDBs),len(self.OG_train_PDBs)+len(self.TSG_train_PDBs)))
            random.shuffle(chk)
            for i in range(0,list_of_ids_temp%self.batch_size):
                removed_list_tsg.append(list_of_ids_temp.pop(chk[i]))#the removal of TSG PDBs
        
        list_of_ids_classes_temp=[]
        for i in range(0,len(self.OG_train_PDBs)):
            list_of_ids_classes_temp.append(np.array([1,0,0],dtype=float))
        #the removal of TSG PDBs
        for i in range(0,(len(self.TSG_train_PDBs)-len(removed_list_tsg))):
            list_of_ids_classes_temp.append(np.array([0,1,0],dtype=float))       
        for i in range(0,len(self.Fusion_train_PDBs)):
            list_of_ids_classes_temp.append(np.array([0,0,1],dtype=float))   
        for i in range(0,len(self.overlapped_PDBs_ONGO)):
            list_of_ids_classes_temp.append(np.array([0.5,0,0.5]))   
        for i in range(0,len(self.overlapped_PDBs_TSG)):
            list_of_ids_classes_temp.append(np.array([0,0.5,0.5]))   
            
        list_of_ids=[]
        list_of_id_classes=[]
        
        if self.shuffle:
            mapIndexPosition = list(zip(list_of_ids_temp, list_of_ids_classes_temp))
            random.shuffle(mapIndexPosition)
            list_of_ids_temp, list_of_ids_classes_temp = zip(*mapIndexPosition)   

        for i in range(0,self.iter_train//len(list_of_ids_temp)):
            list_of_ids = list_of_ids + deepcopy(list_of_ids_temp)
            list_of_id_classes = list_of_id_classes + deepcopy(list_of_ids_classes_temp)
        
        list_of_ids = list_of_ids + deepcopy(list_of_ids_temp[0:self.iter_train%len(list_of_ids_classes_temp)])           
        list_of_id_classes = list_of_id_classes + deepcopy(list_of_ids_classes_temp[0:self.iter_train%len(list_of_ids_classes_temp)])
        
        self.list_of_ids = deepcopy(list_of_ids)
        self.list_of_id_classes=deepcopy(list_of_id_classes)
    
        os.chdir('/')
        os.chdir(self.main_directory)
        if self.validation_split>0:
            pickle.dump(self.list_of_ids, open("V_list_of_ids.p", "wb"))  
            pickle.dump(self.list_of_id_classes, open("V_list_of_id_classes.p", "wb"))
            pickle.dump(removed_list_tsg, open("V_removed_list_tsg_unbalanced.p", "wb"))  
        else:
            pickle.dump(self.list_of_ids, open("list_of_ids.p", "wb"))  
            pickle.dump(self.list_of_id_classes, open("list_of_id_classes.p", "wb"))
            pickle.dump(removed_list_tsg, open("removed_list_tsg_unbalanced.p", "wb"))  

    def adding(self,Fusion_chk,index_ongo,index_tsg,index_fusion,index_ongo_fusion,index_tsg_fusion,OG_IN=True):
        '''
        This function add the elements of PDBs from the stack
        Fusion_chk: True;means add the Fusion PDB in the list
        
        This check the Fusion class if it's empty only move on to the overlapped PDBs
        Where the weightage is splited equally
        '''
        list_of_ids=self.list_of_ids
        list_of_id_classes=self.list_of_id_classes
        # adding the ONGO elements
        if OG_IN:
            if len(index_ongo)>0: 
                list_of_ids.append(self.OG_train_PDBs[index_ongo.pop()])
                list_of_id_classes.append(np.array([1,0,0],dtype=float))
            else:
                print("ONGO idexes reset")
                index_ongo = self.index_stack_chk(index_ongo_chk=True,index_tsg_chk=False,index_fusion_chk=False)
                list_of_ids.append(self.OG_train_PDBs[index_ongo.pop()])
                list_of_id_classes.append(np.array([1,0,0],dtype=float))
            
        if Fusion_chk:
            # adding the Fusion elements
            # print("Fusion element added")
            if len(index_fusion)>0:
                list_of_ids.append(self.Fusion_train_PDBs[index_fusion.pop()])
                list_of_id_classes.append(np.array([0,0,1],dtype=float))
                overlap_chk=False
            else:
                overlap_chk=True
        # adding the TSG elements
        if len(index_tsg)>0:
            list_of_ids.append(self.TSG_train_PDBs[index_tsg.pop()])
            list_of_id_classes.append(np.array([0,1,0],dtype=float))
        else:
            print("TSG idexes reset")
            index_tsg = self.index_stack_chk(index_ongo_chk=False,index_tsg_chk=True,index_fusion_chk=False)
            list_of_ids.append(self.TSG_train_PDBs[index_tsg.pop()])
            list_of_id_classes.append(np.array([0,1,0],dtype=float))
        

        if Fusion_chk and overlap_chk:
            if not len(index_ongo_fusion)>0:
                index_ongo_fusion = self.index_stack_overlapped_chk(True,False)
                if not len(index_tsg_fusion)>0:
                    print("Since both overlapped done thus Fusion idexes are reseten")
                    index_fusion = self.index_stack_chk(index_ongo_chk=False,index_tsg_chk=False,index_fusion_chk=True)
                    index_tsg_fusion = self.index_stack_overlapped_chk(False,True)
                else:
                    list_of_ids.append(self.overlapped_PDBs_TSG[index_tsg_fusion.pop()])
                    list_of_id_classes.append(np.array([0,0.5,0.5],dtype=float))
            else:
                list_of_ids.append(self.overlapped_PDBs_ONGO[index_ongo_fusion.pop()])
                list_of_id_classes.append(np.array([0.5,0,0.5],dtype=float))
                            
        
        self.list_of_ids = list_of_ids
        self.list_of_id_classes = list_of_id_classes
       
        return index_ongo,index_tsg,index_fusion,index_ongo_fusion,index_tsg_fusion



    def adding_fully_clean(self,Fusion_chk,index_ongo,index_tsg,index_fusion,OG_IN=True):
        '''
        This function add the elements of PDBs from the stack
        Fusion_chk: True;means add the Fusion PDB in the list
        '''
        clean_list_of_ids=self.clean_list_of_ids
        clean_list_of_id_classes=self.clean_list_of_id_classes
        # adding the ONGO elements
        if OG_IN:
            if len(index_ongo)>0: 
                clean_list_of_ids.append(self.OG_train_PDBs[index_ongo.pop()])
                clean_list_of_id_classes.append(0)
            else:
                print("ONGO idexes reset")
                index_ongo = self.index_stack_chk(index_ongo_chk=True,index_tsg_chk=False,index_fusion_chk=False)
                clean_list_of_ids.append(self.OG_train_PDBs[index_ongo.pop()])
                clean_list_of_id_classes.append(0)
        
        if Fusion_chk:
            # adding the Fusion elements
            # print("Fusion element added")
            if len(index_fusion)>0:
                clean_list_of_ids.append(self.Fusion_train_PDBs[index_fusion.pop()])
                clean_list_of_id_classes.append(2)
            else:
                print("Fusion idexes reset")
                index_fusion = self.index_stack_chk(index_ongo_chk=False,index_tsg_chk=False,index_fusion_chk=True)
                clean_list_of_ids.append(self.Fusion_train_PDBs[index_fusion.pop()])
                clean_list_of_id_classes.append(2)
        # adding the TSG elements
        if len(index_tsg)>0:
            clean_list_of_ids.append(self.TSG_train_PDBs[index_tsg.pop()])
            clean_list_of_id_classes.append(1)
        else:
            print("TSG idexes reset")
            index_tsg = self.index_stack_chk(index_ongo_chk=False,index_tsg_chk=True,index_fusion_chk=False)
            clean_list_of_ids.append(self.TSG_train_PDBs[index_tsg.pop()])
            clean_list_of_id_classes.append(1)
        
        self.clean_list_of_ids = clean_list_of_ids
        self.clean_list_of_id_classes = clean_list_of_id_classes
        return index_ongo,index_tsg,index_fusion

    def index_stack_overlapped_chk(self,index_ongo_fusion_chk,index_tsg_fusion_chk):
        '''
        This function is used for mainintaining balanced data set 
        thus it shuffled each time, while creating new data
        
        check the stack if any of them is empty then creat a new stack
        This way always change the train data when it cycles
        
        index_ongo_fusion_chk,index_tsg_fusion_chk if one of them is true then take the stack shuffle and return it
        return the newly stacks index_ongo_fusion index_tsg or index_fusion
        '''
        shuffle = self.shuffle
        if index_ongo_fusion_chk:
            print("ONGO Fusion overlapped idexes reset")
            index_ongo_fusion= np.arange(len(self.overlapped_PDBs_ONGO))
            if shuffle:
                np.random.shuffle(index_ongo_fusion)
            return list(index_ongo_fusion)
        if index_tsg_fusion_chk:
            print("TSG Fusion overlapped idexes reset")
            index_tsg_fusion = np.arange(len(self.overlapped_PDBs_TSG))
            if  shuffle:
                np.random.shuffle(index_tsg_fusion)
            return list(index_tsg_fusion)


    def index_stack_chk(self,index_ongo_chk,index_tsg_chk,index_fusion_chk):
        '''
        This function is used for mainintaining balanced data set 
        thus it shuffled each time, while creating new data
        
        This function check the stack if any of them is empty then creat a new stack
        This way always change the train data when it cycles
        
        index_ongo_chk,index_tsg_chk,index_fusion_chk=If one of them is true then take the stack shuffle and return it
        return the newly stacks index_ongo or index_tsg or index_fusion
        '''
        shuffle =self.shuffle
        if index_ongo_chk:
            print("ONGO idexes reset")
            index_ongo= np.arange(len(self.OG_train_PDBs))
            if shuffle:
                np.random.shuffle(index_ongo)
            return list(index_ongo)
        if index_tsg_chk:
            print("TSG idexes reset")
            index_tsg = np.arange(len(self.TSG_train_PDBs))
            if  shuffle:
                np.random.shuffle(index_tsg)
            return list(index_tsg)
        if index_fusion_chk:
            print("Fusion idexes reset")
            index_fusion = np.arange(len(self.Fusion_train_PDBs))
            if  shuffle:
                np.random.shuffle(index_fusion)
            return list(index_fusion)


    def __data_generation_full_clean(self, list_IDs_temp,list_IDs_class_temp):
        'Generates data containing batch_size samples only this part iws editted' # X : (n_samples, *dim, n_channels)
        # Initialization
        x1= np.empty((self.batch_size,*self.dim, self.n_channels))
        x2= np.empty((self.batch_size,*self.dim, self.n_channels))
        x3= np.empty((self.batch_size,*self.dim, self.n_channels))
        if self.Full_clean:
            y = np.empty((self.batch_size), dtype=int)
        else:
            raise ("Miss use of Function __data_generation_full_clean")
        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            data_temp = np.load(ID)
            x1[i,] = deepcopy(data_temp[0,:,:,:])
            x2[i,] = deepcopy(data_temp[1,:,:,:])
            x3[i,] = deepcopy(data_temp[2,:,:,:])
            y[i] = self.labels[ID]
            if  self.labels[ID]>2:
                print('Wrongly placed id: ',ID)
        if len(list_IDs_temp)>self.batch_size:
            print('-----exceed batch size')
        X=[x1,x2,x3]
        return X,keras.utils.to_categorical(y, num_classes=self.n_classes)
  
    def __data_generation(self, list_IDs_temp,list_IDs_class_temp):
      'Generates data containing batch_size samples only this part iws editted' # X : (n_samples, *dim, n_channels)
      # Initialization
      x1= np.empty((self.batch_size,*self.dim, self.n_channels))
      x2= np.empty((self.batch_size,*self.dim, self.n_channels))
      x3= np.empty((self.batch_size,*self.dim, self.n_channels))
      if not self.Full_clean:
          y = np.empty((self.batch_size,self.n_classes), dtype=float)
      else:
          raise ("Miss use of Function __data_generation")
      # Generate data
      for i, ID in enumerate(list_IDs_temp):
          # Store sample
          data_temp = np.load(ID)
          x1[i,] = deepcopy(data_temp[0,:,:,:])
          x2[i,] = deepcopy(data_temp[1,:,:,:])
          x3[i,] = deepcopy(data_temp[2,:,:,:])
          y[i,:] = deepcopy(list_IDs_class_temp[i])
    
      X=[x1,x2,x3]
      return X,y
          
  
    def __getitem__(self, index):
      'Generate one batch of data'
      if self.Full_clean:
          # Generate indexes of the batch
          if len(self.clean_list_of_ids)>(index+1)*self.batch_size:
              indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
    #          print('here01')
          else:
              print("wqdddddddddddddddddddddddddddddddddddddddddddddddddddddddddddd")
              indexes = self.indexes[index*self.batch_size-(self.batch_size-(len(self.clean_list_of_ids)-index*self.batch_size)):len(self.clean_list_of_ids)]
    #          print(len(indexes))
              # Find list of IDs
          list_IDs_temp = [self.clean_list_of_ids[k] for k in indexes]
          list_IDs_class_temp= [self.labels[ID] for ID in list_IDs_temp]
          
          # Generate data

          if index==0:
              print('len(self.clean_list_of_ids: ', len(self.clean_list_of_ids))
              print('(index+1)*self.batch_size: ',(index+1)*self.batch_size)
              print('len(self.indexes): ',len(self.indexes))
              print('len(indexes): ',len(indexes))
              # print('__getitem__: ',index)
              # print(list_IDs_temp)
              print("")
              print("")

          X, y = self.__data_generation_full_clean(list_IDs_temp,list_IDs_class_temp)
      else:
          if len(self.list_of_ids)>(index+1)*self.batch_size:
              indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
    #          print('here01')
          else:
              indexes = self.indexes[index*self.batch_size-(32-(len(self.list_of_ids)-index*self.batch_size)):len(self.list_of_ids)]
    #          print(len(indexes))
              # Find list of IDs
          list_IDs_temp = [self.list_of_ids[k] for k in indexes]
          list_IDs_class_temp= [self.list_of_id_classes[k] for k in indexes]

          X, y = self.__data_generation(list_IDs_temp,list_IDs_class_temp)
      return X, y


class Data_test:
    'Generates 2D projected data for Keras'
    def __init__(self, list_IDs, labels, batch_size=32, dim = (200,200), n_channels=17 ,n_classes=3):
        'Initialization'
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.dim = dim
        self.indexes = np.arange(len(self.list_IDs))

    def getitem_test(self, index):
      'Generate one batch of data'
      # Generate indexes of the batch
      indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
    
      # Find list of IDs
      list_IDs_temp = [self.list_IDs[k] for k in indexes]
    
      # Generate data
      X, y = self.data_generation_test(list_IDs_temp)
      return X, y      
    
    def data_generation_test(self, list_IDs_temp):
      'Generates data containing batch_size samples only this part iws editted' # X : (n_samples, *dim, n_channels)
       # Initialization
      if len(list_IDs_temp)<self.batch_size:
#              print("Intialised")
#              print("")
#              print("Intialised")
              x1= np.empty((len(list_IDs_temp),*self.dim, self.n_channels))
              x2= np.empty((len(list_IDs_temp),*self.dim, self.n_channels))
              x3= np.empty((len(list_IDs_temp),*self.dim, self.n_channels))
              y = np.empty((len(list_IDs_temp)), dtype=int)
      else:       
          x1= np.empty((self.batch_size,*self.dim, self.n_channels))
          x2= np.empty((self.batch_size,*self.dim, self.n_channels))
          x3= np.empty((self.batch_size,*self.dim, self.n_channels))
          y = np.empty((self.batch_size), dtype=int)
      X=[]
      # Generate data
      for i, ID in enumerate(list_IDs_temp):
          # Store sample
          data_temp = np.load(ID)
          x1[i,] = deepcopy(data_temp[0,:,:,:])
          x2[i,] = deepcopy(data_temp[1,:,:,:])
          x3[i,] = deepcopy(data_temp[2,:,:,:])

          # Store class
          y[i] = self.labels[ID]
      X=[x1,x2,x3]
      return X, keras.utils.to_categorical(y, num_classes=self.n_classes)
  
    def __len__(self):
      'Denotes the number of batches per epoch'
      return int(np.floor(len(self.list_IDs) / self.batch_size))
    
    def __getitem_list_id__(self, index):
      'Generate one batch of data'
      # Generate indexes of the batch
      if len(self.list_IDs)>(index+1)*self.batch_size:
          indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
#          print('here01')
      else:
          indexes = self.indexes[index*self.batch_size-(32-(len(self.list_IDs)-index*self.batch_size)):len(self.list_IDs)]
#          print(len(indexes))
      # Find list of IDs
      list_IDs_temp = [self.list_IDs[k] for k in indexes]
      return list_IDs_temp
  

ModuleNotFoundError: No module named 'keras'

In [ ]:
from model_train_class_dat_valid_overall_vin_4 import ovear_all_training
'''Models'''

from Deep_model_architechtures import model_par_inception_residual_mentor_vin_1
from Deep_model_architechtures import model_inception_residual_mentor_vin_1
'''Not all the inception is done here'''
from copy import deepcopy
import pickle
import os


'''To save the results'''
overall_history_log_list=[]
highest_test_accuracy_list=[]
model_name_list=[]

def chk_to_save(overall_history_log_list,highest_test_accuracy_list,model_name_list,highest_test_accuracy,overall_history_log,not_satisfied,model_name):
    '''To save the results'''

    highest_test_accuracy_list.append(deepcopy(highest_test_accuracy))
    overall_history_log_list.append(deepcopy(overall_history_log))
    model_name_list.append(deepcopy(model_name))
    if not_satisfied:
        return overall_history_log_list,highest_test_accuracy_list,model_name_list
    else:
        '''To save the results for future reference'''
        os.chdir('/')
        os.chdir('scratch/optimaly_tilted/Train')
        pickle.dump(overall_history_log_list, open("overall_history_log.p", "wb"))  
        pickle.dump(highest_test_accuracy_list, open("highest_test_accuracy_list.p", "wb"))  
        pickle.dump(model_name_list, open("model_name_list.p", "wb"))  
        return overall_history_log_list,highest_test_accuracy_list,model_name_list
    
    
def call_channel(channel_number,activation,number_of_overall_iertations_to_the_model,model_loss,optimizer,Full_clean=True,balance_batch_must=False,validation_split=0.15,tower_min_max_only=False,shuffle=False,SITE_MUST=True):
    '''
    Default values
    
    Full_clean=True
    
    validation_split=0.2
        
    balance_batch_must=False
    tower_min_max_only=False
    '''
    print('-------------  training channel no: ',channel_number)
    print('')
    print('-------------  Activation function: ',activation)
    print('')

    '''To save the results'''
    overall_history_log_list=[]
    highest_test_accuracy_list=[]
    model_name_list=[]
    satisfied=False
    not_satisfied=True
#    model_inc_layer_1_chk=0

#    if not_satisfied:
#        '''model_chk `1'''
#        model_chk=1
#        model_train = ovear_all_training(channels=channel_number,number_of_overall_iertations_to_the_model=number_of_overall_iertations_to_the_model,model_loss=model_loss,optimize=optimizer,Full_clean=Full_clean,balance_batch_must=balance_batch_must,validation_split=validation_split,shuffle=shuffle,SITE_MUST=SITE_MUST)
#        model_chk_model = model_inception_only_mentor_vin_1(channels=channel_number,tower_min_max_only=tower_min_max_only,activation=activation)
#        model,model_name= model_chk_model.model_maker()#d1=80,d2=16
#        highest_test_accuracy,overall_history_log,not_satisfied =model_train.model_parameter_train(model,model_name)
#        del model
#        del model_train
#        overall_history_log_list,highest_test_accuracy_list,model_name_list =  chk_to_save(overall_history_log_list,highest_test_accuracy_list,model_name_list,highest_test_accuracy,overall_history_log,not_satisfied,model_name)

    if not_satisfied:
        '''model_chk `1'''
        model_train = ovear_all_training(channels=channel_number,number_of_overall_iertations_to_the_model=number_of_overall_iertations_to_the_model,model_loss=model_loss,optimize=optimizer,Full_clean=Full_clean,balance_batch_must=balance_batch_must,validation_split=validation_split,shuffle=shuffle,SITE_MUST=SITE_MUST)
        model_chk_model = model_par_inception_residual_mentor_vin_1(channels=channel_number,tower_min_max_only=tower_min_max_only,activation=activation)
        model,model_name= model_chk_model.model_maker()#d1=80,d2=16
        highest_test_accuracy,overall_history_log,not_satisfied =model_train.model_parameter_train(model,model_name)
        del model
        del model_train
        overall_history_log_list,highest_test_accuracy_list,model_name_list =  chk_to_save(overall_history_log_list,highest_test_accuracy_list,model_name_list,highest_test_accuracy,overall_history_log,not_satisfied,model_name)

    if not_satisfied:
        '''model_chk `1'''
        model_train = ovear_all_training(channels=channel_number,number_of_overall_iertations_to_the_model=number_of_overall_iertations_to_the_model,model_loss=model_loss,optimize=optimizer,Full_clean=Full_clean,balance_batch_must=balance_batch_must,validation_split=validation_split,shuffle=shuffle,SITE_MUST=SITE_MUST)
        model_chk_model = model_inception_residual_mentor_vin_1(channels=channel_number,tower_min_max_only=tower_min_max_only,activation=activation)
        model,model_name= model_chk_model.model_maker()#d1=80,d2=16
        highest_test_accuracy,overall_history_log,not_satisfied =model_train.model_parameter_train(model,model_name)
        del model
        del model_train
        overall_history_log_list,highest_test_accuracy_list,model_name_list =  chk_to_save(overall_history_log_list,highest_test_accuracy_list,model_name_list,highest_test_accuracy,overall_history_log,not_satisfied,model_name) 
    '''memory issue run on higher GPU machine'''
#    if not_satisfied:
#        '''model_chk `1'''
#        model_train = ovear_all_training(channels=channel_number,number_of_overall_iertations_to_the_model=number_of_overall_iertations_to_the_model,model_loss=model_loss,optimize=optimizer,Full_clean=Full_clean,balance_batch_must=balance_batch_must,validation_split=validation_split,shuffle=shuffle,SITE_MUST=SITE_MUST)
#        model_chk_model = model_par_inception_residual_mentor_vin_1(channels=channel_number,tower_min_max_only=tower_min_max_only,activation=activation)
#        model,model_name= model_chk_model.model_maker(f_inc=2,f_d=1)
#        highest_test_accuracy,overall_history_log,not_satisfied =model_train.model_parameter_train(model,model_name)
#        del model
#        del model_train
#        overall_history_log_list,highest_test_accuracy_list,model_name_list =  chk_to_save(overall_history_log_list,highest_test_accuracy_list,model_name_list,highest_test_accuracy,overall_history_log,not_satisfied,model_name)
#
#    if not_satisfied:
#        '''model_chk `1'''
#        model_train = ovear_all_training(channels=channel_number,number_of_overall_iertations_to_the_model=number_of_overall_iertations_to_the_model,model_loss=model_loss,optimize=optimizer,Full_clean=Full_clean,balance_batch_must=balance_batch_must,validation_split=validation_split,shuffle=shuffle,SITE_MUST=SITE_MUST)
#        model_chk_model = model_inception_residual_mentor_vin_1(channels=channel_number,tower_min_max_only=tower_min_max_only,activation=activation)
#        model,model_name= model_chk_model.model_maker(f_inc=2,f_d=1)
#        highest_test_accuracy,overall_history_log,not_satisfied =model_train.model_parameter_train(model,model_name)
#        del model
#        del model_train
#        overall_history_log_list,highest_test_accuracy_list,model_name_list =  chk_to_save(overall_history_log_list,highest_test_accuracy_list,model_name_list,highest_test_accuracy,overall_history_log,not_satisfied,model_name) 
#     
#    

#    if not_satisfied:
#        '''model_chk `1'''
#        model_chk=1
#        model_train = ovear_all_training(channels=channel_number,number_of_overall_iertations_to_the_model=number_of_overall_iertations_to_the_model,model_loss=model_loss,optimize=optimizer,Full_clean=Full_clean,balance_batch_must=balance_batch_must,validation_split=validation_split,shuffle=shuffle,SITE_MUST=SITE_MUST)
#        model_chk_model = model_par_inception_only_mentor_vin_1(channels=channel_number,tower_min_max_only=tower_min_max_only,activation=activation)
#        model,model_name= model_chk_model.model_maker()#d1=80,d2=16
#        highest_test_accuracy,overall_history_log,not_satisfied =model_train.model_parameter_train(model,model_name)
#        del model
#        del model_train
#        overall_history_log_list,highest_test_accuracy_list,model_name_list =  chk_to_save(overall_history_log_list,highest_test_accuracy_list,model_name_list,highest_test_accuracy,overall_history_log,not_satisfied,model_name)
  
  
    if  not_satisfied:
        print("Try with different dataset or model")
        return False

    else:
        print(channel_number, ' satisfied some where chk')
        return True

#first find which dataset works best with the model
Full_clean=False
balance_batch_must=True
validation_split=0.2
tower_min_max_only=True
number_of_overall_iertations_to_the_model=2

activation =  'swish'

#chk_bal_tow_2_swish =call_channel(21,activation,number_of_overall_iertations_to_the_model,model_loss='msle',optimizer='RMSprop',Full_clean=Full_clean,balance_batch_must=True,tower_min_max_only=True)
#if not chk_bal_tow_2_swish:
chk_bal_2_swish = call_channel(20,activation,number_of_overall_iertations_to_the_model,model_loss='msle',optimizer='RMSprop',Full_clean=Full_clean,balance_batch_must=True,tower_min_max_only=False,SITE_MUST=False)
if not chk_bal_2_swish:
    print(" -------------------- Running on SITE must new prop with threshold changed ----------------- ")
    chk_bal_2_swish_SITE_MUST =call_channel(21,activation,number_of_overall_iertations_to_the_model,model_loss='msle',optimizer='RMSprop',Full_clean=Full_clean,balance_batch_must=True,tower_min_max_only=False,SITE_MUST=True)
    if not chk_bal_2_swish_SITE_MUST:
        print(" --------------------- Running on SITE must with old threshold  ---------------")
        chk_bal_2_swish_SITE_MUST_old =call_channel(17,activation,number_of_overall_iertations_to_the_model,model_loss='msle',optimizer='RMSprop',Full_clean=Full_clean,balance_batch_must=True,tower_min_max_only=False,SITE_MUST=True,fusion_test_chk=0.2)

if chk_bal_2_swish or chk_bal_2_swish_SITE_MUST:
    chk_bal_2_swish_with_fusion = call_channel(20,activation,number_of_overall_iertations_to_the_model,model_loss='msle',optimizer='RMSprop',Full_clean=Full_clean,balance_batch_must=True,tower_min_max_only=False,SITE_MUST=False,fusion_test_chk=0.2)
    if not chk_bal_2_swish_with_fusion:
        chk_bal_2_swish_SITE_MUST_with_fusion =call_channel(21,activation,number_of_overall_iertations_to_the_model,model_loss='msle',optimizer='RMSprop',Full_clean=Full_clean,balance_batch_must=True,tower_min_max_only=False,SITE_MUST=True,fusion_test_chk=0.2)


#    if not chk_bal_2_swish:
#        activation =  'relu'
#        chk_bal_tow_2_relu =call_channel(21,activation,number_of_overall_iertations_to_the_model,model_loss='msle',optimizer='RMSprop',Full_clean=Full_clean,balance_batch_must=True,tower_min_max_only=True)
#        if not chk_bal_tow_2_relu:
#            chk_bal_2_relu =call_channel(21,activation,number_of_overall_iertations_to_the_model,model_loss='msle',optimizer='RMSprop',Full_clean=Full_clean,balance_batch_must=True,tower_min_max_only=False)
#    
#            activation =  'swish'                   
#            print("")
#            if not chk_bal_2_relu:
#                print('')
#                print('')
#                print('<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<----  Shuffle activated  here-on   number_of_overall_iertations_to_the_model: 2 ---->>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
#                print('')
#                number_of_overall_iertations_to_the_model=1
#                print('')
#                chk_bal_tow_2_shuf_1_swish =call_channel(21,activation,number_of_overall_iertations_to_the_model,model_loss='msle',optimizer='RMSprop',Full_clean=Full_clean,balance_batch_must=True,tower_min_max_only=True,shuffle=True)
#                if not chk_bal_tow_2_shuf_1_swish:   
#                    chk_bal_2_shuf_1_swish =call_channel(21,activation,number_of_overall_iertations_to_the_model,model_loss='msle',optimizer='RMSprop',Full_clean=Full_clean,balance_batch_must=True,tower_min_max_only=False,shuffle=True)
#                    if not chk_bal_2_shuf_1_swish:
#                        print('')
#                        print('')
#                        print('<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<----  Shuffled & Not balanced number_of_overall_iertations_to_the_model: 1 ---->>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
#                        print('')
#                        number_of_overall_iertations_to_the_model=1
#                        print('')
#                        chk_bal_tow_2_shuf_Not_bal1_swish =call_channel(21,activation,number_of_overall_iertations_to_the_model,model_loss='msle',optimizer='RMSprop',Full_clean=Full_clean,balance_batch_must=True,tower_min_max_only=True,shuffle=True)
#                        if not chk_bal_tow_2_shuf_Not_bal1_swish:
#                            chk_bal_2_shuf_Not_bal1_swish =call_channel(21,activation,number_of_overall_iertations_to_the_model,model_loss='msle',optimizer='RMSprop',Full_clean=Full_clean,balance_batch_must=True,tower_min_max_only=False,shuffle=True)
#                            if not chk_bal_2_shuf_Not_bal1_swish:
#                                print('')
#                                print('')
#                                print('<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<----  Shuffle activated  here-on   number_of_overall_iertations_to_the_model: 2 ---->>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
#                                print('')
#                                number_of_overall_iertations_to_the_model=1
#                                print('')
#                                chk_bal_tow_2_shuf_1_relu =call_channel(21,activation,number_of_overall_iertations_to_the_model,model_loss='msle',optimizer='RMSprop',Full_clean=Full_clean,balance_batch_must=True,tower_min_max_only=True,shuffle=True)
#                                if not chk_bal_tow_2_shuf_1_relu:   
#                                    chk_bal_2_shuf_1_relu =call_channel(21,activation,number_of_overall_iertations_to_the_model,model_loss='msle',optimizer='RMSprop',Full_clean=Full_clean,balance_batch_must=True,tower_min_max_only=False,shuffle=True)
#                                    if not chk_bal_2_shuf_1_relu:
#                                        print('')
#                                        print('')
#                                        print('<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<----  Shuffled & Not balanced number_of_overall_iertations_to_the_model: 1 ---->>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
#                                        print('')
#                                        number_of_overall_iertations_to_the_model=1
#                                        print('')
#                                        chk_bal_tow_2_shuf_Not_bal1_relu =call_channel(21,activation,number_of_overall_iertations_to_the_model,model_loss='msle',optimizer='RMSprop',Full_clean=Full_clean,balance_batch_must=True,tower_min_max_only=True,shuffle=True)
#                                        if not chk_bal_tow_2_shuf_Not_bal1_relu:
#                                            chk_bal_2_shuf_Not_bal1_relu =call_channel(21,activation,number_of_overall_iertations_to_the_model,model_loss='msle',optimizer='RMSprop',Full_clean=Full_clean,balance_batch_must=True,tower_min_max_only=False,shuffle=True)



# chk_bal_2 =call_channel(21,number_of_overall_iertations_to_the_model,model_loss='msle',optimizer='RMSprop',balance_batch_must=True)
# chk_bal_0 =call_channel(21,number_of_overall_iertations_to_the_model,model_loss='categorical_crossentropy',optimizer='Adam',balance_batch_must=True)

# chk_bal_0 =call_channel(21,number_of_overall_iertations_to_the_model,model_loss='categorical_crossentropy',optimizer='Adam',balance_batch_must=True)
# chk_bal_3 =call_channel(21,number_of_overall_iertations_to_the_model,model_loss='categorical_hinge',optimizer='Adagrad',balance_batch_must=True)
# chk_bal_4= call_channel(21,number_of_overall_iertations_to_the_model,model_loss='categorical_hinge',optimizer='Adadelta',balance_batch_must=True)

In [ ]:
#%%
#import tensorflow as tf
from data_generation_valid_vin3 import DataGenerator_splited 
from data_generation_valid_vin3 import Data_test


from tensorflow import keras
from copy import deepcopy
import numpy as np
import os
import pickle

from time import sleep# to make the GPU cool down
import gc

class ovear_all_training:
    def __init__(self,channels,test_accuracy=82,total_iteration=100,number_of_overall_iertations_to_the_model=10,validation_steps_check=5,minimum_iteration_validation_chk=1,sleep_time=30, model_loss='MSE', optimize='Adam',Full_clean=True,balance_batch_must=True,validation_split=0.15,shuffle=False,threshold=True,batch_size=32,SITE_MUST=True,fusion_test_chk=0):
        '''Initialization
        channels        : the depth of the input
        test_accuracy   : the wanted minimum test accuracy(since I obtained 86% in earlier model accidently/ but training accuracy is 85.--)
        total_iteration : This decide the number of time the data set is used; since the dataset is feed forward and backward once mean epoch count is one)
                            preferes if valid split 0.15 then 82    
                                if valid split 0.2 then 73   
        number_of_overall_iertations_to_the_model: To validate the model architechture gonna satisfy the condition or not, 
                            thus the training is done on  different ways like changing the hyper aprameters(like depth in each layer or kernal size)
                            of the model at each time
                            and check the performance changed or not   
        sleep_time       : define the waiting time(in milli seconds) to cool down between training sessions
        
        **
        checking the model whether it stuck in the suboptimal or optimal
        
        minimum_iteration_validation_chk: How many iterations(minimum should be 1) should run before going on check it reached optimal or not

        Inorder to do this first check the training accuracy cross the limitation such as 
                self.train_acc_limit=0.82 thus training accuracy must cross the given accuracy

            if the training accuracy cross the limitation then check Validation accuracy
            First check the validation acuracy is higher than 0.7(70%);
                
                validation_steps_check(limit assigned as 5): How many times the validation accuracy stucked in the same position; 
                                            to avoid leaving the optimal model
                                            if the accuracy is <70% then it is reset
                                            
            if validation accuracy <70%:
                validation_low_steps_check(limit assigned as 10): If the validation accuracy less than 70 for long time while the training accuracy is high

      validation_split: gives the validation data seperately
      '''
        self.model_loss=model_loss#'MSE'#"mean squared error"
        self.optimizer=optimize#'Adam'

        self.channels = channels
        print('channel initialised: ',self.channels)

        self.sleep_time = sleep_time
		
        #the parameters for data split
        self.validation_split = validation_split#0.2 
        self.shuffle = shuffle
        self.balance_batch_must=balance_batch_must
        self.Full_clean = Full_clean
        self.batch_size=batch_size
#        self.batch_size=10
        print("")
        print("Over ridden self.batch_size as ",self.batch_size)
        print("")
        self.load_train_size=64


        self.test_accuracy=test_accuracy #82 is default the wanted minimum test accuracy
        self.total_iteration=total_iteration #earlier v102
        self.number_of_overall_iertations_to_the_model=number_of_overall_iertations_to_the_model
        self.validation_steps_check = validation_steps_check#earliet its 2
        self.minimum_iteration_validation_chk=minimum_iteration_validation_chk
        
        self.validation_acc_chk_limit = 0.595
        self.validation_acc_OG_TSG_limit = 0.712#0.81
        self.train_acc_limit=0.82#to check the validation if it cross the training limitations 
        self.validation_low_steps_check=10
        self.fusion_test_chk=fusion_test_chk#0.2 is earlier
        print("Here doesn't consider fusion at all")
        self.epochs =55#howmany timne the same data used for training/ or feed through the network forward and backward
       
        optimal_tilted=False
        if SITE_MUST:
            self.SITE_MUST=True
            if channels==21:
                self.main_dir=''.join(['scratch/optimaly_tilted_21'])
                if threshold and optimal_tilted:
                    '''Different threshold condition for surface define'''
                    self.main_dir =''.join(['scratch/21_aminoacids__thrsh_new_SITE_MUST'])
                elif threshold:
                    self.main_dir =''.join(['scratch/21_aminoacids_NO_TILT_thrsh_new_SITE_MUST'])
            elif channels==20:
                self.main_dir=''.join(['scratch/neg_size_20_21_aminoacids'])
            elif channels==17:
                self.main_dir=''.join(['scratch/optimaly_tilted_17'])
            elif  channels==15:
                if threshold:
                    self.main_dir=''.join(['scratch/15_aminoacids_thrsh_new_SITE_MUST'])
        else:
            self.SITE_MUST=False
            if channels==20:
#                self.main_dir=''.join(['scratch/optimaly_tilted_21'])
                if threshold:
                    '''Different threshold condition for surface define'''
                    self.main_dir =''.join(['scratch/20_aminoacids__thrsh'])
            if channels==14:
                if threshold:
                    self.main_dir=''.join(['scratch/neg_14_21_aminoacids'])       

    def validation_accuracy_reached_optimal(self,validation_accuracy_new,validation_accuracy_last,validation_count):
        '''
        This fucntion check
        if the training stucked in sub optimal point then retrive the model, to check the performance
       
        validation_count        : check howmany times the validation accuracy hasn't changed
        validation_accuracy_last: The validation accuracy got for last time
        
        it returns
            to stop the training(True) or (False)
        '''
        if validation_accuracy_new==validation_accuracy_last:
            validation_count = validation_count+1
            if validation_count>self.validation_steps_check:
                return True,self.validation_steps_check-1
            else:
                return False,validation_count
        else:
            return False,0#reset the validation count to 0
     
    def validation_accuracy_reached_low_optimal(self,validation_accuracy_new,validation_accuracy_last,validation_low_count):
        '''
        This fucntion check
        if the training stucked in sub optimal point then retrive the model, to check the performance
       
        validation_count        : check howmany times the validation accuracy hasn't changed
        validation_accuracy_last: The validation accuracy got for last time
        
        it returns
            to stop the training(True) or (False)
        '''
        if validation_accuracy_new==validation_accuracy_last:
            validation_low_count = validation_low_count + 1 
            if validation_low_count > self.validation_low_steps_check:
                return True,validation_low_count
            else:
                return False,validation_low_count
        else:
            return False,0#reset the validation count to 0    
        
    def validation_accuracy_reached_optimal_loop(self,validation_accuracy_new,validation_accuracy_last_1,validation_accuracy_last,validation_count_loop):
        '''
        This fucntion check
        if the training stucked in sub optimal point then retrive the model, to check the performance
        this check the optimal solution fell in loop(varying between optimal solution back and forward)
       
        validation_count        : check howmany times the validation accuracy hasn't changed
        validation_accuracy_last: The validation accuracy got for last time
        
        it returns
            to stop the training(True) or (False)
        '''
        # print('validation_accuracy_new: ',validation_accuracy_new)
        # print('validation_accuracy_last_1: ',validation_accuracy_last_1)
        # print('validation_accuracy_last: ',validation_accuracy_last)
        # print('validation_count_loop: ',validation_count_loop)
        if (validation_accuracy_new==validation_accuracy_last) and (validation_accuracy_last==validation_accuracy_last_1):

            validation_count_loop = validation_count_loop+1
            if validation_count_loop > (self.validation_steps_check):
                return True,((self.validation_steps_check)-1) # to check whether it stuctk in suboptimal again
            else:
                return False,validation_count_loop+1
        elif validation_accuracy_last<validation_accuracy_new:
            return False,0
        else:
            return False,(validation_count_loop-1)

        
    def model_parameter_train(self,model,model_name):
       
        number_of_overall_iertations_to_the_model=  self.number_of_overall_iertations_to_the_model
        
        total_iteration = self.total_iteration   
        train_data_dir= ''.join([self.main_dir,'/Train'])

        os.chdir('/')
        os.chdir(self.main_dir)
        
        '''To load the train dataset'''
        train_labels = pickle.load(open("train_labels_dic.p", "rb"))
        '''To load the test dataset'''
        test_labels = pickle.load(open("test_labels_dic.p", "rb"))


        '''train the model
        
        If Adam optimizer or RMSprop: it take care of the leraning rate after it intialised
        
        '''
        if self.model_loss=="MSE":
            if self.optimizer=='RMSprop':
                model.compile(loss='mean_squared_error',
                      optimizer=keras.optimizers.RMSprop(),
                      metrics=['accuracy'])
            elif self.optimizer=='Adam':
                 model.compile(loss='mean_squared_error',
                      optimizer=keras.optimizers.Adagrad(),
                      metrics=['accuracy'])
        elif  self.model_loss == 'msle':
            if self.optimizer=='RMSprop':
                model.compile(loss='mean_squared_logarithmic_error',
                  optimizer=keras.optimizers.RMSprop(),
                  metrics=['accuracy'])
            elif self.optimizer=='Adam':
                 model.compile(loss='mean_squared_logarithmic_error',
                      optimizer=keras.optimizers.Adagrad(),
                      metrics=['accuracy'])
        elif self.model_loss == 'categorical_crossentropy':
            if self.optimizer=='RMSprop':
                model.compile(loss='categorical_crossentropy',
                              optimizer=keras.optimizers.RMSprop(),
                              metrics=['accuracy'])
            elif self.optimizer=='Adam':
                 model.compile(loss='categorical_crossentropy',
                      optimizer=keras.optimizers.Adagrad(),
                      metrics=['accuracy'])    
        elif self.model_loss == 'sparse_categorical_crossentropy' and self.optimizer=='RMSprop':
            model.compile(loss='sparse_categorical_crossentropy',
                          optimizer=keras.optimizers.RMSprop(),
                          metrics=['accuracy'])
        elif self.model_loss =='categorical_hinge':
           if self.optimizer=='Adagrad':
              model.compile(loss='categorical_hinge',
                          optimizer=keras.optimizers.Adagrad(),
                          metrics=['accuracy'])
           elif self.optimizer=='Adadelta':
              model.compile(loss='categorical_hinge',
                          optimizer=keras.optimizers.Adadelta(),
                          metrics=['accuracy'])
        '''To break this while if it auceed the limiattions provided'''

        #temperory variables
        total_sub_optimal_loop=0
        self.highest_test_accuracy=0
        while number_of_overall_iertations_to_the_model>0:
            
            print('')
            if total_sub_optimal_loop>5:
                print('The mdel exceed the suboptimal limitations')
                break# while loop Session break

            '''To check the validation accuracy stuck in suboptimal'''
            self.validation_low_count=0#if the validation accuracy less than 50 for more than 15 iterations continiously 
                                    #check the test and flush it as new iteration
            correct=0 
            self.validation_count=0
            self.validation_accuracy_last=0
            self.validation_accuracy_last_1=-1#to make both different
            self.validation_count_loop=0
    
            print('')
            print('')
            print('')
            print('')
            print('*****************************************---------------------->>>>>>>>>>>>>>>>>>>>>>>>>>> ******************')
            print('')
            print('Remaining iterations ', number_of_overall_iertations_to_the_model, ' in model ', model_name)
            print('')
            print('loss function: ',self.model_loss)
            print('')
            print('Optimiser function: ',self.optimizer)
            print('')
            print('Highest accuracy upto now sessions in test set: ',self.highest_test_accuracy)
            print('')
#            training_generator = DataGenerator_splited(train_labels,self.main_dir,batch_size=self.batch_size,n_channels=self.channels,Full_clean=self.Full_clean,balance_batch_must=self.balance_batch_must,validation_split=self.validation_split,shuffle=self.shuffle)
            training_generator = DataGenerator_splited(train_labels,self.main_dir,batch_size=self.load_train_size,n_channels=self.channels,Full_clean=self.Full_clean,balance_batch_must=self.balance_batch_must,validation_split=self.validation_split,shuffle=self.shuffle)
            

            if number_of_overall_iertations_to_the_model != self.number_of_overall_iertations_to_the_model:
                # to avoid unnecessary reinitialisation in the beginning
                model = self.shuffle_weights(model)
                print('Reintialised :)')
                
            print("check the test set")      
            session_break,itera_chk,correct,test_probabilities,pdbs_test=self.test_per_chk(model,validation_cond_low=False,session_break=False)
            print("checking the test set done : )")      

            overall_history_log=[]
            validation_history_log=[]
            
            for itera_train in range(0,total_iteration):
                gc.collect()
            #    print(history.history.keys())
            #    print('validation_accuracy: ',history.history['val_acc'][-1])
                train_correct=0
                session_break=False
#                for fetch in range(0,-(-training_generator.fetch_size()//self.batch_size)):
                for fetch in range(0,-(-training_generator.fetch_size()//self.load_train_size)):
                    os.chdir('/')
                    os.chdir(train_data_dir)
                    # Train model on dataset
                    x_train, y_train = training_generator.__getitem__(itera_train)
    #                train_ids_temp = training_generator.__getitem_list_id(itera)
                    if self.batch_size<32:
                        history = model.fit(x_train, y_train,batch_size=self.batch_size,epochs=1)#,validation_split=0.2)
                        train_correct =train_correct+ history.history['acc'][-1]*self.load_train_size
#                        print(history.history.keys())
#                        if itera_train==0:
#                            if fetch==1:
#                                print('History accuarcys look like')
#                                print(history.history['acc'])
                    else:
                        history = model.fit(x_train, y_train,batch_size=self.batch_size,epochs=1)#,validation_split=0.2)
#                        history = model.fit(x_train, y_train,batch_size=16,epochs=1)#,validation_split=0.2)
                        train_correct =train_correct+ history.history['acc'][-1]*self.load_train_size
                    train_acc=(train_correct/((fetch+1)*self.load_train_size))
                    if train_acc>0.91 and fetch%4==0 and itera_train>9:
                        session_break,while_break_suboptimal,while_break_correct = self.overall_model_per_chk(train_acc,itera_train,validation_history_log,model,model_name)
                        if session_break:
                            break
#                    print('Accuracy in fetch: ',fetch,' ', history.history['acc'][-1])
                if session_break:
                    break
                else:
                    train_acc = train_correct/training_generator.fetch_size()
                    print("")
                    print("Overall train iteration of ",itera_train,"'s accuracy is ",train_acc*100,' %')
                    print("")

                    session_break,while_break_suboptimal,while_break_correct = self.overall_model_per_chk(train_acc,itera_train,validation_history_log,model,model_name)
                if session_break:
                    break              
                
            '''To break the overall while'''
            if while_break_correct:
                print('Model suceed the limitations :)')
                if self.SITE_MUST:
                    model_name_latest_part=''.join([model_name[0:-3],'_SITE_',self.model_loss,'_',self.optimizer,'_',str(int(100*(correct/len(test_labels))))])
                else:
                    model_name_latest_part=''.join([model_name[0:-3],'_',self.model_loss,'_',self.optimizer,'_',str(int(100*(correct/len(test_labels))))])
                if self.Full_clean and self.balance_batch_must:
                    model_name_latest=''.join([model_name_latest_part,'_Full_clean_bal_bat.h5'])
                elif self.Full_clean:
                    model_name_latest=''.join([model_name_latest_part,'_Full_clean.h5'])
                elif self.balance_batch_must:
                    model_name_latest=''.join([model_name_latest_part,'_bal_bat.h5'])
                else:
                    model_name_latest=''.join([model_name_latest_part,'.h5'])
             
                print("")                
                print("suceed model")
                print(model_name_latest)
                print("")
                model.save(model_name_latest)
                gc.collect()
                del model
                break
            
            elif while_break_suboptimal:
                print("since stuck in suboptimal doing the iteration agian")
                overall_history_log=[]
                validation_history_log=[]
                total_sub_optimal_loop=total_sub_optimal_loop+1
            else:
                print("Session redo")
                number_of_overall_iertations_to_the_model=number_of_overall_iertations_to_the_model-1
                overall_history_log=[]
                validation_history_log=[]
        print('Highest_test_accuracy: ',self.highest_test_accuracy)
        if not (while_break_correct):
            gc.collect()
            del model
     
        history_all=[validation_history_log,overall_history_log]    
        if 100*(correct/len(test_labels))>=self.test_accuracy:
            print('')
            print(str(self.sleep_time),' Sec break :) satisfied ')
            sleep(self.sleep_time)
            print('')
            return self.highest_test_accuracy,history_all,False#return not_satisfied as false
        else:
            print('')
            print(str(self.sleep_time),' Sec break ')
            sleep(self.sleep_time)
            print('')
            return self.highest_test_accuracy,history_all,True##return not_satisfied as True
        


    def shuffle_weights(self,model, weights=None):
        """Randomly permute the weights in `model`, or the given `weights`.
        This is a fast approximation of re-initializing the weights of a model.
        Assumes weights are distributed independently of the dimensions of the weight tensors
          (i.e., the weights have the same distribution along each dimension).
        :param Model model: Modify the weights of the given model.
        :param list(ndarray) weights: The model's weights will be replaced by a random permutation of these weights.
          If `None`, permute the model's current weights.
        """
        if weights is None:
            weights = model.get_weights()
        
        weights = [np.random.permutation(w.flat).reshape(w.shape) for w in weights]
        # Faster, but less random: only permutes along the first dimension
        # weights = [np.random.permutation(w) for w in weights]
        model.set_weights(weights)
        return model
   
    def create_train_accuracy_for_whole_set(self,model,train_data_dir,test_probabilities,pdbs_test):
        '''
        This function go through the whole training data set and produce the corresponding results
        '''
        os.chdir('/')
        os.chdir(self.main_dir)
        pickle.dump(test_probabilities, open("test_probabilities.p", "wb"))  
        pickle.dump(pdbs_test, open("pdbs_test.p", "wb"))  
        
        train_test_list_IDs = pickle.load(open("train_list_ids.p", "rb"))
        train_test_labels= pickle.load(open("train_labels_dic.p", "rb"))
        
        os.chdir('/')
        os.chdir(train_data_dir)
        train_test_generator = Data_test(train_test_list_IDs, train_test_labels,batch_size=self.batch_size,n_channels=self.channels)
        train_test_probabilities=[]
        pdbs_train_test=[]
        c_train_test=0
        for itera in range(0,len(train_test_list_IDs)//self.batch_size):
            x_train_test, y_train_test = train_test_generator.getitem_test(itera)
            test_ids_temp = train_test_generator.__getitem_list_id__(itera)
        
            train_test_scores = model.evaluate(x_train_test, y_train_test, verbose=2)
            probabilities = model.predict(x_train_test)
           
            train_test_probabilities.append(deepcopy(probabilities))
            pdbs_train_test.append(deepcopy(test_ids_temp))
            c_train_test=c_train_test+int(train_test_scores[1]*self.batch_size)
            
        list_IDs_temp=[]
        for itera in range((len(train_test_list_IDs)-len(train_test_list_IDs)%self.batch_size),len(train_test_list_IDs)):
            list_IDs_temp.append(train_test_list_IDs[itera])
        
        x_test, y_test = train_test_generator.data_generation_test(list_IDs_temp)
        test_scores = model.evaluate(x_test, y_test, verbose=2)
        
        c_train_test=c_train_test+int(test_scores[1]*(len(train_test_list_IDs)%self.batch_size))
        probabilities = model.predict(x_test)
        train_test_probabilities.append(deepcopy(probabilities))
        pdbs_train_test.append(deepcopy(list_IDs_temp))
        
        os.chdir('/')
        os.chdir(self.main_dir)
        pickle.dump(train_test_probabilities, open("train_test_probabilities.p", "wb"))  
        pickle.dump(pdbs_train_test, open("pdbs_train_test.p", "wb"))
        print("Corresponding_training accuracy: ",100*c_train_test/len(train_test_list_IDs)," %")
        os.chdir('/')
        os.chdir(train_data_dir)
        
        
    def validating_per_chk(self,valid_list_IDs,valid_generator,model,validation_history_log,itera_train):
        '''validating performance check'''
        train_data_dir= ''.join([self.main_dir,'/Train'])
        os.chdir('/')
        os.chdir(train_data_dir)
        valid_may_sat=False

        valid_correct=0
        valid_correct_combo=[]
        
        break_valid=False
        valid_chk_rest_TSG_OK=True
        valid_chk_rest_ONGO_OK=True
        valid_chk_rest_Fusion_OK=True
        for itera in range(0,len(valid_list_IDs)//self.batch_size):
            x_valid, y_valid = valid_generator.getitem_test(itera)
        
            valid_scores = model.evaluate(x_valid, y_valid, verbose=2)
            print("Valid_iter: ",itera," Validation acc: ",valid_scores[1])
            valid_correct=valid_correct+int(valid_scores[1]*self.batch_size)
            valid_acc=valid_correct/((itera+1)*self.batch_size)
            break_valid,valid_chk_rest_ONGO_OK,valid_chk_rest_TSG_OK,valid_chk_rest_Fusion_OK,valid_acc= self.validation_break_chk(valid_scores,itera,valid_correct,valid_acc,break_valid,valid_chk_rest_TSG_OK,valid_chk_rest_ONGO_OK,valid_chk_rest_Fusion_OK)
            valid_correct_combo.append(valid_scores[1])     
            if break_valid:
                break
            
        if valid_chk_rest_ONGO_OK and valid_chk_rest_TSG_OK and valid_chk_rest_Fusion_OK:
            valid_ids_temp=[]
            for itera in range((len(valid_list_IDs)-len(valid_list_IDs)%self.batch_size),len(valid_list_IDs)):
                valid_ids_temp.append(valid_list_IDs[itera])
            
            x_valid, y_valid = valid_generator.data_generation_test(valid_ids_temp)
            valid_scores = model.evaluate(x_valid, y_valid, verbose=2)
            print("Valid_iter final Validation acc: ",valid_scores[1])

            valid_correct=valid_correct+int(valid_scores[1]*(len(valid_list_IDs)%self.batch_size))
            valid_acc=valid_correct/len(valid_list_IDs)
            print('Validation accuracy in iteration ',itera_train,': ', 100*valid_acc, '%')
            validation_history_log.append([itera_train,valid_acc])
            if valid_acc>self.train_acc_limit:
                print('Validation accuracy:', 100*valid_acc, '%')
            test_OG_chk=False
            test_TSG_chk=False
            for val_acc_i in range(0,len(valid_correct_combo)):
                if val_acc_i<3:
                    if valid_correct_combo[val_acc_i]>= self.validation_acc_OG_TSG_limit:
                        test_OG_chk=True
                if 3<val_acc_i<7:
                    if valid_correct_combo[val_acc_i]>= self.validation_acc_OG_TSG_limit:
                        test_TSG_chk=True
            if test_OG_chk and test_TSG_chk and valid_acc >= self.validation_acc_chk_limit:
                valid_may_sat=True

        return valid_acc,valid_may_sat,validation_history_log
    
    def overall_model_per_chk(self,train_acc,itera_train,validation_history_log,model,model_name):
        '''
        check the overall peroformance of the model
        '''
        '''Initialising validation steps'''
        validation_low_count=self.validation_low_count
        validation_count=self.validation_count
        validation_accuracy_last=self.validation_accuracy_last
        validation_count_loop=self.validation_count_loop
        validation_accuracy_last_1=self.validation_accuracy_last_1
        
        train_data_dir= ''.join([self.main_dir,'/Train'])
        
        while_break_suboptimal=False
        while_break_correct=False
        session_break=False
        os.chdir('/')
        os.chdir(self.main_dir)

        if self.validation_split>0:
            valid_labels = pickle.load(open("valid_labels_dic.p","rb"))
            valid_list_IDs = pickle.load(open("valid_ids.p", "rb"))
            valid_generator = Data_test(valid_list_IDs, valid_labels,batch_size=self.batch_size,n_channels=self.channels)           
            valid_acc,valid_may_sat,validation_history_log = self.validating_per_chk(valid_list_IDs,valid_generator,model,validation_history_log,itera_train)
        
        if (train_acc>self.train_acc_limit) or (valid_acc>(self.test_accuracy/100) or valid_acc>= self.validation_acc_chk_limit):
            '''To check it reaches over all optimal  Given limit validation 71%'''
            if (train_acc>self.train_acc_limit and itera_train>46):
                validation_cond_low,validation_low_count= self.validation_accuracy_reached_low_optimal(valid_acc,validation_accuracy_last,validation_low_count)
                validation_cond,validation_count =  self.validation_accuracy_reached_optimal(valid_acc,validation_accuracy_last,validation_count)
            else:
                validation_cond_low=False
                validation_cond=False

            validation_cond_loop,validation_count_loop =  self.validation_accuracy_reached_optimal_loop(valid_acc,validation_accuracy_last_1,validation_accuracy_last,validation_count_loop)
            validation_accuracy_last_1=deepcopy(validation_accuracy_last)
            validation_accuracy_last=deepcopy(valid_acc)

            if validation_cond:
                '''
                if accuracy above 70% only considered
                    else reset the conditioin
                '''
                if valid_acc<self.train_acc_limit:
                    validation_count=0
                    validation_cond=False
                else:
                    print("Validation accuracy optimal")
                
            if (((valid_acc>(self.test_accuracy/100)) or (valid_may_sat) or (validation_cond)) or validation_cond_loop or validation_cond_low) and (itera_train+1)>self.minimum_iteration_validation_chk:
               
                os.chdir('/')
                os.chdir(self.main_dir)
                
                '''To load the test dataset'''
                test_labels = pickle.load(open("test_labels_dic.p", "rb"))
                session_break,itera_chk,correct,test_probabilities,pdbs_test=self.test_per_chk(model,validation_cond_low,session_break)
                
                if itera_chk:#to avoid unnecessary evaluation in test set 

                    os.chdir('/')
                    os.chdir(train_data_dir)
                    if self.highest_test_accuracy<100*(correct/len(test_labels)):
                        self.highest_test_accuracy=100*(correct/len(test_labels))           
                        self.create_train_accuracy_for_whole_set(model,train_data_dir,test_probabilities,pdbs_test)
                        
                    if (100*(correct/len(test_labels)))>=85 or ((100*(correct/len(test_labels)))>=self.test_accuracy and self.channels !=17):
                        while_break_correct=True
                        session_break=True#to break the iteration For loop
                    elif  ((100*(correct/len(test_labels)))> 81 and self.channels != 17):
                        if self.SITE_MUST:                           
                            #'''For saving the suboptimal soultion like if the results obtained like 80%'''
                            model_name_81_part=''.join([model_name[0:-3],'_SITE_',self.model_loss,'_',self.optimizer,'_',str(int(100*(correct/len(test_labels))))])
                        else:
                            model_name_81_part=''.join([model_name[0:-3],'_',self.model_loss,'_',self.optimizer,'_',str(int(100*(correct/len(test_labels))))])

                        if self.Full_clean and self.balance_batch_must:
                            model_name_81=''.join([model_name_81_part,'_Full_clean_bal_bat.h5'])
                        elif self.Full_clean:
                            model_name_81=''.join([model_name_81_part,'_Full_clean.h5'])
                        elif self.balance_batch_must:
                            model_name_81=''.join([model_name_81_part,'_bal_bat.h5'])
                        else:
                            model_name_81=''.join([model_name_81_part,'.h5'])

                        model.save(model_name_81)
                        sleep(self.sleep_time)
                    
                    elif  ((100*(correct/len(test_labels)))> 71 and self.channels != 17):
                        if self.SITE_MUST:       
                            #'''For saving the suboptimal soultion like if the results obtained like 80%'''
                            model_name_71_part=''.join([model_name[0:-3],'_SITE_',self.model_loss,'_',self.optimizer,'_',str(int(100*(correct/len(test_labels))))])
                        else:
                            model_name_71_part=''.join([model_name[0:-3],'_',self.model_loss,'_',self.optimizer,'_',str(int(100*(correct/len(test_labels))))])
                           
                        if self.Full_clean and self.balance_batch_must:
                            model_name_71=''.join([model_name_71_part,'_Full_clean_bal_bat.h5'])
                        elif self.Full_clean:
                            model_name_71=''.join([model_name_71_part,'_Full_clean.h5'])
                        elif self.balance_batch_must:
                            model_name_71=''.join([model_name_71_part,'_bal_bat.h5'])
                        else:
                            model_name_71=''.join([model_name_71_part,'.h5'])

                        model.save(model_name_71)
                        sleep(self.sleep_time)
                        
                if validation_cond_loop and itera_chk and (100*(correct/len(test_labels)))<self.test_accuracy:
                    while_break_suboptimal=True
                    session_break=True       
                else:
                    validation_cond_loop=False
                    validation_count_loop=0
        
        self.validation_low_count=validation_low_count
        self.validation_count=validation_count
        self.validation_accuracy_last=validation_accuracy_last
        self.validation_count_loop=validation_count_loop
        self.validation_accuracy_last_1=validation_accuracy_last_1
        
        gc.collect()
        return session_break,while_break_suboptimal,while_break_correct
    
    def test_per_chk(self,model,validation_cond_low,session_break):
        
        test_data_dir= ''.join([self.main_dir,'/Test'])     
        train_data_dir= ''.join([self.main_dir,'/Train'])

        os.chdir('/')
        os.chdir(self.main_dir)
        
        '''To load the test dataset'''
        test_list_IDs = pickle.load(open("test_list_ids.p", "rb"))
        test_labels = pickle.load(open("test_labels_dic.p", "rb"))
        test_generator = Data_test(test_list_IDs, test_labels,batch_size=self.batch_size,n_channels=self.channels)

        print(' ')
        print("Since epach size is 1 one PDB used once per training ")
        print(' ')
        os.chdir('/')
        os.chdir(test_data_dir)
        #% testring
        correct=0
        
        if self.SITE_MUST:
            Fusion_test_size=18
            TSG_test_size=82
        else:
            Fusion_test_size=27
            TSG_test_size=131
        
        list_IDs_temp_all=[]
        list_IDs_temp=[]
        '''check the fusion class'''           
        for itera in range(len(test_labels)-Fusion_test_size,len(test_labels)):
            list_IDs_temp.append(test_list_IDs[itera])
            if len(list_IDs_temp)==self.batch_size:
                list_IDs_temp_all.append(deepcopy(list_IDs_temp))
                list_IDs_temp=[]                
        if len(list_IDs_temp)>0:
           list_IDs_temp_all.append(deepcopy(list_IDs_temp))
#        print('')
#        print('len(list_IDs_temp_all): ',len(list_IDs_temp_all))
#        print('')
        Fusion_test_correct=0
        for list_IDs_temp in list_IDs_temp_all:
            if len(list_IDs_temp)>0:
                x_test, y_test = test_generator.data_generation_test(list_IDs_temp)
                test_scores = model.evaluate(x_test, y_test, verbose=2)
                Fusion_test_correct=Fusion_test_correct+test_scores[1]*(len(list_IDs_temp))
        Fusion_test_acc = Fusion_test_correct/Fusion_test_size

#            print('Test loss    :', test_scores[0])
        print('Fusion test accuracy:', 100*Fusion_test_acc, '%')
        itera_chk=False #if TSG and Fusion not satisfied 
        if validation_cond_low and test_scores[1]<self.fusion_test_chk:
            print("")
            print("Stuck in Local Minimum")
            print("Starting new iteration in Same **Session")
            print("")
            os.chdir('/')
            os.chdir(train_data_dir)
            session_break=True
        else:
            test_acc_tsg=0
            list_IDs_temp_all=[]
            list_IDs_temp=[]
            '''check the TSG class'''
            for itera in range(len(test_labels)-TSG_test_size-Fusion_test_size,len(test_labels)-Fusion_test_size):
                list_IDs_temp.append(test_list_IDs[itera])
                if len(list_IDs_temp)==self.batch_size:
                    list_IDs_temp_all.append(deepcopy(list_IDs_temp))
                    list_IDs_temp=[]                
            if len(list_IDs_temp)>0:
               list_IDs_temp_all.append(deepcopy(list_IDs_temp))

            test_acc_tsg_correct=0
            for list_IDs_temp in list_IDs_temp_all:
                if len(list_IDs_temp)>0:
                    x_test, y_test = test_generator.data_generation_test(list_IDs_temp)
                    test_scores = model.evaluate(x_test, y_test, verbose=2)
                    test_acc_tsg_correct = test_acc_tsg_correct + test_scores[1]*len(list_IDs_temp)
           
            test_acc_tsg = test_acc_tsg_correct/TSG_test_size
            print('TSG test accuracy:', 100*test_acc_tsg, '%')
            if validation_cond_low and test_acc_tsg<self.validation_acc_OG_TSG_limit:
                print("")
                print("Stuck in Local Minimum Due to TSG not satisfied")
                print("Starting new iteration in Same **Session")
                print("")
                os.chdir('/')
                os.chdir(train_data_dir)
                session_break=True
            
        test_probabilities=[]
        pdbs_test=[]
        correct=0   
        if Fusion_test_acc>=self.fusion_test_chk and test_acc_tsg>=self.validation_acc_OG_TSG_limit:
            print('Check the Test set overall performance')
            itera_chk=True#to avoid unnecessary evaluation in test set 
            for itera in range(0,len(test_labels)//self.batch_size):
                x_test, y_test = test_generator.getitem_test(itera)
                test_ids_temp = test_generator.__getitem_list_id__(itera)

                test_scores = model.evaluate(x_test, y_test, verbose=2)
                probabilities = model.predict(x_test)
                test_probabilities.append(deepcopy(probabilities))
                pdbs_test.append(deepcopy(test_ids_temp))
                correct=correct+int(test_scores[1]*self.batch_size)
                print('In iter: ',itera ,'Test acc: ',test_scores[1])

        else:
            if test_acc_tsg<self.validation_acc_OG_TSG_limit:
                print("TSG test set haven't got higher than " ,self.validation_acc_OG_TSG_limit*100 ,"% accuracy so no test check")
                #to avoid unnecessary evaluation in test set 
                print("Back to normal iteration")
            else:
                print("Fusion test set haven't got higher than " ,self.fusion_test_chk*100 ,"% accuracy so no test check")
                #to avoid unnecessary evaluation in test set 
                print("Back to normal iteration")
        
        if itera_chk:
            '''To check the last test batch'''
            list_IDs_temp=[]
            for itera in range((len(test_labels)-len(test_labels)%self.batch_size),len(test_labels)):
                list_IDs_temp.append(test_list_IDs[itera])
            if len(list_IDs_temp)>0:
                x_test, y_test = test_generator.data_generation_test(list_IDs_temp)
                test_scores = model.evaluate(x_test, y_test, verbose=2)
                print('In last iter test acc: ',test_scores[1])

                correct=correct+int(test_scores[1]*(len(list_IDs_temp)))
                probabilities = model.predict(x_test)
                test_probabilities.append(deepcopy(probabilities))
                pdbs_test.append(deepcopy(list_IDs_temp))

            print("")
            print("Overall test accuracy: ",100*correct/len(test_labels))
        return session_break,itera_chk,correct,test_probabilities,pdbs_test
    
    def validation_break_chk(self,valid_scores,itera,valid_correct,valid_acc,break_valid,valid_chk_rest_TSG_OK,valid_chk_rest_ONGO_OK,valid_chk_rest_Fusion_OK):
        '''
        This function used for early stopping in validation evaluation
        '''
        os.chdir('/')
        os.chdir(self.main_dir)
        
        lengths_summery_dic = pickle.load(open("lengths_summery_dic.p", "rb")) 
        valid_OG_len = lengths_summery_dic['valid_OG_len']
        valid_TSG_len = lengths_summery_dic['valid_TSG_len']
        valid_len = lengths_summery_dic['valid_len']
        
        if itera==0 and valid_scores[1]<0.1:
            valid_chk_rest_ONGO_OK=False
            valid_acc=valid_TSG_len/valid_len#since only fit with TSG
            print("Validation since Not fit with ONGO")
            break_valid=True
            
        elif itera==(-(-valid_OG_len//self.batch_size))+1 and valid_scores[1]<0.1:
            valid_chk_rest_TSG_OK=False
            valid_acc=valid_OG_len/valid_len#since only fit with ONGO
            print("Validation since NOt fit with TSG")
            break_valid=True
            
        elif itera== itera==(-(-(valid_OG_len+valid_TSG_len)//self.batch_size))+1 and valid_scores[1]<0.1:
#            valid_chk_rest_Fusion_OK=False#Not fit with Fusion at all
#            valid_acc = valid_correct/valid_len
#            print("Validation since Not fit with Fusion")
#            break_valid=True
            print("Warning Fusion is not checked in Validation")
        train_data_dir= ''.join([self.main_dir,'/Train'])
        os.chdir('/')
        os.chdir(train_data_dir)
        
        return break_valid,valid_chk_rest_ONGO_OK,valid_chk_rest_TSG_OK,valid_chk_rest_Fusion_OK,valid_acc